In [1]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.linear_model import LogisticRegression, ElasticNet, SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, train_test_split, KFold, RandomizedSearchCV, cross_val_score, cross_validate, cross_val_predict
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, balanced_accuracy_score, classification_report
from sklearn.ensemble import VotingClassifier
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier


### reset data

In [2]:
pheno = pd.read_csv("phen.csv", index_col=0)
pheno.drop(["c15x"], axis = 1, inplace=True)
# pheno.drop(["Unnamed: 0"], axis = 1, inplace=True) # I have no idea why that column even exists...
# pheno = pd.DataFrame(pheno) # why this isn't already in a data frame I don't understand...

geno = pd.read_csv("geno_top1000.csv", index_col=0)
cci = pd.read_csv("cci_top0700.csv", index_col=0)


In [3]:
cci

,EN_L2_3_IT_to_IN_ADARB2,EN_L3_5_IT_3_to_IN_ADARB2,EN_L3_5_IT_3_to_IN_VIP,EN_L3_5_IT_3_to_PC,EN_L5_6_NP_to_EN_L3_5_IT_3,EN_L5_6_NP_to_EN_L6_IT_1,EN_L5_6_NP_to_IN_SST,EN_L5_6_NP_to_IN_VIP,EN_L6B_to_Astro,EN_L6B_to_EN_L3_5_IT_2,...,SMC_to_IN_VIP,VLMC_to_EN_L6B,VLMC_to_EN_L6_IT_1,VLMC_to_IN_LAMP5_RELN,EN_L3_5_IT_2_to_Immune,EN_L3_5_IT_3_to_Immune,Immune_to_Astro,Immune_to_IN_ADARB2,Immune_to_IN_SST,SMC_to_EN_NF
donor,,,,,,,,,,,,,,,,,,,,,
M10233,7.769182,9.483466,8.197416,2.448168,9.181288,9.830031,7.641515,6.895149,9.614054,9.483070,...,0.0,11.338024,11.781849,9.927001,0.0,0.0,0.0,0.0,0.0,0.0
M10282,7.982867,7.654124,7.592075,1.294796,5.821084,4.029851,3.713464,5.752696,6.911094,7.173840,...,0.0,7.482350,6.931712,8.847744,0.0,0.0,0.0,0.0,0.0,0.0
M10730,0.000000,0.000000,8.958613,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
M107983,9.082034,9.805885,8.963099,2.723138,0.000000,0.000000,0.000000,0.000000,7.816516,7.365934,...,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
M10886,16.531090,17.652431,17.426262,3.772673,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
M97728,11.732697,11.121620,11.672375,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
M97856,13.184781,12.989894,12.002464,3.958501,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
M97977,7.521141,8.932145,9.128830,1.564973,10.573837,11.208066,7.541481,8.927317,8.948425,8.709168,...,0.0,14.919066,9.820222,5.370577,0.0,0.0,0.0,0.0,0.0,0.0


### info

In [3]:
pos = sum(np.array(pheno["label"] == 1))
neg = sum(np.array(pheno["label"] == 0))
print(pos, "positive cases,", neg, "negative cases")
# So this data is already imbalanced!

378 positive cases, 150 negative cases


In [4]:
seed = 0
X_train_geno, X_test_geno, y_train_geno, y_test_geno = train_test_split(geno, pheno, test_size=0.20, random_state=seed)
X_train_cci, X_test_cci, y_train_cci, y_test_cci = train_test_split(cci, pheno, test_size=0.20, random_state=seed)


In [69]:
type(y_train_geno["label"])

pandas.core.series.Series

# Turn into data frame for MogoNet

In [16]:
X_train_geno.to_csv("1_tr.csv", sep=',', header=False, index=False, encoding='utf-8')
X_test_geno.to_csv("1_te.csv", sep=',', header=False, index=False, encoding='utf-8')

X_train_cci.to_csv("2_tr.csv", sep=',', header=False, index=False, encoding='utf-8')
X_test_cci.to_csv("2_te.csv", sep=',', header=False, index=False, encoding='utf-8')

y_train_geno.to_csv("labels_tr.csv", sep=',', header=False, index=False, encoding='utf-8')
y_test_geno.to_csv("labels_te.csv", sep=',', header=False, index=False, encoding='utf-8')

# Calculating the PPV and NPV

In [15]:
y_pred = cross_val_predict(svm_pipeline3, cdata3, pheno["label"], cv=5)

In [16]:
answers = pheno["label"].to_numpy()

In [17]:
true_pos = 0
false_neg = 0
false_pos = 0
true_neg = 0

for i in range(len(answers)):
    # print(i, answers[i], y_pred[i])
    if (answers[i]==1 and y_pred[i]==1):
        # print("this statement should only print once before the next iteration (1)")
        true_pos += 1
    
    elif (answers[i]==1 and y_pred[i]==0):
        # print("this statement should only print once before the next iteration (2)")
        false_neg += 1
    
    elif (answers[i]==0 and y_pred[i]==1):
        # print("this statement should only print once before the next iteration (3)")
        false_pos += 1
    
    elif (answers[i]==0 and y_pred[i]==0):
        # print("this statement should only print once before the next iteration (4)")
        true_neg += 1
    
    else:
        print("Something went wrong.")
        break
    
print(true_pos, "true positives")
print(false_neg, "false negatives")
print(false_pos, "false positives")
print(true_neg, "true negatives")
print("Sensitivity =", true_pos / (true_pos + false_neg))
print("Specificity =", true_neg / (true_neg + false_pos))
print("PPV =", true_pos / (true_pos + false_pos))
print("NPV =", true_neg / (true_neg + false_neg))

345 true positives
33 false negatives
67 false positives
83 true negatives
Sensitivity = 0.9126984126984127
Specificity = 0.5533333333333333
PPV = 0.837378640776699
NPV = 0.7155172413793104


In [99]:
print(classification_report(pheno["label"], y_pred))

              precision    recall  f1-score   support

           0       0.58      0.49      0.53       150
           1       0.81      0.86      0.83       378

    accuracy                           0.75       528
   macro avg       0.69      0.68      0.68       528
weighted avg       0.74      0.75      0.75       528



In [94]:
70/119

0.5882352941176471

## 1. kNN

In [54]:
knn_pipeline = Pipeline(steps = [("scale", StandardScaler()),
                                 ("pca", PCA(n_components=20)),
                                 ("knn", KNeighborsClassifier(n_neighbors=7))])


In [10]:
geno["chr1:5284719:T:C"].isna().sum()

0

In [11]:
KNeighborsClassifier().fit(X_train_geno, y_train_geno['label'])

KNeighborsClassifier()

In [12]:
knn_pipeline.fit(X_train_geno, y_train_geno['label'])
#predictions = knn_pipeline.predict(X_test_geno)

Pipeline(steps=[('scale', StandardScaler()), ('pca', PCA()),
                ('knn', KNeighborsClassifier())])

In [13]:
import sklearn
sklearn.__version__

'1.2.1'

y_train_geno

### 5-fold CV

In [16]:
geno.isna().sum().value_counts()

0    6314
dtype: int64

In [18]:
pheno["label"].value_counts()

1    378
0    150
Name: label, dtype: int64

In [52]:
knn_scores = cross_validate(knn_pipeline, geno, pheno["label"], cv=5, scoring=["accuracy",
                                                                             "roc_auc",
                                                                             "f1",
                                                                             "balanced_accuracy"])
knn_scores

{'fit_time': array([6.83509302, 8.13026309, 7.68731523, 7.73508692, 7.69154668]),
 'score_time': array([0.185776  , 0.20511198, 0.17073107, 0.22175097, 0.19497609]),
 'test_accuracy': array([0.73584906, 0.68867925, 0.68867925, 0.65714286, 0.75238095]),
 'test_roc_auc': array([0.64254386, 0.62763158, 0.59188596, 0.61666667, 0.73066667]),
 'test_f1': array([0.83908046, 0.81142857, 0.80701754, 0.78313253, 0.85227273]),
 'test_balanced_accuracy': array([0.56359649, 0.5004386 , 0.52061404, 0.5       , 0.56666667])}

In [53]:
print("ACC")
print("mean =", np.mean(knn_scores["test_accuracy"]))
print("std =", np.std(knn_scores["test_accuracy"]), "\n")
print("AUC")
print("mean =", np.mean(knn_scores["test_roc_auc"]))
print("std =", np.std(knn_scores["test_roc_auc"]), "\n")
print("F1")
print("mean =", np.mean(knn_scores["test_f1"]))
print("std =", np.std(knn_scores["test_f1"]), "\n")
print("BAS")
print("mean =", np.mean(knn_scores["test_balanced_accuracy"]))
print("std =", np.std(knn_scores["test_balanced_accuracy"]), "\n")

ACC
mean = 0.7045462713387242
std = 0.03469475402677266 

AUC
mean = 0.641878947368421
std = 0.04737435193539712 

F1
mean = 0.818586366490309
std = 0.02448193196080986 

BAS
mean = 0.5302631578947368
std = 0.029444168596302476 



In [46]:
grid_search_knn = GridSearchCV(estimator=knn_pipeline,
                               param_grid={"knn__weights": ["uniform", "distance"],
                                           "knn__algorithm": ["auto", "ball_tree", "kd_tree", "brute"]},
                               scoring="balanced_accuracy", # scoring can be "accuracy", "roc_auc", "f1"
                               return_train_score=True, # or else you'll never see it!
                               cv=KFold(n_splits=5, shuffle=True, random_state=0),
                               verbose = 3) # to get a consistent set of settings


In [47]:
grid_search_knn.fit(geno, pheno["label"])
grid_search_knn.best_params_

Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV 1/5] END knn__algorithm=auto, knn__weights=uniform;, score=(train=0.641, test=0.497) total time=   5.7s
[CV 2/5] END knn__algorithm=auto, knn__weights=uniform;, score=(train=0.607, test=0.549) total time=   6.0s
[CV 3/5] END knn__algorithm=auto, knn__weights=uniform;, score=(train=0.627, test=0.559) total time=   6.1s
[CV 4/5] END knn__algorithm=auto, knn__weights=uniform;, score=(train=0.633, test=0.555) total time=   6.2s
[CV 5/5] END knn__algorithm=auto, knn__weights=uniform;, score=(train=0.619, test=0.549) total time=   5.9s
[CV 1/5] END knn__algorithm=auto, knn__weights=distance;, score=(train=1.000, test=0.513) total time=   5.9s
[CV 2/5] END knn__algorithm=auto, knn__weights=distance;, score=(train=1.000, test=0.575) total time=   6.2s
[CV 3/5] END knn__algorithm=auto, knn__weights=distance;, score=(train=1.000, test=0.553) total time=   5.8s
[CV 4/5] END knn__algorithm=auto, knn__weights=distance;, score=(train=1.

{'knn__algorithm': 'ball_tree', 'knn__weights': 'uniform'}

- INITIAL
    - ACC = 0.734
    - AUC = 0.598
    - F1 = 0.835

- add MinMaxScaling (StandardScaling)
    - ACC = 0.734 (0.750)
    - AUC = 0.598 (0.622)
    - F1 = 0.835 (0.846)
    - GO WITH STANDARDSCALING
    
- add PCA with 10 components
    - ACC = 0.719
    - AUC = 0.558
    - F1 = 0.827
    
- add better parameters
{'knn__algorithm': 'ball_tree',
 'knn__weights': 'distance',
 'pca__n_components': 10}
    - ACC = 0.703
    - AUC = 0.539
    - F1 = 0.816
    
- factor out PCA
    - ACC = 0.750
    - AUC = 0.622
    - F1 = 0.846

## 2. Logistic Regression

In [105]:
lr_pipeline = Pipeline(steps = [("pca", PCA(n_components=80)),
                                ("logistic_regression", LogisticRegression(max_iter=2000))
                               ])


In [91]:
grid_search_lr = GridSearchCV(estimator=lr_pipeline,
                              param_grid={"logistic_regression__C": [0.01, 0.05, 0.1, 0.25, 0.5, 1, 5, 10, 20],
                                          "logistic_regression__l1_ratio": [0, 0.25, 0.50, 0.75, 1],
                                          "logistic_regression__solver": ["lbfgs", "saga"]},
                              scoring="balanced_accuracy", # scoring can be "accuracy", "roc_auc", "f1"
                              return_train_score=True, # or else you'll never see it!
                              cv=KFold(n_splits=5, shuffle=True, random_state=0),
                              verbose = 3) # to get a consistent set of settings



In [92]:
grid_search_lr.fit(geno, pheno["label"])
grid_search_lr.best_params_

Fitting 5 folds for each of 90 candidates, totalling 450 fits


/Users/ryanburczak/anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(


[CV 1/5] END logistic_regression__C=0.01, logistic_regression__l1_ratio=0, logistic_regression__solver=lbfgs;, score=(train=0.804, test=0.598) total time=   5.4s


/Users/ryanburczak/anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(


[CV 2/5] END logistic_regression__C=0.01, logistic_regression__l1_ratio=0, logistic_regression__solver=lbfgs;, score=(train=0.785, test=0.643) total time=   4.9s


...

[CV 4/5] END logistic_regression__C=20, logistic_regression__l1_ratio=1, logistic_regression__solver=saga;, score=(train=0.843, test=0.597) total time=   5.6s


/Users/ryanburczak/anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(


[CV 5/5] END logistic_regression__C=20, logistic_regression__l1_ratio=1, logistic_regression__solver=saga;, score=(train=0.826, test=0.725) total time=   4.9s


/Users/ryanburczak/anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(


{'logistic_regression__C': 10,
 'logistic_regression__l1_ratio': 0,
 'logistic_regression__solver': 'lbfgs'}

### 5-fold CV

In [94]:
lr_scores = cross_validate(lr_pipeline, geno, pheno["label"], cv=5, scoring=["accuracy",
                                                                             "roc_auc",
                                                                             "f1",
                                                                             "balanced_accuracy"])
lr_scores


/Users/ryanburczak/anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/Users/ryanburczak/anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/Users/ryanburczak/anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/Users/ryanburczak/anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/Users/ryanburczak/anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'.

{'fit_time': array([5.20199108, 5.29056883, 5.17891884, 4.99698997, 5.24181294]),
 'score_time': array([0.06211686, 0.05378938, 0.05788898, 0.05834508, 0.05470586]),
 'test_accuracy': array([0.67924528, 0.73584906, 0.75471698, 0.75238095, 0.78095238]),
 'test_roc_auc': array([0.66622807, 0.77982456, 0.74254386, 0.80711111, 0.80133333]),
 'test_f1': array([0.78481013, 0.82051282, 0.8375    , 0.82432432, 0.84768212]),
 'test_balanced_accuracy': array([0.5745614 , 0.65438596, 0.65745614, 0.70666667, 0.72666667])}

In [95]:
print("ACC")
print("mean =", np.mean(lr_scores["test_accuracy"]))
print("std =", np.std(lr_scores["test_accuracy"]), "\n")
print("AUC")
print("mean =", np.mean(lr_scores["test_roc_auc"]))
print("std =", np.std(lr_scores["test_roc_auc"]), "\n")
print("F1")
print("mean =", np.mean(lr_scores["test_f1"]))
print("std =", np.std(lr_scores["test_f1"]), "\n")
print("BAS")
print("mean =", np.mean(lr_scores["test_balanced_accuracy"]))
print("std =", np.std(lr_scores["test_balanced_accuracy"]), "\n")

ACC
mean = 0.74062893081761
std = 0.033921566054177545 

AUC
mean = 0.759408187134503
std = 0.05180165039077913 

F1
mean = 0.8229658781249443
std = 0.021381569237695283 

BAS
mean = 0.6639473684210526
std = 0.05269166161213 



## 3. Support Vector Machine

In [85]:
svm_pipeline = Pipeline(steps = [("scale", MinMaxScaler()),
                                 ("pca", PCA(n_components=50)),
                                 ("svm", SVC(C=20, kernel='linear', gamma='scale'))])


### 5-fold CV

In [63]:
# SAVE! This is for 50 SNPs
svm_scores = cross_validate(svm_pipeline, geno, pheno["label"], cv=5, scoring=["accuracy",
                                                                             "roc_auc",
                                                                             "f1",
                                                                             "balanced_accuracy"])
svm_scores


{'fit_time': array([0.08540702, 0.04142284, 0.04720497, 0.05819273, 0.05846095]),
 'score_time': array([0.00499392, 0.00351906, 0.00341892, 0.00327229, 0.00334287]),
 'test_accuracy': array([0.77358491, 0.72641509, 0.71698113, 0.78095238, 0.82857143]),
 'test_roc_auc': array([0.77719298, 0.69561404, 0.68377193, 0.78044444, 0.84355556]),
 'test_f1': array([0.85      , 0.80794702, 0.80769231, 0.86060606, 0.8875    ]),
 'test_balanced_accuracy': array([0.68070175, 0.66798246, 0.63114035, 0.65666667, 0.74      ])}

In [42]:
# SAVE! This is for 500 SNPs
svm_scores = cross_validate(svm_pipeline, geno, pheno["label"], cv=5, scoring=["accuracy",
                                                                             "roc_auc",
                                                                             "f1",
                                                                             "balanced_accuracy"])
svm_scores


{'fit_time': array([1.06979394, 0.83294487, 0.84188914, 0.60053778, 0.56945992]),
 'score_time': array([0.07360315, 0.00874686, 0.05008984, 0.02363706, 0.08029103]),
 'test_accuracy': array([0.76415094, 0.82075472, 0.81132075, 0.82857143, 0.81904762]),
 'test_roc_auc': array([0.78684211, 0.87236842, 0.88815789, 0.9       , 0.90844444]),
 'test_f1': array([0.84076433, 0.87581699, 0.85714286, 0.87837838, 0.87741935]),
 'test_balanced_accuracy': array([0.68421053, 0.77412281, 0.82807018, 0.8       , 0.75333333])}

In [49]:
# Save! This is for 1500 SNPs
svm_scores = cross_validate(svm_pipeline, geno, pheno["label"], cv=5, scoring=["accuracy",
                                                                             "roc_auc",
                                                                             "f1",
                                                                             "balanced_accuracy"])
svm_scores


{'fit_time': array([5.6289289 , 5.72625113, 5.93933892, 4.98739314, 4.63046288]),
 'score_time': array([0.01447606, 0.01482987, 0.01452017, 0.41652894, 0.0288651 ]),
 'test_accuracy': array([0.79245283, 0.75471698, 0.83962264, 0.84761905, 0.8       ]),
 'test_roc_auc': array([0.78421053, 0.85131579, 0.88157895, 0.92266667, 0.90044444]),
 'test_f1': array([0.8625    , 0.84146341, 0.88275862, 0.8974359 , 0.86624204]),
 'test_balanced_accuracy': array([0.70394737, 0.6372807 , 0.8377193 , 0.78333333, 0.72      ])}

In [56]:
# Save! This is for 200 SNPs
svm_scores = cross_validate(svm_pipeline, geno, pheno["label"], cv=5, scoring=["accuracy",
                                                                             "roc_auc",
                                                                             "f1",
                                                                             "balanced_accuracy"])
svm_scores


{'fit_time': array([0.72332907, 0.64281297, 0.71711397, 0.4096477 , 0.57874203]),
 'score_time': array([0.00746107, 0.00827527, 0.02792406, 0.15670609, 0.03943706]),
 'test_accuracy': array([0.78301887, 0.79245283, 0.77358491, 0.75238095, 0.83809524]),
 'test_roc_auc': array([0.84561404, 0.85877193, 0.79912281, 0.80755556, 0.83422222]),
 'test_f1': array([0.8516129 , 0.85897436, 0.84415584, 0.83544304, 0.89171975]),
 'test_balanced_accuracy': array([0.71754386, 0.72412281, 0.71096491, 0.65666667, 0.76666667])}

In [64]:
print("ACC")
print("mean =", np.mean(svm_scores["test_accuracy"]))
print("std =", np.std(svm_scores["test_accuracy"]), "\n")
print("AUC")
print("mean =", np.mean(svm_scores["test_roc_auc"]))
print("std =", np.std(svm_scores["test_roc_auc"]), "\n")
print("F1")
print("mean =", np.mean(svm_scores["test_f1"]))
print("std =", np.std(svm_scores["test_f1"]), "\n")
print("BAS")
print("mean =", np.mean(svm_scores["test_balanced_accuracy"]))
print("std =", np.std(svm_scores["test_balanced_accuracy"]), "\n")

ACC
mean = 0.7653009883198563
std = 0.0404070767207671 

AUC
mean = 0.7561157894736843
std = 0.059289021786280094 

F1
mean = 0.8427490776331836
std = 0.03102983439257796 

BAS
mean = 0.6752982456140352
std = 0.03623835290118846 



In [141]:
grid_search_svm = GridSearchCV(estimator=svm_pipeline,
                              param_grid={"svm__C": [0.01, 0.05, 0.1, 0.25, 0.5, 1, 5, 10, 20],
                                          "svm__kernel": ["linear", "poly", "rbf", "sigmoid"],
                                          "svm__gamma": ["scale", "auto"]},
                              scoring="balanced_accuracy", # scoring can be "accuracy", "roc_auc", "f1"
                              return_train_score=True, # or else you'll never see it!
                              cv=KFold(n_splits=5, shuffle=True, random_state=0),
                              verbose = 3) # to get a consistent set of settings)


In [142]:
grid_search_svm.fit(geno, pheno["label"])
grid_search_svm.best_params_

Fitting 5 folds for each of 72 candidates, totalling 360 fits
[CV 1/5] END svm__C=0.01, svm__gamma=scale, svm__kernel=linear;, score=(train=0.755, test=0.522) total time=   6.0s
[CV 2/5] END svm__C=0.01, svm__gamma=scale, svm__kernel=linear;, score=(train=0.742, test=0.645) total time=   5.3s
[CV 3/5] END svm__C=0.01, svm__gamma=scale, svm__kernel=linear;, score=(train=0.708, test=0.721) total time=   5.3s
[CV 4/5] END svm__C=0.01, svm__gamma=scale, svm__kernel=linear;, score=(train=0.744, test=0.603) total time=   5.1s
[CV 5/5] END svm__C=0.01, svm__gamma=scale, svm__kernel=linear;, score=(train=0.728, test=0.656) total time=   5.2s
[CV 1/5] END svm__C=0.01, svm__gamma=scale, svm__kernel=poly;, score=(train=0.500, test=0.500) total time=   5.5s
[CV 2/5] END svm__C=0.01, svm__gamma=scale, svm__kernel=poly;, score=(train=0.500, test=0.500) total time=   5.0s
[CV 3/5] END svm__C=0.01, svm__gamma=scale, svm__kernel=poly;, score=(train=0.500, test=0.500) total time=   5.3s
[CV 4/5] END svm

[CV 2/5] END svm__C=0.05, svm__gamma=auto, svm__kernel=rbf;, score=(train=0.500, test=0.500) total time=   5.4s
[CV 3/5] END svm__C=0.05, svm__gamma=auto, svm__kernel=rbf;, score=(train=0.500, test=0.500) total time=   5.3s
[CV 4/5] END svm__C=0.05, svm__gamma=auto, svm__kernel=rbf;, score=(train=0.500, test=0.500) total time=   5.5s
[CV 5/5] END svm__C=0.05, svm__gamma=auto, svm__kernel=rbf;, score=(train=0.500, test=0.500) total time=   5.3s
[CV 1/5] END svm__C=0.05, svm__gamma=auto, svm__kernel=sigmoid;, score=(train=0.527, test=0.535) total time=   4.9s
[CV 2/5] END svm__C=0.05, svm__gamma=auto, svm__kernel=sigmoid;, score=(train=0.509, test=0.513) total time=   4.9s
[CV 3/5] END svm__C=0.05, svm__gamma=auto, svm__kernel=sigmoid;, score=(train=0.516, test=0.570) total time=   5.3s
[CV 4/5] END svm__C=0.05, svm__gamma=auto, svm__kernel=sigmoid;, score=(train=0.546, test=0.555) total time=   5.6s
[CV 5/5] END svm__C=0.05, svm__gamma=auto, svm__kernel=sigmoid;, score=(train=0.519, tes

[CV 4/5] END svm__C=0.25, svm__gamma=auto, svm__kernel=linear;, score=(train=0.746, test=0.697) total time=   4.7s
[CV 5/5] END svm__C=0.25, svm__gamma=auto, svm__kernel=linear;, score=(train=0.754, test=0.637) total time=   5.3s
[CV 1/5] END svm__C=0.25, svm__gamma=auto, svm__kernel=poly;, score=(train=1.000, test=0.495) total time=   4.9s
[CV 2/5] END svm__C=0.25, svm__gamma=auto, svm__kernel=poly;, score=(train=1.000, test=0.536) total time=   5.0s
[CV 3/5] END svm__C=0.25, svm__gamma=auto, svm__kernel=poly;, score=(train=1.000, test=0.559) total time=   5.0s
[CV 4/5] END svm__C=0.25, svm__gamma=auto, svm__kernel=poly;, score=(train=1.000, test=0.531) total time=   5.2s
[CV 5/5] END svm__C=0.25, svm__gamma=auto, svm__kernel=poly;, score=(train=1.000, test=0.561) total time=   5.3s
[CV 1/5] END svm__C=0.25, svm__gamma=auto, svm__kernel=rbf;, score=(train=0.500, test=0.500) total time=   5.3s
[CV 2/5] END svm__C=0.25, svm__gamma=auto, svm__kernel=rbf;, score=(train=0.500, test=0.500) 

[CV 2/5] END svm__C=1, svm__gamma=scale, svm__kernel=sigmoid;, score=(train=0.572, test=0.512) total time=   4.4s
[CV 3/5] END svm__C=1, svm__gamma=scale, svm__kernel=sigmoid;, score=(train=0.543, test=0.665) total time=   4.0s
[CV 4/5] END svm__C=1, svm__gamma=scale, svm__kernel=sigmoid;, score=(train=0.608, test=0.555) total time=   5.3s
[CV 5/5] END svm__C=1, svm__gamma=scale, svm__kernel=sigmoid;, score=(train=0.575, test=0.489) total time=   4.9s
[CV 1/5] END svm__C=1, svm__gamma=auto, svm__kernel=linear;, score=(train=0.767, test=0.540) total time=   5.2s
[CV 2/5] END svm__C=1, svm__gamma=auto, svm__kernel=linear;, score=(train=0.738, test=0.623) total time=   5.7s
[CV 3/5] END svm__C=1, svm__gamma=auto, svm__kernel=linear;, score=(train=0.723, test=0.651) total time=   5.3s
[CV 4/5] END svm__C=1, svm__gamma=auto, svm__kernel=linear;, score=(train=0.751, test=0.562) total time=   5.2s
[CV 5/5] END svm__C=1, svm__gamma=auto, svm__kernel=linear;, score=(train=0.750, test=0.617) tot

[CV 1/5] END svm__C=10, svm__gamma=scale, svm__kernel=rbf;, score=(train=1.000, test=0.497) total time=   4.5s
[CV 2/5] END svm__C=10, svm__gamma=scale, svm__kernel=rbf;, score=(train=1.000, test=0.581) total time=   4.9s
[CV 3/5] END svm__C=10, svm__gamma=scale, svm__kernel=rbf;, score=(train=1.000, test=0.639) total time=   5.0s
[CV 4/5] END svm__C=10, svm__gamma=scale, svm__kernel=rbf;, score=(train=1.000, test=0.627) total time=   4.6s
[CV 5/5] END svm__C=10, svm__gamma=scale, svm__kernel=rbf;, score=(train=1.000, test=0.650) total time=   4.8s
[CV 1/5] END svm__C=10, svm__gamma=scale, svm__kernel=sigmoid;, score=(train=0.560, test=0.515) total time=   4.5s
[CV 2/5] END svm__C=10, svm__gamma=scale, svm__kernel=sigmoid;, score=(train=0.571, test=0.557) total time=   4.6s
[CV 3/5] END svm__C=10, svm__gamma=scale, svm__kernel=sigmoid;, score=(train=0.508, test=0.642) total time=   4.8s
[CV 4/5] END svm__C=10, svm__gamma=scale, svm__kernel=sigmoid;, score=(train=0.548, test=0.515) tota

{'svm__C': 20, 'svm__gamma': 'scale', 'svm__kernel': 'linear'}

## 4. Stochastic Gradient Descent

In [43]:
sgd_pipeline = Pipeline(steps = [("sgd", SGDClassifier(max_iter = 1000,
                                                      alpha=0.0001,
                                                      l1_ratio=0.5,
                                                      penalty=None))])

### 5-fold CV

In [44]:
sgd_scores = cross_validate(sgd_pipeline, geno, pheno["label"], cv=5, scoring=["accuracy",
                                                                             "roc_auc",
                                                                             "f1",
                                                                             "balanced_accuracy"])
sgd_scores


{'fit_time': array([0.09930086, 0.08293295, 0.08175492, 0.08608198, 0.09544492]),
 'score_time': array([0.04435706, 0.05913091, 0.04528618, 0.03732896, 0.03986406]),
 'test_accuracy': array([0.71698113, 0.78301887, 0.76415094, 0.8       , 0.76190476]),
 'test_roc_auc': array([0.72631579, 0.80175439, 0.79078947, 0.83866667, 0.79288889]),
 'test_f1': array([0.8       , 0.85889571, 0.83443709, 0.86451613, 0.83221477]),
 'test_balanced_accuracy': array([0.66140351, 0.67719298, 0.71447368, 0.73      , 0.71333333])}

In [45]:
print("ACC")
print("mean =", np.mean(sgd_scores["test_accuracy"]))
print("std =", np.std(sgd_scores["test_accuracy"]), "\n")
print("AUC")
print("mean =", np.mean(sgd_scores["test_roc_auc"]))
print("std =", np.std(sgd_scores["test_roc_auc"]), "\n")
print("F1")
print("mean =", np.mean(sgd_scores["test_f1"]))
print("std =", np.std(sgd_scores["test_f1"]), "\n")
print("BAS")
print("mean =", np.mean(sgd_scores["test_balanced_accuracy"]))
print("std =", np.std(sgd_scores["test_balanced_accuracy"]), "\n")

ACC
mean = 0.7652111410601977
std = 0.027804926798252912 

AUC
mean = 0.7900830409356725
std = 0.03625251211417524 

F1
mean = 0.8380127371494233
std = 0.022934245552324997 

BAS
mean = 0.699280701754386
std = 0.025669154824175713 



In [41]:
grid_search_sgd = GridSearchCV(estimator=sgd_pipeline,
                              param_grid={"sgd__penalty": ["elasticnet", None],
                                          "sgd__alpha": [0.0001, 0.001, 0.01, 0.05, 0.1, 0.25, 0.5, 1, 5, 10],
                                          "sgd__l1_ratio": [0, 0.25, 0.50, 0.75, 1]},
                              scoring="balanced_accuracy", # scoring can be "accuracy", "roc_auc", "f1"
                              return_train_score=True, # or else you'll never see it!
                              cv=KFold(n_splits=5, shuffle=True, random_state=0),
                              verbose = 3) # to get a consistent set of settings)


In [42]:
grid_search_sgd.fit(geno, pheno["label"])
grid_search_sgd.best_params_

Fitting 5 folds for each of 100 candidates, totalling 500 fits
[CV 1/5] END sgd__alpha=0.0001, sgd__l1_ratio=0, sgd__penalty=elasticnet;, score=(train=1.000, test=0.667) total time=   0.2s
[CV 2/5] END sgd__alpha=0.0001, sgd__l1_ratio=0, sgd__penalty=elasticnet;, score=(train=1.000, test=0.664) total time=   0.3s
[CV 3/5] END sgd__alpha=0.0001, sgd__l1_ratio=0, sgd__penalty=elasticnet;, score=(train=1.000, test=0.672) total time=   0.2s
[CV 4/5] END sgd__alpha=0.0001, sgd__l1_ratio=0, sgd__penalty=elasticnet;, score=(train=1.000, test=0.597) total time=   0.2s
[CV 5/5] END sgd__alpha=0.0001, sgd__l1_ratio=0, sgd__penalty=elasticnet;, score=(train=0.986, test=0.656) total time=   0.2s
[CV 1/5] END sgd__alpha=0.0001, sgd__l1_ratio=0, sgd__penalty=None;, score=(train=1.000, test=0.667) total time=   0.1s
[CV 2/5] END sgd__alpha=0.0001, sgd__l1_ratio=0, sgd__penalty=None;, score=(train=1.000, test=0.671) total time=   0.1s
[CV 3/5] END sgd__alpha=0.0001, sgd__l1_ratio=0, sgd__penalty=None;

[CV 3/5] END sgd__alpha=0.001, sgd__l1_ratio=0.25, sgd__penalty=None;, score=(train=1.000, test=0.636) total time=   0.1s
[CV 4/5] END sgd__alpha=0.001, sgd__l1_ratio=0.25, sgd__penalty=None;, score=(train=1.000, test=0.716) total time=   0.1s
[CV 5/5] END sgd__alpha=0.001, sgd__l1_ratio=0.25, sgd__penalty=None;, score=(train=1.000, test=0.686) total time=   0.1s
[CV 1/5] END sgd__alpha=0.001, sgd__l1_ratio=0.5, sgd__penalty=elasticnet;, score=(train=1.000, test=0.714) total time=   0.2s
[CV 2/5] END sgd__alpha=0.001, sgd__l1_ratio=0.5, sgd__penalty=elasticnet;, score=(train=1.000, test=0.689) total time=   0.2s
[CV 3/5] END sgd__alpha=0.001, sgd__l1_ratio=0.5, sgd__penalty=elasticnet;, score=(train=1.000, test=0.704) total time=   0.2s
[CV 4/5] END sgd__alpha=0.001, sgd__l1_ratio=0.5, sgd__penalty=elasticnet;, score=(train=1.000, test=0.645) total time=   0.2s
[CV 5/5] END sgd__alpha=0.001, sgd__l1_ratio=0.5, sgd__penalty=elasticnet;, score=(train=1.000, test=0.712) total time=   0.2s

[CV 5/5] END sgd__alpha=0.01, sgd__l1_ratio=0.75, sgd__penalty=elasticnet;, score=(train=0.984, test=0.759) total time=   0.2s


/Users/ryanburczak/anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_stochastic_gradient.py:702: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(


[CV 1/5] END sgd__alpha=0.01, sgd__l1_ratio=0.75, sgd__penalty=None;, score=(train=0.992, test=0.675) total time=   2.6s
[CV 2/5] END sgd__alpha=0.01, sgd__l1_ratio=0.75, sgd__penalty=None;, score=(train=1.000, test=0.698) total time=   1.1s
[CV 3/5] END sgd__alpha=0.01, sgd__l1_ratio=0.75, sgd__penalty=None;, score=(train=1.000, test=0.668) total time=   0.6s
[CV 4/5] END sgd__alpha=0.01, sgd__l1_ratio=0.75, sgd__penalty=None;, score=(train=1.000, test=0.614) total time=   0.7s
[CV 5/5] END sgd__alpha=0.01, sgd__l1_ratio=0.75, sgd__penalty=None;, score=(train=1.000, test=0.695) total time=   0.5s
[CV 1/5] END sgd__alpha=0.01, sgd__l1_ratio=1, sgd__penalty=elasticnet;, score=(train=0.987, test=0.626) total time=   0.5s
[CV 2/5] END sgd__alpha=0.01, sgd__l1_ratio=1, sgd__penalty=elasticnet;, score=(train=0.998, test=0.692) total time=   0.7s
[CV 3/5] END sgd__alpha=0.01, sgd__l1_ratio=1, sgd__penalty=elasticnet;, score=(train=0.985, test=0.654) total time=   0.3s
[CV 4/5] END sgd__alpha

...

[CV 2/5] END sgd__alpha=10, sgd__l1_ratio=1, sgd__penalty=None;, score=(train=0.500, test=0.513) total time=   3.5s
[CV 3/5] END sgd__alpha=10, sgd__l1_ratio=1, sgd__penalty=None;, score=(train=0.583, test=0.709) total time=   2.9s


/Users/ryanburczak/anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_stochastic_gradient.py:702: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(


[CV 4/5] END sgd__alpha=10, sgd__l1_ratio=1, sgd__penalty=None;, score=(train=0.677, test=0.693) total time=   3.6s
[CV 5/5] END sgd__alpha=10, sgd__l1_ratio=1, sgd__penalty=None;, score=(train=0.498, test=0.500) total time=   3.6s


/Users/ryanburczak/anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_stochastic_gradient.py:702: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(


{'sgd__alpha': 0.0001, 'sgd__l1_ratio': 0.5, 'sgd__penalty': None}

In [ ]:
sgd_scores = cross_validate(sgd_pipeline, geno, pheno["label"], cv=5, scoring=["accuracy",
                                                                             "roc_auc",
                                                                             "f1",
                                                                             "balanced_accuracy"])
sgd_scores


In [ ]:
print("ACC")
print("mean =", np.mean(sgd_scores["test_accuracy"]))
print("std =", np.std(sgd_scores["test_accuracy"]), "\n")
print("AUC")
print("mean =", np.mean(sgd_scores["test_roc_auc"]))
print("std =", np.std(sgd_scores["test_roc_auc"]), "\n")
print("F1")
print("mean =", np.mean(sgd_scores["test_f1"]))
print("std =", np.std(sgd_scores["test_f1"]), "\n")
print("BAS")
print("mean =", np.mean(sgd_scores["test_balanced_accuracy"]))
print("std =", np.std(sgd_scores["test_balanced_accuracy"]), "\n")

## 5. XGBoost

In [9]:
xgb_pipeline = Pipeline(steps = [("scale", StandardScaler()),
                                 ("pca", PCA(n_components=100)),
                                 ("xgb", XGBClassifier(max_depth=2,
                                                      reg_alpha=1,
                                                      reg_lambda=0.5))])


### 5-fold CV

In [10]:
xgb_scores = cross_validate(xgb_pipeline, geno, pheno["label"], cv=5, scoring=["accuracy",
                                                                             "roc_auc",
                                                                             "f1",
                                                                             "balanced_accuracy"])
xgb_scores



{'fit_time': array([5.37190318, 5.19544816, 5.01101804, 5.22928214, 5.24353504]),
 'score_time': array([0.07609987, 0.05981302, 0.07812405, 0.06719375, 0.20496607]),
 'test_accuracy': array([0.71698113, 0.79245283, 0.78301887, 0.79047619, 0.76190476]),
 'test_roc_auc': array([0.70789474, 0.7504386 , 0.81447368, 0.77155556, 0.784     ]),
 'test_f1': array([0.81481481, 0.86904762, 0.85714286, 0.86419753, 0.8502994 ]),
 'test_balanced_accuracy': array([0.60087719, 0.66359649, 0.6872807 , 0.68333333, 0.62333333])}

In [11]:
print("ACC")
print("mean =", np.mean(xgb_scores["test_accuracy"]))
print("std =", np.std(xgb_scores["test_accuracy"]), "\n")
print("AUC")
print("mean =", np.mean(xgb_scores["test_roc_auc"]))
print("std =", np.std(xgb_scores["test_roc_auc"]), "\n")
print("F1")
print("mean =", np.mean(xgb_scores["test_f1"]))
print("std =", np.std(xgb_scores["test_f1"]), "\n")
print("BAS")
print("mean =", np.mean(xgb_scores["test_balanced_accuracy"]))
print("std =", np.std(xgb_scores["test_balanced_accuracy"]), "\n")

ACC
mean = 0.7689667565139263
std = 0.028157024548928537 

AUC
mean = 0.7656725146198831
std = 0.03555954136631335 

F1
mean = 0.8511004446134185
std = 0.019222039891974105 

BAS
mean = 0.6516842105263156
std = 0.0340464531575164 



In [7]:
grid_search_xgb = GridSearchCV(estimator=xgb_pipeline,
                               param_grid={"xgb__max_depth": [2, 4, 6, 8, 10],
                                           "xgb__reg_alpha": [0.1, 0.25, 0.5, 1, 5, 10],
                                           "xgb__reg_lambda": [0.1, 0.25, 0.5, 1, 5, 10]},
                               scoring="balanced_accuracy",
                               return_train_score=True, # or else you'll never see it!
                               cv=KFold(n_splits=5, shuffle=True, random_state=0),
                               verbose = 3) # to get a consistent set of settings


In [8]:
grid_search_xgb.fit(geno, pheno["label"])
grid_search_xgb.best_params_

Fitting 5 folds for each of 180 candidates, totalling 900 fits
[CV 1/5] END xgb__max_depth=2, xgb__reg_alpha=0.1, xgb__reg_lambda=0.1;, score=(train=1.000, test=0.614) total time=   5.1s
[CV 2/5] END xgb__max_depth=2, xgb__reg_alpha=0.1, xgb__reg_lambda=0.1;, score=(train=1.000, test=0.591) total time=   5.1s
[CV 3/5] END xgb__max_depth=2, xgb__reg_alpha=0.1, xgb__reg_lambda=0.1;, score=(train=1.000, test=0.569) total time=   5.2s
[CV 4/5] END xgb__max_depth=2, xgb__reg_alpha=0.1, xgb__reg_lambda=0.1;, score=(train=1.000, test=0.708) total time=   4.8s
[CV 5/5] END xgb__max_depth=2, xgb__reg_alpha=0.1, xgb__reg_lambda=0.1;, score=(train=1.000, test=0.635) total time=   5.3s
[CV 1/5] END xgb__max_depth=2, xgb__reg_alpha=0.1, xgb__reg_lambda=0.25;, score=(train=1.000, test=0.529) total time=   4.7s
[CV 2/5] END xgb__max_depth=2, xgb__reg_alpha=0.1, xgb__reg_lambda=0.25;, score=(train=1.000, test=0.635) total time=   5.0s
[CV 3/5] END xgb__max_depth=2, xgb__reg_alpha=0.1, xgb__reg_lambda=

[CV 2/5] END xgb__max_depth=2, xgb__reg_alpha=0.5, xgb__reg_lambda=0.25;, score=(train=1.000, test=0.581) total time=   4.7s
[CV 3/5] END xgb__max_depth=2, xgb__reg_alpha=0.5, xgb__reg_lambda=0.25;, score=(train=1.000, test=0.529) total time=   5.1s
[CV 4/5] END xgb__max_depth=2, xgb__reg_alpha=0.5, xgb__reg_lambda=0.25;, score=(train=1.000, test=0.697) total time=   5.1s
[CV 5/5] END xgb__max_depth=2, xgb__reg_alpha=0.5, xgb__reg_lambda=0.25;, score=(train=1.000, test=0.596) total time=   4.9s
[CV 1/5] END xgb__max_depth=2, xgb__reg_alpha=0.5, xgb__reg_lambda=0.5;, score=(train=1.000, test=0.560) total time=   5.0s
[CV 2/5] END xgb__max_depth=2, xgb__reg_alpha=0.5, xgb__reg_lambda=0.5;, score=(train=1.000, test=0.594) total time=   4.8s
[CV 3/5] END xgb__max_depth=2, xgb__reg_alpha=0.5, xgb__reg_lambda=0.5;, score=(train=1.000, test=0.572) total time=   5.0s
[CV 4/5] END xgb__max_depth=2, xgb__reg_alpha=0.5, xgb__reg_lambda=0.5;, score=(train=1.000, test=0.649) total time=   5.0s
[CV 

[CV 5/5] END xgb__max_depth=2, xgb__reg_alpha=5, xgb__reg_lambda=0.5;, score=(train=0.932, test=0.703) total time=   5.3s
[CV 1/5] END xgb__max_depth=2, xgb__reg_alpha=5, xgb__reg_lambda=1;, score=(train=0.925, test=0.551) total time=   5.2s
[CV 2/5] END xgb__max_depth=2, xgb__reg_alpha=5, xgb__reg_lambda=1;, score=(train=0.914, test=0.609) total time=   5.0s
[CV 3/5] END xgb__max_depth=2, xgb__reg_alpha=5, xgb__reg_lambda=1;, score=(train=0.899, test=0.549) total time=   5.2s
[CV 4/5] END xgb__max_depth=2, xgb__reg_alpha=5, xgb__reg_lambda=1;, score=(train=0.964, test=0.672) total time=   5.0s
[CV 5/5] END xgb__max_depth=2, xgb__reg_alpha=5, xgb__reg_lambda=1;, score=(train=0.928, test=0.669) total time=   5.1s
[CV 1/5] END xgb__max_depth=2, xgb__reg_alpha=5, xgb__reg_lambda=5;, score=(train=0.922, test=0.542) total time=   5.3s
[CV 2/5] END xgb__max_depth=2, xgb__reg_alpha=5, xgb__reg_lambda=5;, score=(train=0.914, test=0.607) total time=   5.0s
[CV 3/5] END xgb__max_depth=2, xgb__re

[CV 2/5] END xgb__max_depth=4, xgb__reg_alpha=0.1, xgb__reg_lambda=5;, score=(train=1.000, test=0.642) total time=   4.6s
[CV 3/5] END xgb__max_depth=4, xgb__reg_alpha=0.1, xgb__reg_lambda=5;, score=(train=1.000, test=0.538) total time=   5.3s
[CV 4/5] END xgb__max_depth=4, xgb__reg_alpha=0.1, xgb__reg_lambda=5;, score=(train=1.000, test=0.643) total time=   5.0s
[CV 5/5] END xgb__max_depth=4, xgb__reg_alpha=0.1, xgb__reg_lambda=5;, score=(train=1.000, test=0.696) total time=   5.3s
[CV 1/5] END xgb__max_depth=4, xgb__reg_alpha=0.1, xgb__reg_lambda=10;, score=(train=1.000, test=0.596) total time=   4.7s
[CV 2/5] END xgb__max_depth=4, xgb__reg_alpha=0.1, xgb__reg_lambda=10;, score=(train=1.000, test=0.614) total time=   5.0s
[CV 3/5] END xgb__max_depth=4, xgb__reg_alpha=0.1, xgb__reg_lambda=10;, score=(train=1.000, test=0.579) total time=   5.5s
[CV 4/5] END xgb__max_depth=4, xgb__reg_alpha=0.1, xgb__reg_lambda=10;, score=(train=1.000, test=0.625) total time=   5.2s
[CV 5/5] END xgb__ma

[CV 4/5] END xgb__max_depth=4, xgb__reg_alpha=0.5, xgb__reg_lambda=10;, score=(train=1.000, test=0.660) total time=   5.7s
[CV 5/5] END xgb__max_depth=4, xgb__reg_alpha=0.5, xgb__reg_lambda=10;, score=(train=1.000, test=0.631) total time=   4.6s
[CV 1/5] END xgb__max_depth=4, xgb__reg_alpha=1, xgb__reg_lambda=0.1;, score=(train=1.000, test=0.578) total time=   5.2s
[CV 2/5] END xgb__max_depth=4, xgb__reg_alpha=1, xgb__reg_lambda=0.1;, score=(train=1.000, test=0.660) total time=   4.7s
[CV 3/5] END xgb__max_depth=4, xgb__reg_alpha=1, xgb__reg_lambda=0.1;, score=(train=1.000, test=0.558) total time=   4.9s
[CV 4/5] END xgb__max_depth=4, xgb__reg_alpha=1, xgb__reg_lambda=0.1;, score=(train=1.000, test=0.690) total time=   4.6s
[CV 5/5] END xgb__max_depth=4, xgb__reg_alpha=1, xgb__reg_lambda=0.1;, score=(train=1.000, test=0.607) total time=   4.8s
[CV 1/5] END xgb__max_depth=4, xgb__reg_alpha=1, xgb__reg_lambda=0.25;, score=(train=1.000, test=0.611) total time=   4.7s
[CV 2/5] END xgb__max

...

[CV 4/5] END xgb__max_depth=8, xgb__reg_alpha=5, xgb__reg_lambda=0.5;, score=(train=0.992, test=0.632) total time=   5.4s
[CV 5/5] END xgb__max_depth=8, xgb__reg_alpha=5, xgb__reg_lambda=0.5;, score=(train=0.996, test=0.647) total time=   5.6s
[CV 1/5] END xgb__max_depth=8, xgb__reg_alpha=5, xgb__reg_lambda=1;, score=(train=0.996, test=0.625) total time=   5.3s
[CV 2/5] END xgb__max_depth=8, xgb__reg_alpha=5, xgb__reg_lambda=1;, score=(train=0.986, test=0.598) total time=   5.5s
[CV 3/5] END xgb__max_depth=8, xgb__reg_alpha=5, xgb__reg_lambda=1;, score=(train=0.991, test=0.573) total time=   5.3s
[CV 4/5] END xgb__max_depth=8, xgb__reg_alpha=5, xgb__reg_lambda=1;, score=(train=1.000, test=0.702) total time=   5.0s
[CV 5/5] END xgb__max_depth=8, xgb__reg_alpha=5, xgb__reg_lambda=1;, score=(train=1.000, test=0.583) total time=   5.3s
[CV 1/5] END xgb__max_depth=8, xgb__reg_alpha=5, xgb__reg_lambda=5;, score=(train=0.983, test=0.580) total time=   5.2s
[CV 2/5] END xgb__max_depth=8, xgb__

[CV 1/5] END xgb__max_depth=10, xgb__reg_alpha=0.1, xgb__reg_lambda=5;, score=(train=1.000, test=0.576) total time=   5.3s
[CV 2/5] END xgb__max_depth=10, xgb__reg_alpha=0.1, xgb__reg_lambda=5;, score=(train=1.000, test=0.602) total time=   5.5s
[CV 3/5] END xgb__max_depth=10, xgb__reg_alpha=0.1, xgb__reg_lambda=5;, score=(train=1.000, test=0.550) total time=   5.8s
[CV 4/5] END xgb__max_depth=10, xgb__reg_alpha=0.1, xgb__reg_lambda=5;, score=(train=1.000, test=0.631) total time=   5.6s
[CV 5/5] END xgb__max_depth=10, xgb__reg_alpha=0.1, xgb__reg_lambda=5;, score=(train=1.000, test=0.627) total time=   5.5s
[CV 1/5] END xgb__max_depth=10, xgb__reg_alpha=0.1, xgb__reg_lambda=10;, score=(train=1.000, test=0.589) total time=   5.8s
[CV 2/5] END xgb__max_depth=10, xgb__reg_alpha=0.1, xgb__reg_lambda=10;, score=(train=1.000, test=0.604) total time=   5.8s
[CV 3/5] END xgb__max_depth=10, xgb__reg_alpha=0.1, xgb__reg_lambda=10;, score=(train=1.000, test=0.537) total time=   5.3s
[CV 4/5] END 

[CV 2/5] END xgb__max_depth=10, xgb__reg_alpha=0.5, xgb__reg_lambda=10;, score=(train=1.000, test=0.640) total time=   5.4s
[CV 3/5] END xgb__max_depth=10, xgb__reg_alpha=0.5, xgb__reg_lambda=10;, score=(train=1.000, test=0.591) total time=   5.0s
[CV 4/5] END xgb__max_depth=10, xgb__reg_alpha=0.5, xgb__reg_lambda=10;, score=(train=1.000, test=0.631) total time=   5.5s
[CV 5/5] END xgb__max_depth=10, xgb__reg_alpha=0.5, xgb__reg_lambda=10;, score=(train=1.000, test=0.616) total time=   4.8s
[CV 1/5] END xgb__max_depth=10, xgb__reg_alpha=1, xgb__reg_lambda=0.1;, score=(train=1.000, test=0.551) total time=   5.2s
[CV 2/5] END xgb__max_depth=10, xgb__reg_alpha=1, xgb__reg_lambda=0.1;, score=(train=1.000, test=0.589) total time=   4.9s
[CV 3/5] END xgb__max_depth=10, xgb__reg_alpha=1, xgb__reg_lambda=0.1;, score=(train=1.000, test=0.564) total time=   5.2s
[CV 4/5] END xgb__max_depth=10, xgb__reg_alpha=1, xgb__reg_lambda=0.1;, score=(train=1.000, test=0.531) total time=   5.6s
[CV 5/5] END

[CV 4/5] END xgb__max_depth=10, xgb__reg_alpha=10, xgb__reg_lambda=0.1;, score=(train=0.949, test=0.572) total time=   5.2s
[CV 5/5] END xgb__max_depth=10, xgb__reg_alpha=10, xgb__reg_lambda=0.1;, score=(train=0.950, test=0.615) total time=   5.4s
[CV 1/5] END xgb__max_depth=10, xgb__reg_alpha=10, xgb__reg_lambda=0.25;, score=(train=0.936, test=0.558) total time=   5.2s
[CV 2/5] END xgb__max_depth=10, xgb__reg_alpha=10, xgb__reg_lambda=0.25;, score=(train=0.905, test=0.575) total time=   5.4s
[CV 3/5] END xgb__max_depth=10, xgb__reg_alpha=10, xgb__reg_lambda=0.25;, score=(train=0.951, test=0.511) total time=   5.3s
[CV 4/5] END xgb__max_depth=10, xgb__reg_alpha=10, xgb__reg_lambda=0.25;, score=(train=0.961, test=0.589) total time=   5.3s
[CV 5/5] END xgb__max_depth=10, xgb__reg_alpha=10, xgb__reg_lambda=0.25;, score=(train=0.960, test=0.614) total time=   5.0s
[CV 1/5] END xgb__max_depth=10, xgb__reg_alpha=10, xgb__reg_lambda=0.5;, score=(train=0.936, test=0.603) total time=   5.4s
[CV

{'xgb__max_depth': 2, 'xgb__reg_alpha': 1, 'xgb__reg_lambda': 0.5}

## 6. Neural Network

In [78]:
mlp_pipeline = Pipeline(steps = [("scale", StandardScaler()),
                                 ("pca", PCA()),
                                 ("mlp", MLPClassifier(max_iter=1000,
                                                      hidden_layer_sizes=(200,),
                                                      alpha=10))
                                ])

### 5-fold CV

In [79]:
mlp_scores = cross_validate(mlp_pipeline, geno, pheno["label"], cv=5, scoring=["accuracy",
                                                                             "roc_auc",
                                                                             "f1",
                                                                             "balanced_accuracy"])
mlp_scores


{'fit_time': array([3.736552  , 3.29184318, 3.39785194, 3.62900615, 2.29394293]),
 'score_time': array([0.09211087, 0.07240772, 0.15041399, 0.07994008, 0.09283113]),
 'test_accuracy': array([0.74528302, 0.80188679, 0.8490566 , 0.75238095, 0.80952381]),
 'test_roc_auc': array([0.77938596, 0.81666667, 0.85570175, 0.81955556, 0.83244444]),
 'test_f1': array([0.82802548, 0.87116564, 0.89473684, 0.8375    , 0.87179487]),
 'test_balanced_accuracy': array([0.66096491, 0.7004386 , 0.81403509, 0.64666667, 0.73666667])}

In [80]:
print("ACC")
print("mean =", np.mean(mlp_scores["test_accuracy"]))
print("std =", np.std(mlp_scores["test_accuracy"]), "\n")
print("AUC")
print("mean =", np.mean(mlp_scores["test_roc_auc"]))
print("std =", np.std(mlp_scores["test_roc_auc"]), "\n")
print("F1")
print("mean =", np.mean(mlp_scores["test_f1"]))
print("std =", np.std(mlp_scores["test_f1"]), "\n")
print("BAS")
print("mean =", np.mean(mlp_scores["test_balanced_accuracy"]))
print("std =", np.std(mlp_scores["test_balanced_accuracy"]), "\n")

ACC
mean = 0.7916262353998202
std = 0.03850126065432479 

AUC
mean = 0.8207508771929823
std = 0.02484942200154252 

F1
mean = 0.8606445671557841
std = 0.02448257156283956 

BAS
mean = 0.7117543859649123
std = 0.0600433053938767 



In [74]:
# Run once for every score metric
grid_search_mlp = GridSearchCV(estimator=mlp_pipeline,
                               param_grid={"mlp__hidden_layer_sizes": [(100,), (200,), (400,), (800,)],
                                           "mlp__alpha": [0.01, 0.05, 0.1, 0.25, 0.5, 1, 5, 10, 20, 50, 100]},
                               scoring="balanced_accuracy",
                               return_train_score=True, # or else you'll never see it!
                               cv=KFold(n_splits=5, shuffle=True, random_state=0), # to get a consistent set of settings
                               verbose=3)


In [77]:
grid_search_mlp.fit(geno, pheno["label"])
grid_search_mlp.best_params_

Fitting 5 folds for each of 44 candidates, totalling 220 fits
[CV 1/5] END mlp__alpha=0.01, mlp__hidden_layer_sizes=(100,);, score=(train=1.000, test=0.633) total time=   1.6s
[CV 2/5] END mlp__alpha=0.01, mlp__hidden_layer_sizes=(100,);, score=(train=1.000, test=0.677) total time=   1.6s
[CV 3/5] END mlp__alpha=0.01, mlp__hidden_layer_sizes=(100,);, score=(train=1.000, test=0.671) total time=   1.5s
[CV 4/5] END mlp__alpha=0.01, mlp__hidden_layer_sizes=(100,);, score=(train=1.000, test=0.656) total time=   1.9s
[CV 5/5] END mlp__alpha=0.01, mlp__hidden_layer_sizes=(100,);, score=(train=1.000, test=0.631) total time=   1.6s
[CV 1/5] END mlp__alpha=0.01, mlp__hidden_layer_sizes=(200,);, score=(train=1.000, test=0.629) total time=   1.7s
[CV 2/5] END mlp__alpha=0.01, mlp__hidden_layer_sizes=(200,);, score=(train=1.000, test=0.671) total time=   2.1s
[CV 3/5] END mlp__alpha=0.01, mlp__hidden_layer_sizes=(200,);, score=(train=1.000, test=0.651) total time=   1.8s
[CV 4/5] END mlp__alpha=0.

[CV 3/5] END mlp__alpha=0.25, mlp__hidden_layer_sizes=(400,);, score=(train=1.000, test=0.666) total time=   5.5s
[CV 4/5] END mlp__alpha=0.25, mlp__hidden_layer_sizes=(400,);, score=(train=1.000, test=0.698) total time=   7.2s
[CV 5/5] END mlp__alpha=0.25, mlp__hidden_layer_sizes=(400,);, score=(train=1.000, test=0.704) total time=   6.3s
[CV 1/5] END mlp__alpha=0.25, mlp__hidden_layer_sizes=(800,);, score=(train=1.000, test=0.649) total time=   9.2s
[CV 2/5] END mlp__alpha=0.25, mlp__hidden_layer_sizes=(800,);, score=(train=1.000, test=0.688) total time=  11.9s
[CV 3/5] END mlp__alpha=0.25, mlp__hidden_layer_sizes=(800,);, score=(train=1.000, test=0.658) total time=  10.3s
[CV 4/5] END mlp__alpha=0.25, mlp__hidden_layer_sizes=(800,);, score=(train=1.000, test=0.662) total time=  10.3s
[CV 5/5] END mlp__alpha=0.25, mlp__hidden_layer_sizes=(800,);, score=(train=1.000, test=0.711) total time=  10.6s
[CV 1/5] END mlp__alpha=0.5, mlp__hidden_layer_sizes=(100,);, score=(train=1.000, test=0

[CV 2/5] END mlp__alpha=10, mlp__hidden_layer_sizes=(200,);, score=(train=0.995, test=0.745) total time=   3.0s
[CV 3/5] END mlp__alpha=10, mlp__hidden_layer_sizes=(200,);, score=(train=1.000, test=0.693) total time=   2.6s
[CV 4/5] END mlp__alpha=10, mlp__hidden_layer_sizes=(200,);, score=(train=1.000, test=0.698) total time=   2.8s
[CV 5/5] END mlp__alpha=10, mlp__hidden_layer_sizes=(200,);, score=(train=1.000, test=0.770) total time=   3.3s
[CV 1/5] END mlp__alpha=10, mlp__hidden_layer_sizes=(400,);, score=(train=1.000, test=0.649) total time=   3.2s
[CV 2/5] END mlp__alpha=10, mlp__hidden_layer_sizes=(400,);, score=(train=0.995, test=0.704) total time=   4.4s
[CV 3/5] END mlp__alpha=10, mlp__hidden_layer_sizes=(400,);, score=(train=1.000, test=0.665) total time=   3.7s
[CV 4/5] END mlp__alpha=10, mlp__hidden_layer_sizes=(400,);, score=(train=1.000, test=0.668) total time=   4.9s
[CV 5/5] END mlp__alpha=10, mlp__hidden_layer_sizes=(400,);, score=(train=1.000, test=0.750) total time=

[CV 5/5] END mlp__alpha=100, mlp__hidden_layer_sizes=(800,);, score=(train=0.504, test=0.500) total time=   5.9s


{'mlp__alpha': 10, 'mlp__hidden_layer_sizes': (200,)}

## Evaluate grid searches!

I have 6 of these
- grid_search_knn
- grid_search_lr
- grid_search_svm
- grid_search_sgd
- grid_search_xgb
- grid_search_mlp

In [ ]:
def metrics_report(search):
    measures = (["mean_train", "std_train"] if search.return_train_score else []) + [
        "mean_test",
        "std_test",
    ]
    report_cols = ["metric"] + measures
    metrics = sorted(
        list(
            {
                "_".join(col.split("_")[2:])
                for col in search.cv_results_
                if any(i in col for i in measures)
            }
        )
    )
    rows = []
    for metric in metrics:
        row = pd.Series(
            [metric]
            + [
                search.cv_results_[f"{measure}_{metric}"][search.best_index_]
                for measure in measures
            ],
            index=report_cols,
        )
        rows.append(row)
    df_metrics = pd.DataFrame(rows, columns=report_cols)

    return df_metrics

In [ ]:
metrics_report(grid_search_lr)

# CCI data

## 1. kNN

In [34]:
knn_pipeline2 = Pipeline(steps = [("scale", MinMaxScaler()),
                                  ("pca", PCA()),
                                  ("knn", KNeighborsClassifier(n_neighbors=7,
                                                              algorithm="auto",
                                                              weights="distance"))
                                ])


### 5-fold CV

In [6]:
knn_scores2 = cross_validate(knn_pipeline2, cci, pheno["label"], cv=5, scoring=["accuracy",
                                                                                "roc_auc",
                                                                                "f1",
                                                                                "balanced_accuracy"])
knn_scores2


{'fit_time': array([0.50277805, 0.39401007, 0.77454591, 0.42895007, 0.37527227]),
 'score_time': array([0.27784586, 0.17491007, 0.3053391 , 0.3248508 , 0.2884078 ]),
 'test_accuracy': array([0.73584906, 0.74528302, 0.67924528, 0.67619048, 0.67619048]),
 'test_roc_auc': array([0.67938596, 0.69736842, 0.6129386 , 0.60666667, 0.53422222]),
 'test_f1': array([0.84090909, 0.84571429, 0.8       , 0.79761905, 0.79761905]),
 'test_balanced_accuracy': array([0.55350877, 0.57017544, 0.51403509, 0.51333333, 0.51333333])}

In [7]:
print("ACC")
print("mean =", np.mean(knn_scores2["test_accuracy"]))
print("std =", np.std(knn_scores2["test_accuracy"]), "\n")
print("AUC")
print("mean =", np.mean(knn_scores2["test_roc_auc"]))
print("std =", np.std(knn_scores2["test_roc_auc"]), "\n")
print("F1")
print("mean =", np.mean(knn_scores2["test_f1"]))
print("std =", np.std(knn_scores2["test_f1"]), "\n")
print("BAS")
print("mean =", np.mean(knn_scores2["test_balanced_accuracy"]))
print("std =", np.std(knn_scores2["test_balanced_accuracy"]), "\n")

ACC
mean = 0.7025516621743038
std = 0.03120159253292676 

AUC
mean = 0.6261163742690059
std = 0.05815665669815013 

F1
mean = 0.8163722943722943
std = 0.022065481598147935 

BAS
mean = 0.5328771929824561
std = 0.02423126495444895 



In [22]:
grid_search_knn2 = GridSearchCV(estimator=knn_pipeline2,
                                param_grid={"knn__weights": ["uniform", "distance"],
                                            "knn__algorithm": ["auto", "ball_tree", "kd_tree", "brute"]},
                                scoring="balanced_accuracy", # scoring can be "accuracy", "roc_auc", "f1"
                                return_train_score=True, # or else you'll never see it!
                                cv=KFold(n_splits=5, shuffle=True, random_state=0),
                                verbose = 3) # to get a consistent set of settings


grid_search_knn2.fit(cci, pheno["label"])

Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV 1/5] END knn__algorithm=auto, knn__weights=uniform;, score=(train=0.614, test=0.535) total time=   1.0s
[CV 2/5] END knn__algorithm=auto, knn__weights=uniform;, score=(train=0.624, test=0.569) total time=   0.6s
[CV 3/5] END knn__algorithm=auto, knn__weights=uniform;, score=(train=0.607, test=0.525) total time=   0.6s
[CV 4/5] END knn__algorithm=auto, knn__weights=uniform;, score=(train=0.636, test=0.519) total time=   0.5s
[CV 5/5] END knn__algorithm=auto, knn__weights=uniform;, score=(train=0.628, test=0.581) total time=   0.6s
[CV 1/5] END knn__algorithm=auto, knn__weights=distance;, score=(train=1.000, test=0.535) total time=   0.4s
[CV 2/5] END knn__algorithm=auto, knn__weights=distance;, score=(train=1.000, test=0.569) total time=   0.6s
[CV 3/5] END knn__algorithm=auto, knn__weights=distance;, score=(train=1.000, test=0.539) total time=   0.6s
[CV 4/5] END knn__algorithm=auto, knn__weights=distance;, score=(train=1.

GridSearchCV(cv=KFold(n_splits=5, random_state=0, shuffle=True),
             estimator=Pipeline(steps=[('scale', MinMaxScaler()),
                                       ('pca', PCA()),
                                       ('knn',
                                        KNeighborsClassifier(n_neighbors=7))]),
             param_grid={'knn__algorithm': ['auto', 'ball_tree', 'kd_tree',
                                            'brute'],
                         'knn__weights': ['uniform', 'distance']},
             return_train_score=True, scoring='balanced_accuracy', verbose=3)

In [23]:
grid_search_knn2.best_params_

{'knn__algorithm': 'auto', 'knn__weights': 'distance'}

- INITIAL
    - ACC = 0.766
    - AUC = 0.637
    - F1 = 0.865

- add MinMaxScaling (StandardScaling gives the same exact scores!)
    - ACC = 0.781
    - AUC = 0.889
    - F1 = 0.875
    
- add PCA with 10 components (MinMax / Standard)
    - ACC = 0.766 / 0.797
    - AUC = UNDEFINED / 0.895
    - F1 = 0.827 / 0.883
    - STICK WITH STANDARDSCALING
    
For whatever reason, AUC remains undefined with any more than 10 PCA components, so that is where we shall leave that specific parameter.

- add better parameters {'knn__algorithm': 'auto', 'knn__weights': 'uniform'}
    - ACC = 0.797
    - AUC = 0.895
    - F1 = 0.883

## 2. Logistic Regression

In [43]:
lr_pipeline2 = Pipeline(steps = [("scale", MinMaxScaler()),
                                 ("pca", PCA()),
                                 ("logistic_regression", LogisticRegression())
                                ])


### 5-fold CV

In [25]:
lr_scores2 = cross_validate(lr_pipeline2, cci, pheno["label"], cv=5, scoring=["accuracy",
                                                                                "roc_auc",
                                                                                "f1",
                                                                                "balanced_accuracy"])
lr_scores2


/Users/ryanburczak/anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/Users/ryanburczak/anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/Users/ryanburczak/anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/Users/ryanburczak/anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/Users/ryanburczak/anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'.

{'fit_time': array([1.77872586, 1.7406981 , 1.33769798, 1.40927291, 1.43158197]),
 'score_time': array([0.01227784, 0.01026106, 0.03207207, 0.00998998, 0.00966811]),
 'test_accuracy': array([0.76415094, 0.81132075, 0.74528302, 0.76190476, 0.74285714]),
 'test_roc_auc': array([0.81666667, 0.79429825, 0.76622807, 0.73466667, 0.73511111]),
 'test_f1': array([0.8447205 , 0.87654321, 0.83018868, 0.83660131, 0.83018868]),
 'test_balanced_accuracy': array([0.66403509, 0.71710526, 0.65087719, 0.69333333, 0.64      ])}

In [26]:
print("ACC")
print("mean =", np.mean(lr_scores2["test_accuracy"]))
print("std =", np.std(lr_scores2["test_accuracy"]), "\n")
print("AUC")
print("mean =", np.mean(lr_scores2["test_roc_auc"]))
print("std =", np.std(lr_scores2["test_roc_auc"]), "\n")
print("F1")
print("mean =", np.mean(lr_scores2["test_f1"]))
print("std =", np.std(lr_scores2["test_f1"]), "\n")
print("BAS")
print("mean =", np.mean(lr_scores2["test_balanced_accuracy"]))
print("std =", np.std(lr_scores2["test_balanced_accuracy"]), "\n")

ACC
mean = 0.7651033243486074
std = 0.024637068824597957 

AUC
mean = 0.7693941520467836
std = 0.032392140176075945 

F1
mean = 0.8436484744902124
std = 0.017292816955061442 

BAS
mean = 0.6730701754385965
std = 0.02834601313146821 



In [22]:
grid_search_lr2 = GridSearchCV(estimator=lr_pipeline2,
                              param_grid={"logistic_regression__solver": ["lbfgs", "liblinear", "newton-cg", "newton-cholesky", "sag", "saga"],
                                          "logistic_regression__C": [0.01, 0.05, 0.1, 0.25, 0.5, 1, 5, 10, 20],
                                          "logistic_regression__l1_ratio": [0, 0.25, 0.50, 0.75, 1] # 0 is akin to l2, 1 is akin to l1
                                         },
                              scoring="balanced_accuracy", # scoring can be "accuracy", "roc_auc", "f1"
                              return_train_score=True, # or else you'll never see it!
                              cv=KFold(n_splits=5, shuffle=True, random_state=0),
                              verbose = 3) # to get a consistent set of settings


grid_search_lr2.fit(cci, pheno["label"])


Fitting 5 folds for each of 270 candidates, totalling 1350 fits


/Users/ryanburczak/anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(


[CV 1/5] END logistic_regression__C=0.01, logistic_regression__l1_ratio=0, logistic_regression__solver=lbfgs;, score=(train=0.566, test=0.516) total time=   1.2s


/Users/ryanburczak/anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(


[CV 2/5] END logistic_regression__C=0.01, logistic_regression__l1_ratio=0, logistic_regression__solver=lbfgs;, score=(train=0.534, test=0.577) total time=   0.7s


...

/Users/ryanburczak/anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/Users/ryanburczak/anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


[CV 4/5] END logistic_regression__C=20, logistic_regression__l1_ratio=1, logistic_regression__solver=saga;, score=(train=0.977, test=0.778) total time=   2.1s


/Users/ryanburczak/anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/Users/ryanburczak/anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/ryanburczak/anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(


[CV 5/5] END logistic_regression__C=20, logistic_regression__l1_ratio=1, logistic_regression__solver=saga;, score=(train=0.980, test=0.631) total time=   2.1s


GridSearchCV(cv=KFold(n_splits=5, random_state=0, shuffle=True),
             estimator=Pipeline(steps=[('scale', MinMaxScaler()),
                                       ('pca', PCA()),
                                       ('logistic_regression',
                                        LogisticRegression(max_iter=2000))]),
             param_grid={'logistic_regression__C': [0.01, 0.05, 0.1, 0.25, 0.5,
                                                    1, 5, 10, 20],
                         'logistic_regression__l1_ratio': [0, 0.25, 0.5, 0.75,
                                                           1],
                         'logistic_regression__solver': ['lbfgs', 'liblinear',
                                                         'newton-cg',
                                                         'newton-cholesky',
                                                         'sag', 'saga']},
             return_train_score=True, scoring='balanced_accuracy', verbose=3)

In [23]:
grid_search_lr2.best_params_

{'logistic_regression__C': 5,
 'logistic_regression__l1_ratio': 0,
 'logistic_regression__solver': 'saga'}

## 3. Support Vector Machine

In [5]:
svm_pipeline2 = Pipeline(steps = [("scaling", StandardScaler()),
                                  ("pca", PCA()),
                                  ("svm", SVC(C=10, gamma="scale",
                                             kernel="rbf"))])


### 5-fold CV

In [6]:
svm_scores2 = cross_validate(svm_pipeline2, cci, pheno["label"], cv=5, scoring=["accuracy",
                                                                                "roc_auc",
                                                                                "f1",
                                                                                "balanced_accuracy"])
svm_scores2


{'fit_time': array([0.53096223, 0.504251  , 0.4662621 , 0.3628931 , 0.406317  ]),
 'score_time': array([0.23400617, 0.23929095, 0.23874903, 0.25271678, 0.20271301]),
 'test_accuracy': array([0.77358491, 0.79245283, 0.68867925, 0.77142857, 0.7047619 ]),
 'test_roc_auc': array([0.72236842, 0.75482456, 0.67675439, 0.73111111, 0.67822222]),
 'test_f1': array([0.85365854, 0.86075949, 0.79245283, 0.84810127, 0.8       ]),
 'test_balanced_accuracy': array([0.66052632, 0.71403509, 0.58114035, 0.68      , 0.61333333])}

In [7]:
print("ACC")
print("mean =", np.mean(svm_scores2["test_accuracy"]))
print("std =", np.std(svm_scores2["test_accuracy"]), "\n")
print("AUC")
print("mean =", np.mean(svm_scores2["test_roc_auc"]))
print("std =", np.std(svm_scores2["test_roc_auc"]), "\n")
print("F1")
print("mean =", np.mean(svm_scores2["test_f1"]))
print("std =", np.std(svm_scores2["test_f1"]), "\n")
print("BAS")
print("mean =", np.mean(svm_scores2["test_balanced_accuracy"]))
print("std =", np.std(svm_scores2["test_balanced_accuracy"]), "\n")

ACC
mean = 0.7461814914645103
std = 0.041355747900770054 

AUC
mean = 0.7126561403508772
std = 0.030619342781790244 

F1
mean = 0.8309944252535433
std = 0.028769338475374148 

BAS
mean = 0.6498070175438595
std = 0.04732273300209234 



In [24]:
grid_search_svm2 = GridSearchCV(estimator=svm_pipeline2,
                                param_grid={"svm__C": [0.01, 0.05, 0.1, 0.25, 0.5, 1, 5, 10], # 20 is taking way too long
                                            "svm__kernel": ["poly", "rbf", "sigmoid"], # linear is taking way too long
                                            "svm__gamma": ["scale", "auto"]},
                                scoring="balanced_accuracy", # scoring can be "accuracy", "roc_auc", "f1"
                                return_train_score=True, # or else you'll never see it!
                                cv=KFold(n_splits=5, shuffle=True, random_state=0),
                                verbose = 3) # to get a consistent set of settings)

grid_search_svm2.fit(cci, pheno["label"])


Fitting 5 folds for each of 48 candidates, totalling 240 fits
[CV 1/5] END svm__C=0.01, svm__gamma=scale, svm__kernel=poly;, score=(train=0.517, test=0.500) total time=   1.3s
[CV 2/5] END svm__C=0.01, svm__gamma=scale, svm__kernel=poly;, score=(train=0.514, test=0.500) total time=   0.6s
[CV 3/5] END svm__C=0.01, svm__gamma=scale, svm__kernel=poly;, score=(train=0.513, test=0.520) total time=   0.5s
[CV 4/5] END svm__C=0.01, svm__gamma=scale, svm__kernel=poly;, score=(train=0.515, test=0.500) total time=   0.5s
[CV 5/5] END svm__C=0.01, svm__gamma=scale, svm__kernel=poly;, score=(train=0.512, test=0.500) total time=   0.4s
[CV 1/5] END svm__C=0.01, svm__gamma=scale, svm__kernel=rbf;, score=(train=0.500, test=0.500) total time=   0.5s
[CV 2/5] END svm__C=0.01, svm__gamma=scale, svm__kernel=rbf;, score=(train=0.500, test=0.500) total time=   0.5s
[CV 3/5] END svm__C=0.01, svm__gamma=scale, svm__kernel=rbf;, score=(train=0.500, test=0.500) total time=   0.4s
[CV 4/5] END svm__C=0.01, svm

[CV 3/5] END svm__C=0.1, svm__gamma=scale, svm__kernel=sigmoid;, score=(train=0.500, test=0.493) total time=   0.6s
[CV 4/5] END svm__C=0.1, svm__gamma=scale, svm__kernel=sigmoid;, score=(train=0.498, test=0.500) total time=   0.5s
[CV 5/5] END svm__C=0.1, svm__gamma=scale, svm__kernel=sigmoid;, score=(train=0.498, test=0.500) total time=   0.6s
[CV 1/5] END svm__C=0.1, svm__gamma=auto, svm__kernel=poly;, score=(train=0.631, test=0.531) total time=   0.6s
[CV 2/5] END svm__C=0.1, svm__gamma=auto, svm__kernel=poly;, score=(train=0.626, test=0.511) total time=   0.5s
[CV 3/5] END svm__C=0.1, svm__gamma=auto, svm__kernel=poly;, score=(train=0.611, test=0.532) total time=   0.5s
[CV 4/5] END svm__C=0.1, svm__gamma=auto, svm__kernel=poly;, score=(train=0.632, test=0.553) total time=   0.6s
[CV 5/5] END svm__C=0.1, svm__gamma=auto, svm__kernel=poly;, score=(train=0.628, test=0.575) total time=   0.5s
[CV 1/5] END svm__C=0.1, svm__gamma=auto, svm__kernel=rbf;, score=(train=0.500, test=0.500) 

[CV 1/5] END svm__C=0.5, svm__gamma=auto, svm__kernel=sigmoid;, score=(train=0.599, test=0.656) total time=   0.6s
[CV 2/5] END svm__C=0.5, svm__gamma=auto, svm__kernel=sigmoid;, score=(train=0.595, test=0.538) total time=   0.5s
[CV 3/5] END svm__C=0.5, svm__gamma=auto, svm__kernel=sigmoid;, score=(train=0.603, test=0.644) total time=   0.6s
[CV 4/5] END svm__C=0.5, svm__gamma=auto, svm__kernel=sigmoid;, score=(train=0.606, test=0.608) total time=   0.5s
[CV 5/5] END svm__C=0.5, svm__gamma=auto, svm__kernel=sigmoid;, score=(train=0.605, test=0.605) total time=   0.6s
[CV 1/5] END svm__C=1, svm__gamma=scale, svm__kernel=poly;, score=(train=0.653, test=0.562) total time=   0.6s
[CV 2/5] END svm__C=1, svm__gamma=scale, svm__kernel=poly;, score=(train=0.662, test=0.524) total time=   0.5s
[CV 3/5] END svm__C=1, svm__gamma=scale, svm__kernel=poly;, score=(train=0.646, test=0.546) total time=   0.5s
[CV 4/5] END svm__C=1, svm__gamma=scale, svm__kernel=poly;, score=(train=0.669, test=0.577) 

[CV 5/5] END svm__C=10, svm__gamma=scale, svm__kernel=rbf;, score=(train=0.946, test=0.696) total time=   0.5s
[CV 1/5] END svm__C=10, svm__gamma=scale, svm__kernel=sigmoid;, score=(train=0.645, test=0.588) total time=   0.6s
[CV 2/5] END svm__C=10, svm__gamma=scale, svm__kernel=sigmoid;, score=(train=0.627, test=0.501) total time=   0.6s
[CV 3/5] END svm__C=10, svm__gamma=scale, svm__kernel=sigmoid;, score=(train=0.612, test=0.621) total time=   0.4s
[CV 4/5] END svm__C=10, svm__gamma=scale, svm__kernel=sigmoid;, score=(train=0.599, test=0.616) total time=   0.4s
[CV 5/5] END svm__C=10, svm__gamma=scale, svm__kernel=sigmoid;, score=(train=0.577, test=0.564) total time=   0.5s
[CV 1/5] END svm__C=10, svm__gamma=auto, svm__kernel=poly;, score=(train=0.949, test=0.555) total time=   0.6s
[CV 2/5] END svm__C=10, svm__gamma=auto, svm__kernel=poly;, score=(train=0.928, test=0.587) total time=   0.6s
[CV 3/5] END svm__C=10, svm__gamma=auto, svm__kernel=poly;, score=(train=0.947, test=0.541) 

GridSearchCV(cv=KFold(n_splits=5, random_state=0, shuffle=True),
             estimator=Pipeline(steps=[('scaling', StandardScaler()),
                                       ('pca', PCA()), ('svm', SVC())]),
             param_grid={'svm__C': [0.01, 0.05, 0.1, 0.25, 0.5, 1, 5, 10],
                         'svm__gamma': ['scale', 'auto'],
                         'svm__kernel': ['poly', 'rbf', 'sigmoid']},
             return_train_score=True, scoring='balanced_accuracy', verbose=3)

In [25]:
grid_search_svm2.best_params_

{'svm__C': 10, 'svm__gamma': 'scale', 'svm__kernel': 'rbf'}

- INITIAL
    - ACC = 0.766
    - AUC = UNDEFINED
    - F1 = 0.867
    
- add MinMaxScaling (StandardScaling gives the same scores...)
    - ACC = 0.766
    - AUC = UNDEFINED
    - F1 = 0.867
    
- add PCA with 10 components (or 30 components)
    - STILL no change in any of the scores?!? This is stupid!!!
    
- FINAL RESULTS (30 components)
    - ACC = 0.641
    - AUC = 0.530
    - F1 = 0.758
    
- no PCA?
    - ACC = 0.672
    - AUC = 0.510
    - F1 - 0.792

## 4. Stochastic Gradient Descent

In [33]:
sgd_pipeline2 = Pipeline(steps = [("scale", MinMaxScaler()),
                                  ("pca", PCA()),
                                  ("sgd", SGDClassifier(alpha=0.05,
                                                       l1_ratio=0.5,
                                                       penalty=None))])

### 5-fold CV

In [34]:
sgd_scores2 = cross_validate(sgd_pipeline2, cci, pheno["label"], cv=5, scoring=["accuracy",
                                                                                "roc_auc",
                                                                                "f1",
                                                                                "balanced_accuracy"])
sgd_scores2


{'fit_time': array([0.55638576, 0.79905367, 0.40840721, 0.41529131, 0.41889715]),
 'score_time': array([0.22798204, 0.24421811, 0.27653766, 0.25419784, 0.25685   ]),
 'test_accuracy': array([0.80188679, 0.82075472, 0.76415094, 0.75238095, 0.6952381 ]),
 'test_roc_auc': array([0.78903509, 0.8254386 , 0.77149123, 0.73644444, 0.56888889]),
 'test_f1': array([0.86956522, 0.87741935, 0.84076433, 0.82894737, 0.80487805]),
 'test_balanced_accuracy': array([0.71052632, 0.76403509, 0.68421053, 0.68666667, 0.55666667])}

In [35]:
print("ACC")
print("mean =", np.mean(sgd_scores2["test_accuracy"]))
print("std =", np.std(sgd_scores2["test_accuracy"]), "\n")
print("AUC")
print("mean =", np.mean(sgd_scores2["test_roc_auc"]))
print("std =", np.std(sgd_scores2["test_roc_auc"]), "\n")
print("F1")
print("mean =", np.mean(sgd_scores2["test_f1"]))
print("std =", np.std(sgd_scores2["test_f1"]), "\n")
print("BAS")
print("mean =", np.mean(sgd_scores2["test_balanced_accuracy"]))
print("std =", np.std(sgd_scores2["test_balanced_accuracy"]), "\n")

ACC
mean = 0.7668823000898473
std = 0.04353893262564069 

AUC
mean = 0.7382596491228071
std = 0.08941185744464421 

F1
mean = 0.8443148641283491
std = 0.026598933280093724 

BAS
mean = 0.6804210526315789
std = 0.0682145872735261 



In [28]:
grid_search_sgd2 = GridSearchCV(estimator=sgd_pipeline2,
                              param_grid={"sgd__penalty": ["l2", "l1", "elasticnet", None],
                                          "sgd__alpha": [0.0001, 0.001, 0.01, 0.05, 0.1, 0.25, 0.5, 1, 5, 10],
                                          "sgd__l1_ratio": [0, 0.25, 0.50, 0.75, 1]},
                              scoring="balanced_accuracy", # scoring can be "accuracy", "roc_auc", "f1"
                              return_train_score=True, # or else you'll never see it!
                              cv=KFold(n_splits=5, shuffle=True, random_state=0),
                              verbose = 3) # to get a consistent set of settings)


In [29]:
grid_search_sgd2.fit(cci, pheno["label"])
grid_search_sgd2.best_params_

Fitting 5 folds for each of 200 candidates, totalling 1000 fits
[CV 1/5] END sgd__alpha=0.0001, sgd__l1_ratio=0, sgd__penalty=l2;, score=(train=0.720, test=0.591) total time=   4.4s
[CV 2/5] END sgd__alpha=0.0001, sgd__l1_ratio=0, sgd__penalty=l2;, score=(train=0.741, test=0.676) total time=   4.5s
[CV 3/5] END sgd__alpha=0.0001, sgd__l1_ratio=0, sgd__penalty=l2;, score=(train=0.669, test=0.633) total time=   5.0s
[CV 4/5] END sgd__alpha=0.0001, sgd__l1_ratio=0, sgd__penalty=l2;, score=(train=0.722, test=0.650) total time=   5.0s
[CV 5/5] END sgd__alpha=0.0001, sgd__l1_ratio=0, sgd__penalty=l2;, score=(train=0.686, test=0.549) total time=   4.3s
[CV 1/5] END sgd__alpha=0.0001, sgd__l1_ratio=0, sgd__penalty=l1;, score=(train=0.791, test=0.631) total time=   5.1s
[CV 2/5] END sgd__alpha=0.0001, sgd__l1_ratio=0, sgd__penalty=l1;, score=(train=0.754, test=0.626) total time=   5.0s
[CV 3/5] END sgd__alpha=0.0001, sgd__l1_ratio=0, sgd__penalty=l1;, score=(train=0.752, test=0.635) total time=

[CV 3/5] END sgd__alpha=0.0001, sgd__l1_ratio=0.75, sgd__penalty=l1;, score=(train=0.756, test=0.624) total time=   5.1s
[CV 4/5] END sgd__alpha=0.0001, sgd__l1_ratio=0.75, sgd__penalty=l1;, score=(train=0.733, test=0.662) total time=   5.2s
[CV 5/5] END sgd__alpha=0.0001, sgd__l1_ratio=0.75, sgd__penalty=l1;, score=(train=0.752, test=0.593) total time=   5.3s
[CV 1/5] END sgd__alpha=0.0001, sgd__l1_ratio=0.75, sgd__penalty=elasticnet;, score=(train=0.792, test=0.629) total time=   5.2s
[CV 2/5] END sgd__alpha=0.0001, sgd__l1_ratio=0.75, sgd__penalty=elasticnet;, score=(train=0.754, test=0.682) total time=   4.9s
[CV 3/5] END sgd__alpha=0.0001, sgd__l1_ratio=0.75, sgd__penalty=elasticnet;, score=(train=0.765, test=0.661) total time=   4.9s
[CV 4/5] END sgd__alpha=0.0001, sgd__l1_ratio=0.75, sgd__penalty=elasticnet;, score=(train=0.747, test=0.726) total time=   4.6s
[CV 5/5] END sgd__alpha=0.0001, sgd__l1_ratio=0.75, sgd__penalty=elasticnet;, score=(train=0.773, test=0.662) total time=

[CV 1/5] END sgd__alpha=0.001, sgd__l1_ratio=0.25, sgd__penalty=None;, score=(train=0.771, test=0.611) total time=   4.9s
[CV 2/5] END sgd__alpha=0.001, sgd__l1_ratio=0.25, sgd__penalty=None;, score=(train=0.736, test=0.682) total time=   4.8s
[CV 3/5] END sgd__alpha=0.001, sgd__l1_ratio=0.25, sgd__penalty=None;, score=(train=0.801, test=0.628) total time=   4.7s
[CV 4/5] END sgd__alpha=0.001, sgd__l1_ratio=0.25, sgd__penalty=None;, score=(train=0.771, test=0.744) total time=   4.8s
[CV 5/5] END sgd__alpha=0.001, sgd__l1_ratio=0.25, sgd__penalty=None;, score=(train=0.764, test=0.610) total time=   5.0s
[CV 1/5] END sgd__alpha=0.001, sgd__l1_ratio=0.5, sgd__penalty=l2;, score=(train=0.759, test=0.602) total time=   4.7s
[CV 2/5] END sgd__alpha=0.001, sgd__l1_ratio=0.5, sgd__penalty=l2;, score=(train=0.748, test=0.680) total time=  17.4s
[CV 3/5] END sgd__alpha=0.001, sgd__l1_ratio=0.5, sgd__penalty=l2;, score=(train=0.770, test=0.685) total time=   4.6s
[CV 4/5] END sgd__alpha=0.001, sg

[CV 4/5] END sgd__alpha=0.01, sgd__l1_ratio=0, sgd__penalty=l2;, score=(train=0.732, test=0.707) total time=   5.0s
[CV 5/5] END sgd__alpha=0.01, sgd__l1_ratio=0, sgd__penalty=l2;, score=(train=0.738, test=0.603) total time=   5.1s
[CV 1/5] END sgd__alpha=0.01, sgd__l1_ratio=0, sgd__penalty=l1;, score=(train=0.669, test=0.533) total time=   5.0s
[CV 2/5] END sgd__alpha=0.01, sgd__l1_ratio=0, sgd__penalty=l1;, score=(train=0.600, test=0.606) total time=   5.0s
[CV 3/5] END sgd__alpha=0.01, sgd__l1_ratio=0, sgd__penalty=l1;, score=(train=0.654, test=0.601) total time=   4.9s
[CV 4/5] END sgd__alpha=0.01, sgd__l1_ratio=0, sgd__penalty=l1;, score=(train=0.650, test=0.601) total time=   5.0s
[CV 5/5] END sgd__alpha=0.01, sgd__l1_ratio=0, sgd__penalty=l1;, score=(train=0.664, test=0.661) total time=   5.0s
[CV 1/5] END sgd__alpha=0.01, sgd__l1_ratio=0, sgd__penalty=elasticnet;, score=(train=0.747, test=0.573) total time=   5.1s
[CV 2/5] END sgd__alpha=0.01, sgd__l1_ratio=0, sgd__penalty=elas

[CV 3/5] END sgd__alpha=0.01, sgd__l1_ratio=0.75, sgd__penalty=elasticnet;, score=(train=0.639, test=0.621) total time=   5.1s
[CV 4/5] END sgd__alpha=0.01, sgd__l1_ratio=0.75, sgd__penalty=elasticnet;, score=(train=0.687, test=0.601) total time=   5.0s
[CV 5/5] END sgd__alpha=0.01, sgd__l1_ratio=0.75, sgd__penalty=elasticnet;, score=(train=0.685, test=0.661) total time=   5.5s
[CV 1/5] END sgd__alpha=0.01, sgd__l1_ratio=0.75, sgd__penalty=None;, score=(train=0.810, test=0.598) total time=   4.9s
[CV 2/5] END sgd__alpha=0.01, sgd__l1_ratio=0.75, sgd__penalty=None;, score=(train=0.772, test=0.676) total time=   5.3s
[CV 3/5] END sgd__alpha=0.01, sgd__l1_ratio=0.75, sgd__penalty=None;, score=(train=0.803, test=0.590) total time=   5.1s
[CV 4/5] END sgd__alpha=0.01, sgd__l1_ratio=0.75, sgd__penalty=None;, score=(train=0.740, test=0.708) total time=   4.7s
[CV 5/5] END sgd__alpha=0.01, sgd__l1_ratio=0.75, sgd__penalty=None;, score=(train=0.789, test=0.621) total time=   4.9s
[CV 1/5] END s

...

[CV 2/5] END sgd__alpha=1, sgd__l1_ratio=0.5, sgd__penalty=None;, score=(train=0.628, test=0.679) total time=   3.7s
[CV 3/5] END sgd__alpha=1, sgd__l1_ratio=0.5, sgd__penalty=None;, score=(train=0.647, test=0.601) total time=   4.0s
[CV 4/5] END sgd__alpha=1, sgd__l1_ratio=0.5, sgd__penalty=None;, score=(train=0.623, test=0.603) total time=   3.8s
[CV 5/5] END sgd__alpha=1, sgd__l1_ratio=0.5, sgd__penalty=None;, score=(train=0.670, test=0.571) total time=   2.8s
[CV 1/5] END sgd__alpha=1, sgd__l1_ratio=0.75, sgd__penalty=l2;, score=(train=0.500, test=0.500) total time=   4.3s
[CV 2/5] END sgd__alpha=1, sgd__l1_ratio=0.75, sgd__penalty=l2;, score=(train=0.500, test=0.500) total time=   4.8s
[CV 3/5] END sgd__alpha=1, sgd__l1_ratio=0.75, sgd__penalty=l2;, score=(train=0.500, test=0.500) total time=   4.6s
[CV 4/5] END sgd__alpha=1, sgd__l1_ratio=0.75, sgd__penalty=l2;, score=(train=0.500, test=0.500) total time=   3.2s
[CV 5/5] END sgd__alpha=1, sgd__l1_ratio=0.75, sgd__penalty=l2;, sco

[CV 3/5] END sgd__alpha=5, sgd__l1_ratio=0.25, sgd__penalty=l1;, score=(train=0.500, test=0.500) total time=   3.8s
[CV 4/5] END sgd__alpha=5, sgd__l1_ratio=0.25, sgd__penalty=l1;, score=(train=0.500, test=0.500) total time=   3.3s
[CV 5/5] END sgd__alpha=5, sgd__l1_ratio=0.25, sgd__penalty=l1;, score=(train=0.500, test=0.500) total time=   3.4s
[CV 1/5] END sgd__alpha=5, sgd__l1_ratio=0.25, sgd__penalty=elasticnet;, score=(train=0.500, test=0.500) total time=   3.0s
[CV 2/5] END sgd__alpha=5, sgd__l1_ratio=0.25, sgd__penalty=elasticnet;, score=(train=0.500, test=0.500) total time=   3.4s
[CV 3/5] END sgd__alpha=5, sgd__l1_ratio=0.25, sgd__penalty=elasticnet;, score=(train=0.500, test=0.500) total time=   4.0s
[CV 4/5] END sgd__alpha=5, sgd__l1_ratio=0.25, sgd__penalty=elasticnet;, score=(train=0.500, test=0.500) total time=   2.7s
[CV 5/5] END sgd__alpha=5, sgd__l1_ratio=0.25, sgd__penalty=elasticnet;, score=(train=0.500, test=0.500) total time=   2.5s
[CV 1/5] END sgd__alpha=5, sgd__

[CV 3/5] END sgd__alpha=5, sgd__l1_ratio=1, sgd__penalty=None;, score=(train=0.541, test=0.603) total time=   3.8s
[CV 4/5] END sgd__alpha=5, sgd__l1_ratio=1, sgd__penalty=None;, score=(train=0.621, test=0.477) total time=   4.0s
[CV 5/5] END sgd__alpha=5, sgd__l1_ratio=1, sgd__penalty=None;, score=(train=0.583, test=0.540) total time=   4.2s
[CV 1/5] END sgd__alpha=10, sgd__l1_ratio=0, sgd__penalty=l2;, score=(train=0.543, test=0.523) total time=   4.2s
[CV 2/5] END sgd__alpha=10, sgd__l1_ratio=0, sgd__penalty=l2;, score=(train=0.553, test=0.528) total time=   3.6s
[CV 3/5] END sgd__alpha=10, sgd__l1_ratio=0, sgd__penalty=l2;, score=(train=0.525, test=0.602) total time=   3.8s
[CV 4/5] END sgd__alpha=10, sgd__l1_ratio=0, sgd__penalty=l2;, score=(train=0.500, test=0.500) total time=   3.9s
[CV 5/5] END sgd__alpha=10, sgd__l1_ratio=0, sgd__penalty=l2;, score=(train=0.500, test=0.500) total time=   3.4s
[CV 1/5] END sgd__alpha=10, sgd__l1_ratio=0, sgd__penalty=l1;, score=(train=0.500, te

[CV 3/5] END sgd__alpha=10, sgd__l1_ratio=0.75, sgd__penalty=l1;, score=(train=0.500, test=0.500) total time=   2.9s
[CV 4/5] END sgd__alpha=10, sgd__l1_ratio=0.75, sgd__penalty=l1;, score=(train=0.500, test=0.500) total time=   3.7s
[CV 5/5] END sgd__alpha=10, sgd__l1_ratio=0.75, sgd__penalty=l1;, score=(train=0.500, test=0.500) total time=   2.5s
[CV 1/5] END sgd__alpha=10, sgd__l1_ratio=0.75, sgd__penalty=elasticnet;, score=(train=0.500, test=0.500) total time=   3.2s
[CV 2/5] END sgd__alpha=10, sgd__l1_ratio=0.75, sgd__penalty=elasticnet;, score=(train=0.500, test=0.500) total time=   3.4s
[CV 3/5] END sgd__alpha=10, sgd__l1_ratio=0.75, sgd__penalty=elasticnet;, score=(train=0.500, test=0.500) total time=   3.9s
[CV 4/5] END sgd__alpha=10, sgd__l1_ratio=0.75, sgd__penalty=elasticnet;, score=(train=0.500, test=0.500) total time=   4.4s
[CV 5/5] END sgd__alpha=10, sgd__l1_ratio=0.75, sgd__penalty=elasticnet;, score=(train=0.500, test=0.500) total time=   3.4s
[CV 1/5] END sgd__alpha=

{'sgd__alpha': 0.05, 'sgd__l1_ratio': 0.5, 'sgd__penalty': None}

- INITIAL
    - ACC = 0.594
    - AUC = 0.602
    - F1 = 0.675
    
- add MinMaxScaling (StandardScaling)
    - ACC = 0.766 (0.781)
    - AUC = 0.657 (0.688)
    - F1 = 0.854 (0.863)
    - GO WITH STANDARDSCALING
    
- add PCA with 50 components
    - ACC = 0.672
    - AUC = 0.569
    - F1 = 0.779
    
- add better parameters
{'pca__n_components': 20,
 'sgd__alpha': 0.01,
 'sgd__l1_ratio': 1,
 'sgd__penalty': 'l2'}
    - ACC = 0.766
    - AUC = 0.637
    - F1 = 0.865

- change L1 ratio to 0 (since we are using L2 penalty here)
    - ACC = 0.781
    - AUC = 0.698
    - F1 = 0.870
    
- no PCA?
    - 0.734
    - 0.555
    - 0.841

## 5. XGBoost

In [87]:
xgb_pipeline2 = Pipeline(steps = [("scale", MinMaxScaler()),
                                  ("pca", PCA()),
                                  ("xgb", XGBClassifier())
                                 ])

### 5-fold CV

In [88]:
xgb_scores2 = cross_validate(xgb_pipeline2, cci, pheno["label"], cv=5, scoring=["accuracy",
                                                                                "roc_auc",
                                                                                "f1",
                                                                                "balanced_accuracy"])
xgb_scores2


{'fit_time': array([0.97220421, 0.90712595, 1.06277895, 1.40502405, 1.18125105]),
 'score_time': array([0.02789664, 0.02567506, 0.02281022, 0.13384676, 0.08308315]),
 'test_accuracy': array([0.74528302, 0.82075472, 0.75471698, 0.77142857, 0.72380952]),
 'test_roc_auc': array([0.70789474, 0.79385965, 0.66052632, 0.736     , 0.68622222]),
 'test_f1': array([0.84571429, 0.88888889, 0.8452381 , 0.85185185, 0.83040936]),
 'test_balanced_accuracy': array([0.57017544, 0.68333333, 0.61710526, 0.66      , 0.55666667])}

In [86]:
print("ACC")
print("mean =", np.mean(xgb_scores2["test_accuracy"]))
print("std =", np.std(xgb_scores2["test_accuracy"]), "\n")
print("AUC")
print("mean =", np.mean(xgb_scores2["test_roc_auc"]))
print("std =", np.std(xgb_scores2["test_roc_auc"]), "\n")
print("F1")
print("mean =", np.mean(xgb_scores2["test_f1"]))
print("std =", np.std(xgb_scores2["test_f1"]), "\n")
print("BAS")
print("mean =", np.mean(xgb_scores2["test_balanced_accuracy"]))
print("std =", np.std(xgb_scores2["test_balanced_accuracy"]), "\n")

ACC
mean = 0.7462533692722373
std = 0.037352539169601025 

AUC
mean = 0.695961403508772
std = 0.03229419321423855 

F1
mean = 0.841637422247072
std = 0.023270377105976084 

BAS
mean = 0.5975789473684211
std = 0.047218430496115245 



In [82]:
grid_search_xgb2 = GridSearchCV(estimator=xgb_pipeline2,
                               param_grid={"xgb__max_depth": [2, 4, 6, 8, 10],
                                           "xgb__reg_alpha": [0.1, 0.25, 0.5, 1, 5, 10],
                                           "xgb__reg_lambda": [0.1, 0.25, 0.5, 1, 5, 10]},
                               scoring="balanced_accuracy",
                               return_train_score=True, # or else you'll never see it!
                               cv=KFold(n_splits=5, shuffle=True, random_state=0),
                               verbose = 3) # to get a consistent set of settings


In [83]:
grid_search_xgb2.fit(cci, pheno["label"])
grid_search_xgb2.best_params_

Fitting 5 folds for each of 180 candidates, totalling 900 fits
[CV 1/5] END xgb__max_depth=2, xgb__reg_alpha=0.1, xgb__reg_lambda=0.1;, score=(train=1.000, test=0.593) total time=   0.5s
[CV 2/5] END xgb__max_depth=2, xgb__reg_alpha=0.1, xgb__reg_lambda=0.1;, score=(train=1.000, test=0.576) total time=   0.7s
[CV 3/5] END xgb__max_depth=2, xgb__reg_alpha=0.1, xgb__reg_lambda=0.1;, score=(train=1.000, test=0.558) total time=   0.7s
[CV 4/5] END xgb__max_depth=2, xgb__reg_alpha=0.1, xgb__reg_lambda=0.1;, score=(train=1.000, test=0.672) total time=   0.7s
[CV 5/5] END xgb__max_depth=2, xgb__reg_alpha=0.1, xgb__reg_lambda=0.1;, score=(train=1.000, test=0.576) total time=   0.8s
[CV 1/5] END xgb__max_depth=2, xgb__reg_alpha=0.1, xgb__reg_lambda=0.25;, score=(train=1.000, test=0.598) total time=   0.7s
[CV 2/5] END xgb__max_depth=2, xgb__reg_alpha=0.1, xgb__reg_lambda=0.25;, score=(train=1.000, test=0.586) total time=   0.7s
[CV 3/5] END xgb__max_depth=2, xgb__reg_alpha=0.1, xgb__reg_lambda=

[CV 2/5] END xgb__max_depth=2, xgb__reg_alpha=0.5, xgb__reg_lambda=0.25;, score=(train=1.000, test=0.591) total time=   0.8s
[CV 3/5] END xgb__max_depth=2, xgb__reg_alpha=0.5, xgb__reg_lambda=0.25;, score=(train=1.000, test=0.619) total time=   0.8s
[CV 4/5] END xgb__max_depth=2, xgb__reg_alpha=0.5, xgb__reg_lambda=0.25;, score=(train=1.000, test=0.677) total time=   0.8s
[CV 5/5] END xgb__max_depth=2, xgb__reg_alpha=0.5, xgb__reg_lambda=0.25;, score=(train=1.000, test=0.595) total time=   0.8s
[CV 1/5] END xgb__max_depth=2, xgb__reg_alpha=0.5, xgb__reg_lambda=0.5;, score=(train=1.000, test=0.584) total time=   0.8s
[CV 2/5] END xgb__max_depth=2, xgb__reg_alpha=0.5, xgb__reg_lambda=0.5;, score=(train=1.000, test=0.583) total time=   0.8s
[CV 3/5] END xgb__max_depth=2, xgb__reg_alpha=0.5, xgb__reg_lambda=0.5;, score=(train=1.000, test=0.543) total time=   1.3s
[CV 4/5] END xgb__max_depth=2, xgb__reg_alpha=0.5, xgb__reg_lambda=0.5;, score=(train=1.000, test=0.637) total time=   0.8s
[CV 

[CV 5/5] END xgb__max_depth=2, xgb__reg_alpha=5, xgb__reg_lambda=0.5;, score=(train=0.984, test=0.555) total time=   0.7s
[CV 1/5] END xgb__max_depth=2, xgb__reg_alpha=5, xgb__reg_lambda=1;, score=(train=0.983, test=0.535) total time=   0.7s
[CV 2/5] END xgb__max_depth=2, xgb__reg_alpha=5, xgb__reg_lambda=1;, score=(train=0.964, test=0.567) total time=   0.7s
[CV 3/5] END xgb__max_depth=2, xgb__reg_alpha=5, xgb__reg_lambda=1;, score=(train=0.982, test=0.578) total time=   0.7s
[CV 4/5] END xgb__max_depth=2, xgb__reg_alpha=5, xgb__reg_lambda=1;, score=(train=0.989, test=0.595) total time=   0.7s
[CV 5/5] END xgb__max_depth=2, xgb__reg_alpha=5, xgb__reg_lambda=1;, score=(train=0.996, test=0.575) total time=   0.7s
[CV 1/5] END xgb__max_depth=2, xgb__reg_alpha=5, xgb__reg_lambda=5;, score=(train=0.966, test=0.535) total time=   0.7s
[CV 2/5] END xgb__max_depth=2, xgb__reg_alpha=5, xgb__reg_lambda=5;, score=(train=0.973, test=0.567) total time=   0.7s
[CV 3/5] END xgb__max_depth=2, xgb__re

[CV 2/5] END xgb__max_depth=4, xgb__reg_alpha=0.1, xgb__reg_lambda=5;, score=(train=1.000, test=0.549) total time=   0.8s
[CV 3/5] END xgb__max_depth=4, xgb__reg_alpha=0.1, xgb__reg_lambda=5;, score=(train=1.000, test=0.572) total time=   0.9s
[CV 4/5] END xgb__max_depth=4, xgb__reg_alpha=0.1, xgb__reg_lambda=5;, score=(train=1.000, test=0.530) total time=   0.8s
[CV 5/5] END xgb__max_depth=4, xgb__reg_alpha=0.1, xgb__reg_lambda=5;, score=(train=1.000, test=0.589) total time=   0.8s
[CV 1/5] END xgb__max_depth=4, xgb__reg_alpha=0.1, xgb__reg_lambda=10;, score=(train=1.000, test=0.533) total time=   0.9s
[CV 2/5] END xgb__max_depth=4, xgb__reg_alpha=0.1, xgb__reg_lambda=10;, score=(train=1.000, test=0.588) total time=   0.8s
[CV 3/5] END xgb__max_depth=4, xgb__reg_alpha=0.1, xgb__reg_lambda=10;, score=(train=1.000, test=0.580) total time=   0.8s
[CV 4/5] END xgb__max_depth=4, xgb__reg_alpha=0.1, xgb__reg_lambda=10;, score=(train=1.000, test=0.588) total time=   0.8s
[CV 5/5] END xgb__ma

[CV 4/5] END xgb__max_depth=4, xgb__reg_alpha=0.5, xgb__reg_lambda=10;, score=(train=1.000, test=0.553) total time=   1.1s
[CV 5/5] END xgb__max_depth=4, xgb__reg_alpha=0.5, xgb__reg_lambda=10;, score=(train=1.000, test=0.535) total time=   0.9s
[CV 1/5] END xgb__max_depth=4, xgb__reg_alpha=1, xgb__reg_lambda=0.1;, score=(train=1.000, test=0.520) total time=   0.7s
[CV 2/5] END xgb__max_depth=4, xgb__reg_alpha=1, xgb__reg_lambda=0.1;, score=(train=1.000, test=0.588) total time=   0.7s
[CV 3/5] END xgb__max_depth=4, xgb__reg_alpha=1, xgb__reg_lambda=0.1;, score=(train=1.000, test=0.633) total time=   0.8s
[CV 4/5] END xgb__max_depth=4, xgb__reg_alpha=1, xgb__reg_lambda=0.1;, score=(train=1.000, test=0.584) total time=   0.8s
[CV 5/5] END xgb__max_depth=4, xgb__reg_alpha=1, xgb__reg_lambda=0.1;, score=(train=1.000, test=0.609) total time=   0.9s
[CV 1/5] END xgb__max_depth=4, xgb__reg_alpha=1, xgb__reg_lambda=0.25;, score=(train=1.000, test=0.529) total time=   0.8s
[CV 2/5] END xgb__max

...

[CV 4/5] END xgb__max_depth=8, xgb__reg_alpha=5, xgb__reg_lambda=0.5;, score=(train=1.000, test=0.543) total time=   0.8s
[CV 5/5] END xgb__max_depth=8, xgb__reg_alpha=5, xgb__reg_lambda=0.5;, score=(train=1.000, test=0.524) total time=   0.7s
[CV 1/5] END xgb__max_depth=8, xgb__reg_alpha=5, xgb__reg_lambda=1;, score=(train=1.000, test=0.555) total time=   0.7s
[CV 2/5] END xgb__max_depth=8, xgb__reg_alpha=5, xgb__reg_lambda=1;, score=(train=1.000, test=0.536) total time=   0.7s
[CV 3/5] END xgb__max_depth=8, xgb__reg_alpha=5, xgb__reg_lambda=1;, score=(train=1.000, test=0.550) total time=   0.8s
[CV 4/5] END xgb__max_depth=8, xgb__reg_alpha=5, xgb__reg_lambda=1;, score=(train=1.000, test=0.607) total time=   0.7s
[CV 5/5] END xgb__max_depth=8, xgb__reg_alpha=5, xgb__reg_lambda=1;, score=(train=1.000, test=0.509) total time=   0.7s
[CV 1/5] END xgb__max_depth=8, xgb__reg_alpha=5, xgb__reg_lambda=5;, score=(train=1.000, test=0.556) total time=   0.8s
[CV 2/5] END xgb__max_depth=8, xgb__

[CV 1/5] END xgb__max_depth=10, xgb__reg_alpha=0.1, xgb__reg_lambda=5;, score=(train=1.000, test=0.560) total time=   0.9s
[CV 2/5] END xgb__max_depth=10, xgb__reg_alpha=0.1, xgb__reg_lambda=5;, score=(train=1.000, test=0.581) total time=   0.9s
[CV 3/5] END xgb__max_depth=10, xgb__reg_alpha=0.1, xgb__reg_lambda=5;, score=(train=1.000, test=0.560) total time=   1.1s
[CV 4/5] END xgb__max_depth=10, xgb__reg_alpha=0.1, xgb__reg_lambda=5;, score=(train=1.000, test=0.537) total time=   0.9s
[CV 5/5] END xgb__max_depth=10, xgb__reg_alpha=0.1, xgb__reg_lambda=5;, score=(train=1.000, test=0.629) total time=   0.9s
[CV 1/5] END xgb__max_depth=10, xgb__reg_alpha=0.1, xgb__reg_lambda=10;, score=(train=1.000, test=0.527) total time=   0.9s
[CV 2/5] END xgb__max_depth=10, xgb__reg_alpha=0.1, xgb__reg_lambda=10;, score=(train=1.000, test=0.575) total time=   0.9s
[CV 3/5] END xgb__max_depth=10, xgb__reg_alpha=0.1, xgb__reg_lambda=10;, score=(train=1.000, test=0.614) total time=   0.9s
[CV 4/5] END 

[CV 2/5] END xgb__max_depth=10, xgb__reg_alpha=0.5, xgb__reg_lambda=10;, score=(train=1.000, test=0.621) total time=   0.9s
[CV 3/5] END xgb__max_depth=10, xgb__reg_alpha=0.5, xgb__reg_lambda=10;, score=(train=1.000, test=0.567) total time=   0.9s
[CV 4/5] END xgb__max_depth=10, xgb__reg_alpha=0.5, xgb__reg_lambda=10;, score=(train=1.000, test=0.589) total time=   0.9s
[CV 5/5] END xgb__max_depth=10, xgb__reg_alpha=0.5, xgb__reg_lambda=10;, score=(train=1.000, test=0.589) total time=   0.9s
[CV 1/5] END xgb__max_depth=10, xgb__reg_alpha=1, xgb__reg_lambda=0.1;, score=(train=1.000, test=0.560) total time=   0.8s
[CV 2/5] END xgb__max_depth=10, xgb__reg_alpha=1, xgb__reg_lambda=0.1;, score=(train=1.000, test=0.624) total time=   0.8s
[CV 3/5] END xgb__max_depth=10, xgb__reg_alpha=1, xgb__reg_lambda=0.1;, score=(train=1.000, test=0.553) total time=   0.8s
[CV 4/5] END xgb__max_depth=10, xgb__reg_alpha=1, xgb__reg_lambda=0.1;, score=(train=1.000, test=0.519) total time=   1.1s
[CV 5/5] END

[CV 4/5] END xgb__max_depth=10, xgb__reg_alpha=10, xgb__reg_lambda=0.1;, score=(train=0.977, test=0.596) total time=   0.8s
[CV 5/5] END xgb__max_depth=10, xgb__reg_alpha=10, xgb__reg_lambda=0.1;, score=(train=0.952, test=0.466) total time=   0.7s
[CV 1/5] END xgb__max_depth=10, xgb__reg_alpha=10, xgb__reg_lambda=0.25;, score=(train=0.958, test=0.560) total time=   0.7s
[CV 2/5] END xgb__max_depth=10, xgb__reg_alpha=10, xgb__reg_lambda=0.25;, score=(train=0.982, test=0.650) total time=   0.7s
[CV 3/5] END xgb__max_depth=10, xgb__reg_alpha=10, xgb__reg_lambda=0.25;, score=(train=0.960, test=0.545) total time=   0.7s
[CV 4/5] END xgb__max_depth=10, xgb__reg_alpha=10, xgb__reg_lambda=0.25;, score=(train=0.977, test=0.613) total time=   0.7s
[CV 5/5] END xgb__max_depth=10, xgb__reg_alpha=10, xgb__reg_lambda=0.25;, score=(train=0.948, test=0.479) total time=   0.7s
[CV 1/5] END xgb__max_depth=10, xgb__reg_alpha=10, xgb__reg_lambda=0.5;, score=(train=0.962, test=0.555) total time=   0.7s
[CV

{'xgb__max_depth': 2, 'xgb__reg_alpha': 0.5, 'xgb__reg_lambda': 0.25}

## 6. Neural Network

In [7]:
mlp_pipeline2 = Pipeline(steps = [("scale", MinMaxScaler()),
                                  ("pca", PCA()),
                                 ("mlp", MLPClassifier(max_iter=1000))
                                ])

### 5-fold CV

In [111]:
mlp_scores2 = cross_validate(mlp_pipeline2, cci, pheno["label"], cv=5, scoring=["accuracy",
                                                                                "roc_auc",
                                                                                "f1",
                                                                                "balanced_accuracy"])
mlp_scores2


{'fit_time': array([3.58873391, 2.50820804, 2.8719821 , 2.31830001, 2.36891699]),
 'score_time': array([0.01002383, 0.01587009, 0.00994778, 0.01078296, 0.00979185]),
 'test_accuracy': array([0.78301887, 0.83018868, 0.71698113, 0.72380952, 0.73333333]),
 'test_roc_auc': array([0.81008772, 0.79298246, 0.72412281, 0.724     , 0.71466667]),
 'test_f1': array([0.85534591, 0.8875    , 0.80769231, 0.81045752, 0.825     ]),
 'test_balanced_accuracy': array([0.69736842, 0.7504386 , 0.63114035, 0.64666667, 0.62333333])}

In [112]:
print("ACC")
print("mean =", np.mean(mlp_scores2["test_accuracy"]))
print("std =", np.std(mlp_scores2["test_accuracy"]), "\n")
print("AUC")
print("mean =", np.mean(mlp_scores2["test_roc_auc"]))
print("std =", np.std(mlp_scores2["test_roc_auc"]), "\n")
print("F1")
print("mean =", np.mean(mlp_scores2["test_f1"]))
print("std =", np.std(mlp_scores2["test_f1"]), "\n")
print("BAS")
print("mean =", np.mean(mlp_scores2["test_balanced_accuracy"]))
print("std =", np.std(mlp_scores2["test_balanced_accuracy"]), "\n")

ACC
mean = 0.757466307277628
std = 0.04311775888466296 

AUC
mean = 0.7531719298245615
std = 0.04000448328214715 

F1
mean = 0.8371991471963725
std = 0.03031193974158428 

BAS
mean = 0.6697894736842105
std = 0.047857018757783996 



In [108]:
# Run once for every score metric
grid_search_mlp2 = GridSearchCV(estimator=mlp_pipeline2,
                               param_grid={"mlp__hidden_layer_sizes": [(100,), (200,), (400,), (800,)],
                                           "mlp__alpha": [0.01, 0.05, 0.1, 0.25, 0.5, 1, 5, 10, 20, 50, 100]},
                               scoring="balanced_accuracy",
                               return_train_score=True, # or else you'll never see it!
                               cv=KFold(n_splits=5, shuffle=True, random_state=0), # to get a consistent set of settings
                               verbose=3)


In [109]:
grid_search_mlp2.fit(cci, pheno["label"])
grid_search_mlp2.best_params_

Fitting 5 folds for each of 44 candidates, totalling 220 fits
[CV 1/5] END mlp__alpha=0.01, mlp__hidden_layer_sizes=(100,);, score=(train=1.000, test=0.651) total time=   1.8s
[CV 2/5] END mlp__alpha=0.01, mlp__hidden_layer_sizes=(100,);, score=(train=1.000, test=0.704) total time=   2.8s
[CV 3/5] END mlp__alpha=0.01, mlp__hidden_layer_sizes=(100,);, score=(train=0.996, test=0.656) total time=   2.4s
[CV 4/5] END mlp__alpha=0.01, mlp__hidden_layer_sizes=(100,);, score=(train=1.000, test=0.778) total time=   2.2s
[CV 5/5] END mlp__alpha=0.01, mlp__hidden_layer_sizes=(100,);, score=(train=1.000, test=0.665) total time=   2.3s
[CV 1/5] END mlp__alpha=0.01, mlp__hidden_layer_sizes=(200,);, score=(train=1.000, test=0.651) total time=   2.8s
[CV 2/5] END mlp__alpha=0.01, mlp__hidden_layer_sizes=(200,);, score=(train=0.995, test=0.666) total time=   2.3s
[CV 3/5] END mlp__alpha=0.01, mlp__hidden_layer_sizes=(200,);, score=(train=1.000, test=0.649) total time=   3.5s
[CV 4/5] END mlp__alpha=0.

[CV 3/5] END mlp__alpha=0.25, mlp__hidden_layer_sizes=(400,);, score=(train=0.973, test=0.644) total time=   4.1s
[CV 4/5] END mlp__alpha=0.25, mlp__hidden_layer_sizes=(400,);, score=(train=0.974, test=0.690) total time=   4.2s
[CV 5/5] END mlp__alpha=0.25, mlp__hidden_layer_sizes=(400,);, score=(train=0.980, test=0.730) total time=   4.2s
[CV 1/5] END mlp__alpha=0.25, mlp__hidden_layer_sizes=(800,);, score=(train=0.992, test=0.620) total time=   6.4s
[CV 2/5] END mlp__alpha=0.25, mlp__hidden_layer_sizes=(800,);, score=(train=0.986, test=0.633) total time=   7.4s
[CV 3/5] END mlp__alpha=0.25, mlp__hidden_layer_sizes=(800,);, score=(train=0.973, test=0.630) total time=   7.3s
[CV 4/5] END mlp__alpha=0.25, mlp__hidden_layer_sizes=(800,);, score=(train=0.981, test=0.732) total time=   7.3s
[CV 5/5] END mlp__alpha=0.25, mlp__hidden_layer_sizes=(800,);, score=(train=0.970, test=0.684) total time=   5.4s
[CV 1/5] END mlp__alpha=0.5, mlp__hidden_layer_sizes=(100,);, score=(train=0.936, test=0

[CV 2/5] END mlp__alpha=10, mlp__hidden_layer_sizes=(200,);, score=(train=0.509, test=0.513) total time=   1.5s
[CV 3/5] END mlp__alpha=10, mlp__hidden_layer_sizes=(200,);, score=(train=0.500, test=0.500) total time=   1.4s
[CV 4/5] END mlp__alpha=10, mlp__hidden_layer_sizes=(200,);, score=(train=0.532, test=0.559) total time=   1.5s
[CV 5/5] END mlp__alpha=10, mlp__hidden_layer_sizes=(200,);, score=(train=0.516, test=0.520) total time=   1.4s
[CV 1/5] END mlp__alpha=10, mlp__hidden_layer_sizes=(400,);, score=(train=0.513, test=0.500) total time=   2.1s
[CV 2/5] END mlp__alpha=10, mlp__hidden_layer_sizes=(400,);, score=(train=0.500, test=0.500) total time=   2.0s
[CV 3/5] END mlp__alpha=10, mlp__hidden_layer_sizes=(400,);, score=(train=0.509, test=0.500) total time=   1.9s
[CV 4/5] END mlp__alpha=10, mlp__hidden_layer_sizes=(400,);, score=(train=0.504, test=0.500) total time=   2.3s
[CV 5/5] END mlp__alpha=10, mlp__hidden_layer_sizes=(400,);, score=(train=0.512, test=0.500) total time=

[CV 5/5] END mlp__alpha=100, mlp__hidden_layer_sizes=(800,);, score=(train=0.500, test=0.500) total time=   4.7s


{'mlp__alpha': 0.05, 'mlp__hidden_layer_sizes': (100,)}

In [ ]:
grid_search_mlp2.best_params_

- INITIAL
    - ACC = 0.766
    - AUC = 0.670
    - F1 = 0.848
    
- add MinMaxScaling (StandardScaling)
    - ACC = 0.734 (0.750)
    - AUC = 0.598 (0.633)
    - F1 = 0.835 (0.843)
    - STICK WITH STANDARDSCALING
    
- add PCA with 50 components
    - ACC = 0.750
    - AUC = 0.622
    - F1 = 0.846
    
- with 10 components?
    - SAME
    
- add better parameters {'mlp__alpha': 20, 'mlp__hidden_layer_sizes': (100,), 'pca__n_components': 10}
    - ACC = 0.781
    - AUC = 0.889
    - F1 = 0.875
    
- but without scaling?
    - ACC = 0.719
    - AUC = 0.558
    - F1 = 0.827

# Combined Data

# 1. kNN

In [39]:
std_scaler = StandardScaler()
mm_scaler = MinMaxScaler()
# Numpy array to Pandas DF: https://datatofish.com/numpy-array-to-pandas-dataframe/
# Get column names: https://www.geeksforgeeks.org/how-to-get-column-names-in-pandas-dataframe/
# Create a list via list comprehension: https://realpython.com/list-comprehension-python/

In [6]:
pca_cols = ["PCA"+str(i+1) for i in range(20)]
pca_cols

['PCA1',
 'PCA2',
 'PCA3',
 'PCA4',
 'PCA5',
 'PCA6',
 'PCA7',
 'PCA8',
 'PCA9',
 'PCA10',
 'PCA11',
 'PCA12',
 'PCA13',
 'PCA14',
 'PCA15',
 'PCA16',
 'PCA17',
 'PCA18',
 'PCA19',
 'PCA20']

In [40]:
# https://machinelearningmastery.com/standardscaler-and-minmaxscaler-transforms-in-python/

# transform data
# Genotype data was standard scaled, PCA = 20
knn_geno = std_scaler.fit_transform(geno)
pca20 = PCA(n_components=20)
knn_geno_pca = pca20.fit_transform(knn_geno)
# type(geno) # pandas.core.frame.DataFrame
# type(knn_geno) # numpy.ndarray
# knn_geno.shape
knn_geno_pca_df = pd.DataFrame(knn_geno_pca, columns = ["geno.pca"+str(i+1) for i in range(20)])
knn_geno_pca_df

,geno.pca1,geno.pca2,geno.pca3,geno.pca4,geno.pca5,geno.pca6,geno.pca7,geno.pca8,geno.pca9,geno.pca10,geno.pca11,geno.pca12,geno.pca13,geno.pca14,geno.pca15,geno.pca16,geno.pca17,geno.pca18,geno.pca19,geno.pca20
0,8.754207,-19.668128,27.008215,1.614773,-25.327524,-0.700167,21.979202,2.611109,9.204699,-19.883444,-0.871775,12.634627,-12.138910,2.515548,8.853691,-1.428104,-12.784963,-3.510236,-0.670435,-7.413360
1,-13.348778,-20.200833,-5.196266,-5.909217,6.346763,-21.036623,4.064041,17.714650,12.681960,7.584879,-5.190978,0.125452,-12.487603,4.707663,27.516164,28.293431,-4.606099,-8.486755,-8.592183,-2.464734
2,-16.228463,2.225859,12.002636,22.112062,8.477414,-2.115833,3.102545,0.409244,-9.367903,3.915647,-11.290798,-1.422619,6.020148,9.576860,-3.233126,2.468467,-3.839131,9.703975,13.188052,6.146224
3,22.112931,-11.689406,1.786698,-11.430769,5.677688,13.293170,6.795840,8.337721,5.824327,15.439367,-20.153375,5.321275,-4.101302,1.581488,-6.057641,5.280589,11.086177,1.052062,0.570429,-0.900109
4,-6.121578,9.774590,-11.059413,-4.931837,-14.520566,-14.655281,2.248820,-22.633384,5.265403,10.032948,-18.542853,10.266241,-7.432620,2.899213,-15.296417,-6.446711,-1.668170,-2.526677,14.135386,-2.278260
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
523,-7.073278,23.451722,8.288759,-0.666988,-2.928739,-6.515654,3.653363,15.441840,-3.307940,-7.744839,14.614476,-8.145726,4.388944,-7.975492,-10.583697,-2.677259,-5.358727,9.236079,7.359888,2.910670
524,8.030421,-15.942061,27.350271,-3.258791,-3.213009,-11.082555,12.730882,-10.386881,15.140467,9.203607,7.467504,12.798921,8.260412,-2.250423,-2.081809,-7.953810,-7.090528,15.724648,-11.584862,6.092942
525,-36.542397,20.808419,-6.073616,2.979187,-13.132885,-6.464014,-8.832919,1.462512,1.875991,-27.002219,-14.409509,-12.107259,9.156820,-5.779879,-6.123956,5.496465,-0.727821,2.837329,13.514073,4.403713
526,-16.878785,-9.654192,-6.441622,22.443870,7.497023,-6.038239,5.339054,-8.725007,-2.249146,0.157103,-2.387215,-10.421044,6.919647,-1.035866,-4.590424,0.417760,6.908512,-0.712545,-5.285817,4.001891


In [42]:
# https://builtin.com/machine-learning/pca-in-python

# CCI data was standard scaled, then no PCA
knn_cci = mm_scaler.fit_transform(cci)
# knn_cci_pca.shape
knn_cci_df = pd.DataFrame(knn_cci, columns = list(cci.columns))
knn_cci_df

,Astro_to_Astro,Astro_to_EN_L2_3_IT,Astro_to_EN_L3_5_IT_1,Astro_to_EN_L3_5_IT_2,Astro_to_EN_L3_5_IT_3,Astro_to_EN_L5_6_NP,Astro_to_EN_L6B,Astro_to_EN_L6_CT,Astro_to_EN_L6_IT_1,Astro_to_EN_L6_IT_2,...,PVM_to_EN_NF,VLMC_to_EN_NF,EN_L3_5_IT_1_to_EN_NF,EN_L5_ET_to_EN_NF,EN_L6B_to_EN_NF,EN_NF_to_EN_L3_5_IT_1,EN_NF_to_EN_L5_ET,EN_NF_to_EN_L6B,EN_NF_to_SMC,SMC_to_EN_NF
0,0.442536,0.353477,0.481003,0.400985,0.450695,0.434030,0.370786,0.413909,0.448826,0.555685,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.328878,0.274532,0.344674,0.304059,0.302334,0.243900,0.251157,0.290790,0.246103,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.546797,0.283868,0.000000,0.000000,0.409058,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.428541,0.365799,0.384439,0.400338,0.406836,0.000000,0.276461,0.346104,0.000000,0.265997,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.492905,0.592974,0.000000,0.556944,0.705731,0.000000,0.000000,0.479295,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
523,0.349413,0.358951,0.000000,0.343262,0.373900,0.000000,0.000000,0.000000,0.000000,0.358171,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
524,0.535685,0.500754,0.000000,0.445818,0.564546,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
525,0.363129,0.341103,0.357748,0.329670,0.385470,0.342754,0.385469,0.310340,0.360632,0.342676,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
526,0.359605,0.404317,0.427809,0.431414,0.488376,0.394541,0.400131,0.345804,0.403136,0.421706,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Put the combined data together (run 1)

In [43]:
cdata1 = pd.concat([knn_geno_pca_df, knn_cci_df], axis=1)
seed = 0
X_train, X_test, y_train, y_test = train_test_split(cdata1, pheno, test_size=0.20, random_state=seed)
cdata1.shape

(528, 804)

In [51]:
knn_pipeline3 = Pipeline(steps = [("knn", KNeighborsClassifier(n_neighbors=7,
                                                              algorithm="auto",
                                                              weights="distance"))])

In [47]:
grid_search_knn3 = GridSearchCV(estimator=knn_pipeline3,
                                param_grid={"knn__weights": ["uniform", "distance"],
                                            "knn__algorithm": ["auto", "ball_tree", "kd_tree", "brute"]},
                                scoring="balanced_accuracy", # scoring can be "accuracy", "roc_auc", "f1"
                                return_train_score=True, # or else you'll never see it!
                                cv=KFold(n_splits=5, shuffle=True, random_state=0),
                                verbose = 3) # to get a consistent set of settings

grid_search_knn3.fit(cdata1, pheno["label"])
grid_search_knn3.best_params_


Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV 1/5] END knn__algorithm=auto, knn__weights=uniform;, score=(train=0.656, test=0.527) total time=   0.0s
[CV 2/5] END knn__algorithm=auto, knn__weights=uniform;, score=(train=0.628, test=0.575) total time=   0.0s
[CV 3/5] END knn__algorithm=auto, knn__weights=uniform;, score=(train=0.630, test=0.532) total time=   0.0s
[CV 4/5] END knn__algorithm=auto, knn__weights=uniform;, score=(train=0.639, test=0.579) total time=   0.0s
[CV 5/5] END knn__algorithm=auto, knn__weights=uniform;, score=(train=0.633, test=0.536) total time=   0.0s
[CV 1/5] END knn__algorithm=auto, knn__weights=distance;, score=(train=1.000, test=0.527) total time=   0.0s
[CV 2/5] END knn__algorithm=auto, knn__weights=distance;, score=(train=1.000, test=0.575) total time=   0.0s
[CV 3/5] END knn__algorithm=auto, knn__weights=distance;, score=(train=1.000, test=0.532) total time=   0.0s
[CV 4/5] END knn__algorithm=auto, knn__weights=distance;, score=(train=1.

{'knn__algorithm': 'auto', 'knn__weights': 'uniform'}

### 5-fold CV (run 1)

In [52]:
knn_scores3 = cross_validate(knn_pipeline3, cdata1, pheno["label"], cv=5, scoring=["accuracy",
                                                                                   "roc_auc",
                                                                                   "f1",
                                                                                   "balanced_accuracy"])
knn_scores3

{'fit_time': array([0.01102376, 0.00375199, 0.00378203, 0.00379109, 0.00338697]),
 'score_time': array([0.0112462 , 0.01083708, 0.01022291, 0.01107883, 0.01008606]),
 'test_accuracy': array([0.75471698, 0.71698113, 0.68867925, 0.67619048, 0.75238095]),
 'test_roc_auc': array([0.63355263, 0.67719298, 0.60614035, 0.64711111, 0.68777778]),
 'test_f1': array([0.84883721, 0.82954545, 0.80701754, 0.79761905, 0.85057471]),
 'test_balanced_accuracy': array([0.59692982, 0.53026316, 0.52061404, 0.51333333, 0.57666667])}

In [53]:
print("ACC")
print("mean =", np.mean(knn_scores3["test_accuracy"]))
print("std =", np.std(knn_scores3["test_accuracy"]), "\n")
print("AUC")
print("mean =", np.mean(knn_scores3["test_roc_auc"]))
print("std =", np.std(knn_scores3["test_roc_auc"]), "\n")
print("F1")
print("mean =", np.mean(knn_scores3["test_f1"]))
print("std =", np.std(knn_scores3["test_f1"]), "\n")
print("BAS")
print("mean =", np.mean(knn_scores3["test_balanced_accuracy"]))
print("std =", np.std(knn_scores3["test_balanced_accuracy"]), "\n")

ACC
mean = 0.7177897574123989
std = 0.03205852963111827 

AUC
mean = 0.6503549707602339
std = 0.029558481678360747 

F1
mean = 0.826718793594031
std = 0.021453454995350247 

BAS
mean = 0.5475614035087719
std = 0.03310984172356733 



# 2. Logistic Regression

In [54]:
std_scaler = StandardScaler()
mm_scaler = MinMaxScaler()
# Numpy array to Pandas DF: https://datatofish.com/numpy-array-to-pandas-dataframe/
# Get column names: https://www.geeksforgeeks.org/how-to-get-column-names-in-pandas-dataframe/
# Create a list via list comprehension: https://realpython.com/list-comprehension-python/

In [55]:
# https://machinelearningmastery.com/standardscaler-and-minmaxscaler-transforms-in-python/

# transform data
# Genotype data was unscaled, then PCA=80
pca80 = PCA(n_components=80)
lr_geno_pca = pca80.fit_transform(cci)
# type(geno) # pandas.core.frame.DataFrame
# type(lr_geno) # numpy.ndarray
# lr_geno.shape
lr_geno_pca_df = pd.DataFrame(lr_geno_pca, columns = ["geno.pca"+str(i+1) for i in range(80)])
lr_geno_pca_df

,geno.pca1,geno.pca2,geno.pca3,geno.pca4,geno.pca5,geno.pca6,geno.pca7,geno.pca8,geno.pca9,geno.pca10,...,geno.pca71,geno.pca72,geno.pca73,geno.pca74,geno.pca75,geno.pca76,geno.pca77,geno.pca78,geno.pca79,geno.pca80
0,34.670889,-26.908998,-17.252192,-7.068046,20.667683,0.587266,3.362548,-13.992304,0.832845,1.307384,...,-0.324288,1.170405,1.844243,-0.599491,-0.695861,-0.423868,-1.700433,4.194337,-2.212603,0.051609
1,-30.979682,6.855920,-3.428285,31.781495,-6.423387,13.432154,-15.413118,-16.405465,8.814355,-6.496230,...,-0.501565,1.222266,1.082904,0.804357,-1.218939,-1.222906,0.551191,0.457599,-0.157765,-2.641792
2,-147.869311,-28.909048,-5.412683,-1.347331,6.899079,-0.324759,-5.320723,3.760775,3.420749,2.082044,...,-0.449874,3.044765,0.883387,-0.941059,2.999025,2.591667,0.458556,1.530166,0.962572,0.811830
3,-42.547043,6.401305,7.838565,-20.307369,-34.904997,-19.701008,-13.423529,-17.295496,-8.758699,20.679299,...,-3.865021,1.027867,-2.444263,-1.845731,-2.178998,0.951374,-1.669270,-4.128276,-0.822986,-1.132525
4,-60.154159,62.286669,21.004491,-18.888134,-15.234053,-7.646014,-16.999279,6.875095,39.997143,65.982423,...,-6.389891,-1.906564,1.675787,-1.896108,-4.832237,-3.778760,-1.708310,-1.979836,7.838932,3.080478
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
523,-69.796569,21.184132,-3.151401,-59.508042,0.816472,-44.537966,-17.280388,6.566838,-1.884316,21.857966,...,-0.487695,2.297684,3.348058,3.213465,-0.556489,-2.256298,-0.382374,-0.384374,-1.948659,-1.380381
524,-98.824131,42.157786,5.419051,-15.673154,-25.067543,-31.120186,-14.845837,25.887861,21.642254,-12.357327,...,-3.568332,-1.926953,-2.689908,1.779103,3.219086,3.419594,-3.047435,1.815860,3.020347,3.254322
525,12.289871,-39.876640,-9.146093,6.713426,7.007990,0.427844,-12.494847,-4.331310,0.992440,-4.501803,...,1.040808,0.760516,-0.016538,-0.817048,-1.914081,0.494493,1.115651,2.312469,0.395236,0.078824
526,68.601016,-20.199123,-27.774957,1.335159,5.228705,-9.427344,-4.102781,-4.931319,3.229123,-8.548192,...,1.058047,-0.758192,-1.987951,-0.818956,-0.607665,0.714885,-1.671392,0.092261,-0.688016,0.799762


In [60]:
# https://builtin.com/machine-learning/pca-in-python

# CCI data was min max scaled, then no PCA
lr_cci = mm_scaler.fit_transform(cci)
# lr_cci_pca.shape
lr_cci_df = pd.DataFrame(lr_cci, columns = list(cci.columns))
lr_cci_df

,Astro_to_Astro,Astro_to_EN_L2_3_IT,Astro_to_EN_L3_5_IT_1,Astro_to_EN_L3_5_IT_2,Astro_to_EN_L3_5_IT_3,Astro_to_EN_L5_6_NP,Astro_to_EN_L6B,Astro_to_EN_L6_CT,Astro_to_EN_L6_IT_1,Astro_to_EN_L6_IT_2,...,PVM_to_EN_NF,VLMC_to_EN_NF,EN_L3_5_IT_1_to_EN_NF,EN_L5_ET_to_EN_NF,EN_L6B_to_EN_NF,EN_NF_to_EN_L3_5_IT_1,EN_NF_to_EN_L5_ET,EN_NF_to_EN_L6B,EN_NF_to_SMC,SMC_to_EN_NF
0,0.442536,0.353477,0.481003,0.400985,0.450695,0.434030,0.370786,0.413909,0.448826,0.555685,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.328878,0.274532,0.344674,0.304059,0.302334,0.243900,0.251157,0.290790,0.246103,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.546797,0.283868,0.000000,0.000000,0.409058,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.428541,0.365799,0.384439,0.400338,0.406836,0.000000,0.276461,0.346104,0.000000,0.265997,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.492905,0.592974,0.000000,0.556944,0.705731,0.000000,0.000000,0.479295,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
523,0.349413,0.358951,0.000000,0.343262,0.373900,0.000000,0.000000,0.000000,0.000000,0.358171,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
524,0.535685,0.500754,0.000000,0.445818,0.564546,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
525,0.363129,0.341103,0.357748,0.329670,0.385470,0.342754,0.385469,0.310340,0.360632,0.342676,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
526,0.359605,0.404317,0.427809,0.431414,0.488376,0.394541,0.400131,0.345804,0.403136,0.421706,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Put the combined data together (run 2)

In [62]:
cdata2 = pd.concat([lr_geno_pca_df, lr_cci_df], axis=1)
seed = 0
X_train, X_test, y_train, y_test = train_test_split(cdata2, pheno, test_size=0.20, random_state=seed)
cdata2.shape

(528, 864)

In [95]:
lr_pipeline3 = Pipeline(steps = [("logistic_regression", LogisticRegression(max_iter=10000))])

In [70]:
grid_search_lr3 = GridSearchCV(estimator=lr_pipeline3,
                              param_grid={"logistic_regression__solver": ["lbfgs", "saga"],
                                          "logistic_regression__C": [0.01, 0.05, 0.1, 0.25, 0.5, 1, 5, 10, 20],
                                          "logistic_regression__l1_ratio": [0, 0.25, 0.50, 0.75, 1] # 0 is akin to l2, 1 is akin to l1
                                         },
                              scoring="balanced_accuracy", # scoring can be "accuracy", "roc_auc", "f1"
                              return_train_score=True, # or else you'll never see it!
                              cv=KFold(n_splits=5, shuffle=True, random_state=0),
                              verbose = 3) # to get a consistent set of settings

grid_search_lr3.fit(cdata2, pheno["label"])
grid_search_lr3.best_params_


Fitting 5 folds for each of 90 candidates, totalling 450 fits


/Users/ryanburczak/anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(


[CV 1/5] END logistic_regression__C=0.01, logistic_regression__l1_ratio=0, logistic_regression__solver=lbfgs;, score=(train=0.796, test=0.640) total time=   0.4s
[CV 2/5] END logistic_regression__C=0.01, logistic_regression__l1_ratio=0, logistic_regression__solver=lbfgs;, score=(train=0.802, test=0.686) total time=   0.1s


/Users/ryanburczak/anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/Users/ryanburczak/anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(


[CV 3/5] END logistic_regression__C=0.01, logistic_regression__l1_ratio=0, logistic_regression__solver=lbfgs;, score=(train=0.769, test=0.643) total time=   0.2s
[CV 4/5] END logistic_regression__C=0.01, logistic_regression__l1_ratio=0, logistic_regression__solver=lbfgs;, score=(train=0.777, test=0.749) total time=   0.1s


...

[CV 4/5] END logistic_regression__C=20, logistic_regression__l1_ratio=1, logistic_regression__solver=saga;, score=(train=0.818, test=0.715) total time=  14.9s


/Users/ryanburczak/anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(


[CV 5/5] END logistic_regression__C=20, logistic_regression__l1_ratio=1, logistic_regression__solver=saga;, score=(train=0.822, test=0.672) total time=   5.9s


/Users/ryanburczak/anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(


{'logistic_regression__C': 0.01,
 'logistic_regression__l1_ratio': 0,
 'logistic_regression__solver': 'lbfgs'}

### 5-fold CV (run 2)

In [74]:
lr_scores3 = cross_validate(lr_pipeline3, cdata2, pheno["label"], cv=5, scoring=["accuracy",
                                                                                 "roc_auc",
                                                                                 "f1",
                                                                                 "f1_macro",
                                                                                 "balanced_accuracy"])
lr_scores3

/Users/ryanburczak/anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/Users/ryanburczak/anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/Users/ryanburczak/anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/Users/ryanburczak/anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/Users/ryanburczak/anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'.

{'fit_time': array([0.56410217, 0.13536382, 0.81726789, 0.12498569, 0.34410191]),
 'score_time': array([0.00902677, 0.0072639 , 0.01552391, 0.00858116, 0.01327896]),
 'test_accuracy': array([0.74528302, 0.80188679, 0.80188679, 0.76190476, 0.72380952]),
 'test_roc_auc': array([0.83114035, 0.75964912, 0.74692982, 0.70888889, 0.62977778]),
 'test_f1': array([0.83018868, 0.86956522, 0.86956522, 0.83870968, 0.82208589]),
 'test_f1_macro': array([0.66037736, 0.72890026, 0.72890026, 0.69208211, 0.60253231]),
 'test_balanced_accuracy': array([0.65087719, 0.71052632, 0.71052632, 0.68333333, 0.59666667])}

In [75]:
print("ACC")
print("mean =", np.mean(lr_scores3["test_accuracy"]))
print("std =", np.std(lr_scores3["test_accuracy"]), "\n")
print("AUC")
print("mean =", np.mean(lr_scores3["test_roc_auc"]))
print("std =", np.std(lr_scores3["test_roc_auc"]), "\n")
print("F1")
print("mean =", np.mean(lr_scores3["test_f1"]))
print("std =", np.std(lr_scores3["test_f1"]), "\n")
print("BAS")
print("mean =", np.mean(lr_scores3["test_balanced_accuracy"]))
print("std =", np.std(lr_scores3["test_balanced_accuracy"]), "\n")

ACC
mean = 0.7669541778975741
std = 0.030974738195072883 

AUC
mean = 0.7352771929824563
std = 0.06594844809258661 

F1
mean = 0.8460229362035598
std = 0.01992820943297176 

BAS
mean = 0.6703859649122806
std = 0.042907474878287825 



# This was among the best runs! Was it, though?

In [32]:
y_pred = cross_val_predict(knn_pipeline3, cdata1, pheno["label"], cv=5)

In [21]:
print(classification_report(pheno["label"], y_pred))

              precision    recall  f1-score   support

           0       0.69      0.56      0.62        82
           1       0.86      0.91      0.88       238

    accuracy                           0.82       320
   macro avg       0.77      0.74      0.75       320
weighted avg       0.81      0.82      0.82       320



# 3. Support Vector Machine

In [8]:
std_scaler = StandardScaler()
mm_scaler = MinMaxScaler()
# Numpy array to Pandas DF: https://datatofish.com/numpy-array-to-pandas-dataframe/
# Get column names: https://www.geeksforgeeks.org/how-to-get-column-names-in-pandas-dataframe/
# Create a list via list comprehension: https://realpython.com/list-comprehension-python/

In [9]:
# https://machinelearningmastery.com/standardscaler-and-minmaxscaler-transforms-in-python/

# transform data
# Genotype data was min-max scaled, then PCA = 50
svm_geno = mm_scaler.fit_transform(geno)
pca50 = PCA(n_components=50)
svm_geno_pca = pca50.fit_transform(svm_geno)
# type(geno) # pandas.core.frame.DataFrame
# type(svm_geno) # numpy.ndarray
# svm_geno.shape
svm_geno_pca_df = pd.DataFrame(svm_geno_pca, columns = ["geno.pca"+str(i+1) for i in range(50)])
svm_geno_pca_df

,geno.pca1,geno.pca2,geno.pca3,geno.pca4,geno.pca5,geno.pca6,geno.pca7,geno.pca8,geno.pca9,geno.pca10,...,geno.pca41,geno.pca42,geno.pca43,geno.pca44,geno.pca45,geno.pca46,geno.pca47,geno.pca48,geno.pca49,geno.pca50
0,-0.218394,-2.998953,0.726924,1.992512,-1.359678,-4.140429,-0.860840,-1.394309,-0.456346,2.783362,...,1.315561,-1.517947,0.241958,0.123164,-0.633654,-0.905509,0.284912,-0.798752,0.239714,-1.064487
1,-1.195259,-0.528150,2.061422,3.783380,1.103431,-1.335149,-1.044566,0.302295,-0.940837,-2.921212,...,2.207383,0.044698,0.155469,0.278802,1.070365,-1.195685,2.119813,2.275171,1.136632,0.244460
2,-1.452810,-1.642785,-1.293782,-0.459193,-0.575378,2.052799,-1.283994,0.649991,-2.683779,-2.767437,...,1.804058,0.216260,0.904440,0.131513,-0.054375,1.272250,-1.949161,0.039052,-0.294077,-0.685057
3,3.392199,3.343012,-0.151485,2.459676,0.234749,-0.046840,-0.115529,0.645648,1.034298,-1.074296,...,0.211114,0.649514,-1.092587,0.826714,-0.907038,0.159886,-0.729679,1.690416,-0.643916,-0.007751
4,3.227851,-2.885505,0.015485,-0.207667,-0.485184,0.263744,-0.969351,2.966444,-2.449199,0.443139,...,-0.917552,-0.755356,1.619445,0.288732,-1.477905,0.815587,-0.439635,0.075487,1.074391,0.073660
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
523,-0.247869,-0.630124,-0.875965,-2.485689,-1.507442,1.744772,-0.437522,0.826426,-0.007170,-0.026480,...,0.281269,-0.297392,-1.011903,0.285390,0.046690,1.125606,-1.302035,0.767114,1.482063,0.669702
524,-1.546795,-3.166518,-2.755370,3.108519,-2.895537,-0.982428,-0.873220,0.929572,1.049624,-1.263602,...,1.058233,-0.836425,-0.745743,0.040204,2.288991,-0.462255,-2.114687,-1.240055,-0.240122,-0.151898
525,-1.644889,-3.076056,2.051105,-3.430062,0.195815,0.322753,4.027905,0.269251,-1.977825,-2.495703,...,-1.045980,-0.422550,0.359871,2.183865,-1.011425,0.934179,2.240187,-0.520782,-1.093071,-1.661086
526,-1.733543,-1.526172,2.660031,-1.377551,-0.568531,0.405931,-2.925905,0.133525,1.997443,0.676395,...,0.539949,0.362136,0.572941,0.212778,-1.789691,0.505030,0.690612,0.248588,0.822924,0.097065


In [10]:
# https://builtin.com/machine-learning/pca-in-python

# CCI data was Standard Scaled, then no PCA
svm_cci = std_scaler.fit_transform(cci)
# svm_cci_pca.shape
svm_cci_df = pd.DataFrame(svm_cci, columns = list(cci.columns))
svm_cci_df

,Astro_to_Astro,Astro_to_EN_L2_3_IT,Astro_to_EN_L3_5_IT_1,Astro_to_EN_L3_5_IT_2,Astro_to_EN_L3_5_IT_3,Astro_to_EN_L5_6_NP,Astro_to_EN_L6B,Astro_to_EN_L6_CT,Astro_to_EN_L6_IT_1,Astro_to_EN_L6_IT_2,...,IN_SST_to_EN_NF,IN_VIP_to_EN_NF,Immune_to_EN_NF,OPC_to_EN_NF,Oligo_to_EN_NF,EN_L6B_to_EN_NF,EN_NF_to_EN_L3_5_IT_1,EN_NF_to_EN_L6B,EN_NF_to_SMC,SMC_to_EN_NF
0,-0.396960,-0.469253,0.318200,-0.145678,-0.076695,0.605949,0.382534,0.344321,0.529610,1.066127,...,-0.130414,-0.130041,-0.060462,-0.126659,-0.128023,-0.113868,-0.130247,-0.112948,-0.085846,-0.079382
1,-1.030042,-0.869432,-0.265750,-0.620589,-0.763391,-0.121844,-0.067285,-0.155273,-0.260552,-0.925282,...,-0.130414,-0.130041,-0.060462,-0.126659,-0.128023,-0.113868,-0.130247,-0.112948,-0.085846,-0.079382
2,0.183776,-0.822108,-1.742116,-2.110399,-0.269414,-1.055462,-1.011659,-1.335244,-1.219796,-0.925282,...,-0.130414,-0.130041,-0.060462,-0.126659,-0.128023,-0.113868,-0.130247,-0.112948,-0.085846,-0.079382
3,-0.474916,-0.406790,-0.095421,-0.148846,-0.279699,-1.055462,0.027863,0.069183,-1.219796,0.027972,...,-0.130414,-0.130041,-0.060462,-0.126659,-0.128023,-0.113868,-0.130247,-0.112948,-0.085846,-0.079382
4,-0.116407,0.744781,-1.742116,0.618479,1.103753,-1.055462,-1.011659,0.609646,-1.219796,-0.925282,...,-0.130414,-0.130041,-0.060462,-0.126659,-0.128023,-0.113868,-0.130247,-0.112948,-0.085846,-0.079382
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
523,-0.915660,-0.441504,-1.742116,-0.428506,-0.432143,-1.055462,-1.011659,-1.335244,-1.219796,0.358295,...,-0.130414,-0.130041,-0.060462,-0.126659,-0.128023,-0.113868,-0.130247,-0.112948,-0.085846,-0.079382
524,0.121880,0.277308,-1.742116,0.073995,0.450271,-1.055462,-1.011659,-1.335244,-1.219796,-0.925282,...,-0.130414,-0.130041,-0.060462,-0.126659,-0.128023,-0.113868,-0.130247,-0.112948,-0.085846,-0.079382
525,-0.839265,-0.531976,-0.209747,-0.495103,-0.378591,0.256555,0.437744,-0.075941,0.185854,0.302768,...,-0.130414,-0.130041,-0.060462,-0.126659,-0.128023,-0.113868,-0.130247,-0.112948,-0.085846,-0.079382
526,-0.858890,-0.211543,0.090348,0.003415,0.097713,0.454792,0.492874,0.067966,0.351524,0.585988,...,-0.130414,-0.130041,-0.060462,-0.126659,-0.128023,-0.113868,-0.130247,-0.112948,-0.085846,-0.079382


### Put the combined data together (run 3)

In [11]:
cdata3 = pd.concat([svm_geno_pca_df, svm_cci_df], axis=1)
seed = 0
X_train, X_test, y_train, y_test = train_test_split(cdata3, pheno, test_size=0.20, random_state=seed)
cdata3.shape

(528, 750)

In [12]:
svm_pipeline3 = Pipeline(steps = [("svm", SVC(C=5,
                                              gamma='scale',
                                              kernel='rbf'))
                                 ])

In [84]:
grid_search_svm3 = GridSearchCV(estimator=svm_pipeline3,
                                param_grid={"svm__C": [0.01, 0.05, 0.1, 0.25, 0.5, 1, 5, 10], # 20 is taking way too long
                                            "svm__kernel": ["poly", "rbf", "sigmoid"], # linear is taking way too long
                                            "svm__gamma": ["scale", "auto"]},
                                scoring="balanced_accuracy", # scoring can be "accuracy", "roc_auc", "f1"
                                return_train_score=True, # or else you'll never see it!
                                cv=KFold(n_splits=5, shuffle=True, random_state=0),
                                verbose = 3) # to get a consistent set of settings)

grid_search_svm3.fit(cdata3, pheno["label"])
grid_search_svm3.best_params_


Fitting 5 folds for each of 48 candidates, totalling 240 fits
[CV 1/5] END svm__C=0.01, svm__gamma=scale, svm__kernel=poly;, score=(train=0.504, test=0.500) total time=   0.1s
[CV 2/5] END svm__C=0.01, svm__gamma=scale, svm__kernel=poly;, score=(train=0.505, test=0.500) total time=   0.1s
[CV 3/5] END svm__C=0.01, svm__gamma=scale, svm__kernel=poly;, score=(train=0.500, test=0.500) total time=   0.1s
[CV 4/5] END svm__C=0.01, svm__gamma=scale, svm__kernel=poly;, score=(train=0.504, test=0.500) total time=   0.1s
[CV 5/5] END svm__C=0.01, svm__gamma=scale, svm__kernel=poly;, score=(train=0.504, test=0.500) total time=   0.1s
[CV 1/5] END svm__C=0.01, svm__gamma=scale, svm__kernel=rbf;, score=(train=0.500, test=0.500) total time=   0.1s
[CV 2/5] END svm__C=0.01, svm__gamma=scale, svm__kernel=rbf;, score=(train=0.500, test=0.500) total time=   0.1s
[CV 3/5] END svm__C=0.01, svm__gamma=scale, svm__kernel=rbf;, score=(train=0.500, test=0.500) total time=   0.1s
[CV 4/5] END svm__C=0.01, svm

[CV 4/5] END svm__C=0.1, svm__gamma=scale, svm__kernel=sigmoid;, score=(train=0.500, test=0.500) total time=   0.1s
[CV 5/5] END svm__C=0.1, svm__gamma=scale, svm__kernel=sigmoid;, score=(train=0.500, test=0.500) total time=   0.1s
[CV 1/5] END svm__C=0.1, svm__gamma=auto, svm__kernel=poly;, score=(train=0.644, test=0.524) total time=   0.1s
[CV 2/5] END svm__C=0.1, svm__gamma=auto, svm__kernel=poly;, score=(train=0.644, test=0.513) total time=   0.1s
[CV 3/5] END svm__C=0.1, svm__gamma=auto, svm__kernel=poly;, score=(train=0.633, test=0.505) total time=   0.1s
[CV 4/5] END svm__C=0.1, svm__gamma=auto, svm__kernel=poly;, score=(train=0.650, test=0.518) total time=   0.1s
[CV 5/5] END svm__C=0.1, svm__gamma=auto, svm__kernel=poly;, score=(train=0.660, test=0.534) total time=   0.1s
[CV 1/5] END svm__C=0.1, svm__gamma=auto, svm__kernel=rbf;, score=(train=0.500, test=0.500) total time=   0.1s
[CV 2/5] END svm__C=0.1, svm__gamma=auto, svm__kernel=rbf;, score=(train=0.500, test=0.500) total

[CV 4/5] END svm__C=0.5, svm__gamma=auto, svm__kernel=sigmoid;, score=(train=0.629, test=0.655) total time=   0.1s
[CV 5/5] END svm__C=0.5, svm__gamma=auto, svm__kernel=sigmoid;, score=(train=0.622, test=0.603) total time=   0.1s
[CV 1/5] END svm__C=1, svm__gamma=scale, svm__kernel=poly;, score=(train=0.788, test=0.556) total time=   0.1s
[CV 2/5] END svm__C=1, svm__gamma=scale, svm__kernel=poly;, score=(train=0.770, test=0.526) total time=   0.1s
[CV 3/5] END svm__C=1, svm__gamma=scale, svm__kernel=poly;, score=(train=0.792, test=0.552) total time=   0.1s
[CV 4/5] END svm__C=1, svm__gamma=scale, svm__kernel=poly;, score=(train=0.793, test=0.571) total time=   0.1s
[CV 5/5] END svm__C=1, svm__gamma=scale, svm__kernel=poly;, score=(train=0.820, test=0.535) total time=   0.1s
[CV 1/5] END svm__C=1, svm__gamma=scale, svm__kernel=rbf;, score=(train=0.881, test=0.596) total time=   0.1s
[CV 2/5] END svm__C=1, svm__gamma=scale, svm__kernel=rbf;, score=(train=0.797, test=0.569) total time=   

[CV 4/5] END svm__C=10, svm__gamma=scale, svm__kernel=sigmoid;, score=(train=0.652, test=0.686) total time=   0.0s
[CV 5/5] END svm__C=10, svm__gamma=scale, svm__kernel=sigmoid;, score=(train=0.673, test=0.694) total time=   0.0s
[CV 1/5] END svm__C=10, svm__gamma=auto, svm__kernel=poly;, score=(train=1.000, test=0.589) total time=   0.1s
[CV 2/5] END svm__C=10, svm__gamma=auto, svm__kernel=poly;, score=(train=1.000, test=0.634) total time=   0.1s
[CV 3/5] END svm__C=10, svm__gamma=auto, svm__kernel=poly;, score=(train=1.000, test=0.592) total time=   0.1s
[CV 4/5] END svm__C=10, svm__gamma=auto, svm__kernel=poly;, score=(train=1.000, test=0.560) total time=   0.1s
[CV 5/5] END svm__C=10, svm__gamma=auto, svm__kernel=poly;, score=(train=1.000, test=0.617) total time=   0.1s
[CV 1/5] END svm__C=10, svm__gamma=auto, svm__kernel=rbf;, score=(train=1.000, test=0.584) total time=   0.1s
[CV 2/5] END svm__C=10, svm__gamma=auto, svm__kernel=rbf;, score=(train=1.000, test=0.613) total time=   

{'svm__C': 5, 'svm__gamma': 'scale', 'svm__kernel': 'rbf'}

### 5-fold CV (run 3)

In [13]:
svm_scores3 = cross_validate(svm_pipeline3, cdata3, pheno["label"], cv=5, scoring=["accuracy",
                                                                                 "roc_auc",
                                                                                 "f1",
                                                                                 "balanced_accuracy"])
svm_scores3

{'fit_time': array([0.08087707, 0.06212306, 0.06295991, 0.06469584, 0.06152892]),
 'score_time': array([0.03853893, 0.03895807, 0.03998327, 0.03814006, 0.03754187]),
 'test_accuracy': array([0.82075472, 0.86792453, 0.77358491, 0.80952381, 0.78095238]),
 'test_roc_auc': array([0.85482456, 0.88245614, 0.84824561, 0.87777778, 0.856     ]),
 'test_f1': array([0.88050314, 0.9125    , 0.84615385, 0.875     , 0.8516129 ]),
 'test_balanced_accuracy': array([0.74385965, 0.79692982, 0.70087719, 0.71666667, 0.70666667])}

In [14]:
print("ACC")
print("mean =", np.mean(svm_scores3["test_accuracy"]))
print("std =", np.std(svm_scores3["test_accuracy"]), "\n")
print("AUC")
print("mean =", np.mean(svm_scores3["test_roc_auc"]))
print("std =", np.std(svm_scores3["test_roc_auc"]), "\n")
print("F1")
print("mean =", np.mean(svm_scores3["test_f1"]))
print("std =", np.std(svm_scores3["test_f1"]), "\n")
print("BAS")
print("mean =", np.mean(svm_scores3["test_balanced_accuracy"]))
print("std =", np.std(svm_scores3["test_balanced_accuracy"]), "\n")

ACC
mean = 0.8105480682839173
std = 0.03358411245690434 

AUC
mean = 0.8638608187134503
std = 0.013614321654506902 

F1
mean = 0.8731539788067482
std = 0.023658201609994038 

BAS
mean = 0.733
std = 0.035205259267418826 



# 4. Stochastic Gradient Descent

In [12]:
std_scaler = StandardScaler()
mm_scaler = MinMaxScaler()
# Numpy array to Pandas DF: https://datatofish.com/numpy-array-to-pandas-dataframe/
# Get column names: https://www.geeksforgeeks.org/how-to-get-column-names-in-pandas-dataframe/
# Create a list via list comprehension: https://realpython.com/list-comprehension-python/

In [13]:
# https://machinelearningmastery.com/standardscaler-and-minmaxscaler-transforms-in-python/

# transform data
# Genotype data had no scaling and no pca
sgd_geno = np.array(geno)
sgd_geno_df = pd.DataFrame(sgd_geno, columns = list(geno.columns))
sgd_geno_df

,chr1:5284719:T:C,chr1:5286414:T:G,chr1:29385283:T:C,chr1:29390419:G:A,chr1:29390546:C:T,chr1:30051076:G:A,chr1:43183837:A:G,chr1:49120707:A:G,chr1:49127355:G:C,chr1:49199968:G:A,...,chrX:114429288:T:C,chrX:114438661:T:G,chrX:114441059:A:G,chrX:114444757:C:T,chrX:114450759:T:C,chrX:114453552:A:G,chrX:114457175:G:A,chrX:114463117:A:G,chrX:114468124:C:T,chrX:114470502:G:A
0,0.000,0.000,0.958,1.000,0.950,1.000,1.000,1.996,1.996,2.0,...,0.00,0.00,0.00,0.000,0.000,0.000,0.000,0.000,0.000,0.000
1,0.000,0.000,1.971,2.000,1.970,2.000,0.000,0.996,0.996,1.0,...,1.99,1.99,1.99,1.984,1.984,1.984,1.984,1.984,1.984,1.984
2,0.000,0.000,0.999,1.000,0.999,1.000,0.000,1.999,1.999,2.0,...,0.00,0.00,0.00,0.000,0.000,0.000,0.000,0.000,0.000,0.000
3,1.999,2.000,0.000,0.000,0.000,1.000,2.000,0.000,0.000,0.0,...,2.00,2.00,2.00,2.000,2.000,2.000,2.000,2.000,2.000,2.000
4,1.000,1.000,0.976,1.000,0.976,1.003,1.001,0.965,0.965,1.0,...,0.00,0.00,0.00,0.000,0.000,0.000,0.000,0.000,0.000,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
523,0.000,0.000,0.000,0.000,0.000,1.000,0.002,0.987,0.987,1.0,...,1.00,1.00,1.00,1.000,1.000,1.000,1.000,1.000,1.000,1.000
524,0.998,1.000,1.011,1.003,1.000,1.000,0.004,0.000,0.000,0.0,...,2.00,2.00,2.00,2.000,2.000,2.000,2.000,2.000,2.000,2.000
525,2.000,2.000,2.000,2.000,2.000,1.000,1.000,1.000,1.000,1.0,...,0.00,0.00,0.00,0.000,0.000,0.000,0.000,0.000,0.000,0.000
526,1.000,1.000,1.000,1.000,1.000,1.000,0.000,0.986,0.985,1.0,...,2.00,2.00,2.00,2.000,2.000,2.000,2.000,2.000,2.000,2.000


In [14]:
# https://builtin.com/machine-learning/pca-in-python

# CCI data was min-max scaled, then no PCA
sgd_cci = mm_scaler.fit_transform(cci)
# svm_cci_pca.shape
sgd_cci_df = pd.DataFrame(sgd_cci, columns = list(cci.columns))
sgd_cci_df

,Astro_to_Astro,Astro_to_EN_L2_3_IT,Astro_to_EN_L3_5_IT_1,Astro_to_EN_L3_5_IT_2,Astro_to_EN_L3_5_IT_3,Astro_to_EN_L5_6_NP,Astro_to_EN_L6B,Astro_to_EN_L6_CT,Astro_to_EN_L6_IT_1,Astro_to_EN_L6_IT_2,...,PVM_to_EN_NF,VLMC_to_EN_NF,EN_L3_5_IT_1_to_EN_NF,EN_L5_ET_to_EN_NF,EN_L6B_to_EN_NF,EN_NF_to_EN_L3_5_IT_1,EN_NF_to_EN_L5_ET,EN_NF_to_EN_L6B,EN_NF_to_SMC,SMC_to_EN_NF
0,0.442536,0.353477,0.481003,0.400985,0.450695,0.434030,0.370786,0.413909,0.448826,0.555685,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.328878,0.274532,0.344674,0.304059,0.302334,0.243900,0.251157,0.290790,0.246103,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.546797,0.283868,0.000000,0.000000,0.409058,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.428541,0.365799,0.384439,0.400338,0.406836,0.000000,0.276461,0.346104,0.000000,0.265997,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.492905,0.592974,0.000000,0.556944,0.705731,0.000000,0.000000,0.479295,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
523,0.349413,0.358951,0.000000,0.343262,0.373900,0.000000,0.000000,0.000000,0.000000,0.358171,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
524,0.535685,0.500754,0.000000,0.445818,0.564546,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
525,0.363129,0.341103,0.357748,0.329670,0.385470,0.342754,0.385469,0.310340,0.360632,0.342676,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
526,0.359605,0.404317,0.427809,0.431414,0.488376,0.394541,0.400131,0.345804,0.403136,0.421706,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Put the combined data together (run 4)

In [15]:
cdata4 = pd.concat([sgd_geno_df, sgd_cci_df], axis = 1)
seed = 0
X_train, X_test, y_train, y_test = train_test_split(cdata4, pheno, test_size=0.20, random_state=seed)
cdata4.shape

(528, 7098)

In [16]:
sgd_pipeline3 = Pipeline(steps = [("sgd", SGDClassifier(alpha=0.0001,
                                                       l1_ratio=0,
                                                       penalty=None))])

In [133]:
grid_search_sgd3 = GridSearchCV(estimator=sgd_pipeline3,
                                param_grid={"sgd__penalty": ["l2", "l1", "elasticnet", None],
                                            "sgd__alpha": [0.0001, 0.001, 0.01, 0.05, 0.1, 0.25, 0.5, 1, 5, 10],
                                            "sgd__l1_ratio": [0, 0.25, 0.50, 0.75, 1]},
                                scoring="balanced_accuracy", # scoring can be "accuracy", "roc_auc", "f1"
                                return_train_score=True, # or else you'll never see it!
                                cv=KFold(n_splits=5, shuffle=True, random_state=0),
                                verbose = 3) # to get a consistent set of settings)

grid_search_sgd3.fit(cdata4, pheno["label"])
grid_search_sgd3.best_params_


Fitting 5 folds for each of 200 candidates, totalling 1000 fits
[CV 1/5] END sgd__alpha=0.0001, sgd__l1_ratio=0, sgd__penalty=l2;, score=(train=0.886, test=0.605) total time=   0.1s
[CV 2/5] END sgd__alpha=0.0001, sgd__l1_ratio=0, sgd__penalty=l2;, score=(train=1.000, test=0.726) total time=   0.2s
[CV 3/5] END sgd__alpha=0.0001, sgd__l1_ratio=0, sgd__penalty=l2;, score=(train=1.000, test=0.667) total time=   0.2s
[CV 4/5] END sgd__alpha=0.0001, sgd__l1_ratio=0, sgd__penalty=l2;, score=(train=1.000, test=0.733) total time=   0.2s
[CV 5/5] END sgd__alpha=0.0001, sgd__l1_ratio=0, sgd__penalty=l2;, score=(train=1.000, test=0.782) total time=   0.2s
[CV 1/5] END sgd__alpha=0.0001, sgd__l1_ratio=0, sgd__penalty=l1;, score=(train=1.000, test=0.702) total time=   0.3s
[CV 2/5] END sgd__alpha=0.0001, sgd__l1_ratio=0, sgd__penalty=l1;, score=(train=1.000, test=0.693) total time=   0.3s
[CV 3/5] END sgd__alpha=0.0001, sgd__l1_ratio=0, sgd__penalty=l1;, score=(train=1.000, test=0.700) total time=

[CV 3/5] END sgd__alpha=0.0001, sgd__l1_ratio=0.75, sgd__penalty=l1;, score=(train=1.000, test=0.694) total time=   0.3s
[CV 4/5] END sgd__alpha=0.0001, sgd__l1_ratio=0.75, sgd__penalty=l1;, score=(train=1.000, test=0.709) total time=   0.2s
[CV 5/5] END sgd__alpha=0.0001, sgd__l1_ratio=0.75, sgd__penalty=l1;, score=(train=1.000, test=0.676) total time=   0.3s
[CV 1/5] END sgd__alpha=0.0001, sgd__l1_ratio=0.75, sgd__penalty=elasticnet;, score=(train=1.000, test=0.624) total time=   0.2s
[CV 2/5] END sgd__alpha=0.0001, sgd__l1_ratio=0.75, sgd__penalty=elasticnet;, score=(train=1.000, test=0.684) total time=   0.2s
[CV 3/5] END sgd__alpha=0.0001, sgd__l1_ratio=0.75, sgd__penalty=elasticnet;, score=(train=1.000, test=0.690) total time=   0.2s
[CV 4/5] END sgd__alpha=0.0001, sgd__l1_ratio=0.75, sgd__penalty=elasticnet;, score=(train=1.000, test=0.698) total time=   0.2s
[CV 5/5] END sgd__alpha=0.0001, sgd__l1_ratio=0.75, sgd__penalty=elasticnet;, score=(train=1.000, test=0.725) total time=

[CV 2/5] END sgd__alpha=0.001, sgd__l1_ratio=0.25, sgd__penalty=None;, score=(train=1.000, test=0.707) total time=   0.2s
[CV 3/5] END sgd__alpha=0.001, sgd__l1_ratio=0.25, sgd__penalty=None;, score=(train=1.000, test=0.684) total time=   0.2s
[CV 4/5] END sgd__alpha=0.001, sgd__l1_ratio=0.25, sgd__penalty=None;, score=(train=1.000, test=0.762) total time=   0.1s
[CV 5/5] END sgd__alpha=0.001, sgd__l1_ratio=0.25, sgd__penalty=None;, score=(train=1.000, test=0.740) total time=   0.1s
[CV 1/5] END sgd__alpha=0.001, sgd__l1_ratio=0.5, sgd__penalty=l2;, score=(train=1.000, test=0.636) total time=   0.1s
[CV 2/5] END sgd__alpha=0.001, sgd__l1_ratio=0.5, sgd__penalty=l2;, score=(train=1.000, test=0.690) total time=   0.2s
[CV 3/5] END sgd__alpha=0.001, sgd__l1_ratio=0.5, sgd__penalty=l2;, score=(train=1.000, test=0.681) total time=   0.2s
[CV 4/5] END sgd__alpha=0.001, sgd__l1_ratio=0.5, sgd__penalty=l2;, score=(train=1.000, test=0.744) total time=   0.2s
[CV 5/5] END sgd__alpha=0.001, sgd__

[CV 5/5] END sgd__alpha=0.01, sgd__l1_ratio=0, sgd__penalty=l2;, score=(train=1.000, test=0.715) total time=   0.2s
[CV 1/5] END sgd__alpha=0.01, sgd__l1_ratio=0, sgd__penalty=l1;, score=(train=0.991, test=0.685) total time=   0.7s
[CV 2/5] END sgd__alpha=0.01, sgd__l1_ratio=0, sgd__penalty=l1;, score=(train=0.985, test=0.710) total time=   0.6s
[CV 3/5] END sgd__alpha=0.01, sgd__l1_ratio=0, sgd__penalty=l1;, score=(train=0.994, test=0.681) total time=   0.8s
[CV 4/5] END sgd__alpha=0.01, sgd__l1_ratio=0, sgd__penalty=l1;, score=(train=0.998, test=0.697) total time=   0.5s
[CV 5/5] END sgd__alpha=0.01, sgd__l1_ratio=0, sgd__penalty=l1;, score=(train=0.992, test=0.746) total time=   0.7s
[CV 1/5] END sgd__alpha=0.01, sgd__l1_ratio=0, sgd__penalty=elasticnet;, score=(train=1.000, test=0.667) total time=   0.2s
[CV 2/5] END sgd__alpha=0.01, sgd__l1_ratio=0, sgd__penalty=elasticnet;, score=(train=1.000, test=0.697) total time=   0.2s
[CV 3/5] END sgd__alpha=0.01, sgd__l1_ratio=0, sgd__pena

/Users/ryanburczak/anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_stochastic_gradient.py:702: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(


...

[CV 3/5] END sgd__alpha=10, sgd__l1_ratio=1, sgd__penalty=None;, score=(train=0.500, test=0.500) total time=   4.1s


/Users/ryanburczak/anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_stochastic_gradient.py:702: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(


[CV 4/5] END sgd__alpha=10, sgd__l1_ratio=1, sgd__penalty=None;, score=(train=0.616, test=0.586) total time=   4.4s
[CV 5/5] END sgd__alpha=10, sgd__l1_ratio=1, sgd__penalty=None;, score=(train=0.518, test=0.580) total time=   4.0s


/Users/ryanburczak/anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_stochastic_gradient.py:702: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(


{'sgd__alpha': 0.0001, 'sgd__l1_ratio': 0, 'sgd__penalty': None}

### 5-fold CV (run 4)

In [135]:
sgd_scores3 = cross_validate(sgd_pipeline3, cdata4, pheno["label"], cv=5, scoring=["accuracy",
                                                                                 "roc_auc",
                                                                                 "f1",
                                                                                 "balanced_accuracy"])
sgd_scores3

{'fit_time': array([0.10074615, 0.086344  , 0.08946085, 0.09737706, 0.10988712]),
 'score_time': array([0.04354   , 0.04463577, 0.0494101 , 0.04802799, 0.04561305]),
 'test_accuracy': array([0.74528302, 0.81132075, 0.79245283, 0.82857143, 0.75238095]),
 'test_roc_auc': array([0.71535088, 0.79122807, 0.80833333, 0.87688889, 0.79777778]),
 'test_f1': array([0.83018868, 0.87341772, 0.85135135, 0.88461538, 0.81944444]),
 'test_balanced_accuracy': array([0.65087719, 0.7372807 , 0.76447368, 0.76      , 0.72666667])}

In [136]:
print("ACC")
print("mean =", np.mean(sgd_scores3["test_accuracy"]))
print("std =", np.std(sgd_scores3["test_accuracy"]), "\n")
print("AUC")
print("mean =", np.mean(sgd_scores3["test_roc_auc"]))
print("std =", np.std(sgd_scores3["test_roc_auc"]), "\n")
print("F1")
print("mean =", np.mean(sgd_scores3["test_f1"]))
print("std =", np.std(sgd_scores3["test_f1"]), "\n")
print("BAS")
print("mean =", np.mean(sgd_scores3["test_balanced_accuracy"]))
print("std =", np.std(sgd_scores3["test_balanced_accuracy"]), "\n")

ACC
mean = 0.7860017969451932
std = 0.03250605810205301 

AUC
mean = 0.7979157894736841
std = 0.0513945231087357 

F1
mean = 0.8518035162350902
std = 0.024731792231823386 

BAS
mean = 0.727859649122807
std = 0.040963563597769426 



# 5. Extreme Gradient Boosting

In [57]:
std_scaler = StandardScaler()
mm_scaler = MinMaxScaler()
# Numpy array to Pandas DF: https://datatofish.com/numpy-array-to-pandas-dataframe/
# Get column names: https://www.geeksforgeeks.org/how-to-get-column-names-in-pandas-dataframe/
# Create a list via list comprehension: https://realpython.com/list-comprehension-python/

In [58]:
# https://machinelearningmastery.com/standardscaler-and-minmaxscaler-transforms-in-python/

# transform data
# Genotype data was standard scaled, then PCA=50
xgb_geno = std_scaler.fit_transform(geno)
pca100 = PCA(n_components=100)
xgb_geno_pca = pca100.fit_transform(xgb_geno)
# type(geno) # pandas.core.frame.DataFrame
# type(svm_geno) # numpy.ndarray
# svm_geno.shape
xgb_geno_pca_df = pd.DataFrame(xgb_geno_pca, columns = ["geno.pca"+str(i+1) for i in range(100)])
xgb_geno_pca_df

,geno.pca1,geno.pca2,geno.pca3,geno.pca4,geno.pca5,geno.pca6,geno.pca7,geno.pca8,geno.pca9,geno.pca10,...,geno.pca91,geno.pca92,geno.pca93,geno.pca94,geno.pca95,geno.pca96,geno.pca97,geno.pca98,geno.pca99,geno.pca100
0,8.754207,-19.668134,27.008174,1.614149,-25.330959,-0.703709,21.972023,2.595374,9.214891,-19.843672,...,-2.552955,-0.102027,-5.133392,5.646974,-6.032312,-0.852720,-2.561584,-6.175217,1.195081,0.405313
1,-13.348778,-20.200838,-5.196220,-5.909275,6.343626,-21.044578,4.075130,17.699498,12.730816,7.610716,...,-2.439876,-2.646978,-6.313420,4.461187,4.053791,-5.845285,2.132315,-2.641466,-0.282919,0.206858
2,-16.228463,2.225866,12.002672,22.112376,8.481057,-2.117274,3.099436,0.441322,-9.387100,3.881612,...,-4.037604,-1.128473,-0.436512,4.621514,0.431388,1.634094,-2.206626,-7.430335,3.987187,-1.023756
3,22.112931,-11.689412,1.786749,-11.430503,5.677182,13.293841,6.802622,8.336589,5.813914,15.465611,...,3.005203,2.213180,-2.250222,-2.501648,-3.365839,-5.265792,4.997654,-2.117171,0.396308,-0.341153
4,-6.121578,9.774589,-11.059517,-4.932616,-14.520768,-14.664934,2.234190,-22.607765,5.259112,10.056313,...,-0.053941,-0.469654,-1.542700,-0.563890,3.407077,4.959591,-3.574384,-0.218195,3.230055,-0.251995
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
523,-7.073278,23.451719,8.288763,-0.666634,-2.930872,-6.512356,3.664838,15.422408,-3.298949,-7.731114,...,-1.999943,1.319409,7.290038,-3.156983,-2.397718,-2.436522,1.256962,2.775786,5.187729,-1.981849
524,8.030421,-15.942054,27.350003,-3.260164,-3.213903,-11.100605,12.709767,-10.341482,15.194563,9.147729,...,-3.766993,-1.702548,-0.670892,-0.087256,1.620360,-5.550097,-2.015456,2.221767,-2.266851,1.706681
525,-36.542397,20.808432,-6.073657,2.978859,-13.129236,-6.468915,-8.844265,1.497917,1.869278,-27.056679,...,0.583378,-2.094333,7.205838,-1.045786,-5.703841,0.369182,9.244135,2.104103,3.552499,0.124004
526,-16.878785,-9.654193,-6.441480,22.444767,7.495434,-6.022238,5.358932,-8.785029,-2.257243,0.179927,...,2.773585,7.917655,-3.769459,0.582788,-0.787148,-3.303923,2.397732,-0.511614,-6.465220,4.700516


In [59]:
# https://builtin.com/machine-learning/pca-in-python

# CCI data was min-max scaled, then no PCA
xgb_cci = mm_scaler.fit_transform(cci)
# svm_cci_pca.shape
xgb_cci_df = pd.DataFrame(xgb_cci, columns = list(cci.columns))
xgb_cci_df

,Astro_to_Astro,Astro_to_EN_L2_3_IT,Astro_to_EN_L3_5_IT_1,Astro_to_EN_L3_5_IT_2,Astro_to_EN_L3_5_IT_3,Astro_to_EN_L5_6_NP,Astro_to_EN_L6B,Astro_to_EN_L6_CT,Astro_to_EN_L6_IT_1,Astro_to_EN_L6_IT_2,...,PVM_to_EN_NF,VLMC_to_EN_NF,EN_L3_5_IT_1_to_EN_NF,EN_L5_ET_to_EN_NF,EN_L6B_to_EN_NF,EN_NF_to_EN_L3_5_IT_1,EN_NF_to_EN_L5_ET,EN_NF_to_EN_L6B,EN_NF_to_SMC,SMC_to_EN_NF
0,0.442536,0.353477,0.481003,0.400985,0.450695,0.434030,0.370786,0.413909,0.448826,0.555685,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.328878,0.274532,0.344674,0.304059,0.302334,0.243900,0.251157,0.290790,0.246103,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.546797,0.283868,0.000000,0.000000,0.409058,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.428541,0.365799,0.384439,0.400338,0.406836,0.000000,0.276461,0.346104,0.000000,0.265997,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.492905,0.592974,0.000000,0.556944,0.705731,0.000000,0.000000,0.479295,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
523,0.349413,0.358951,0.000000,0.343262,0.373900,0.000000,0.000000,0.000000,0.000000,0.358171,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
524,0.535685,0.500754,0.000000,0.445818,0.564546,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
525,0.363129,0.341103,0.357748,0.329670,0.385470,0.342754,0.385469,0.310340,0.360632,0.342676,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
526,0.359605,0.404317,0.427809,0.431414,0.488376,0.394541,0.400131,0.345804,0.403136,0.421706,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Put the combined data together (run 5)

In [60]:
cdata5 = pd.concat([xgb_geno_pca_df, xgb_cci_df], axis=1)
seed = 0
X_train, X_test, y_train, y_test = train_test_split(cdata5, pheno, test_size=0.20, random_state=seed)
cdata5.shape

(528, 884)

In [61]:
xgb_pipeline3 = Pipeline(steps = [("xgb", XGBClassifier(learning_rate=0.1,
                                                       max_depth=4,
                                                       reg_alpha=0.1,
                                                       reg_lambda=0.25))])

In [12]:
grid_search_xgb3 = GridSearchCV(estimator=xgb_pipeline3,
                                param_grid={"xgb__max_depth": [2, 4, 6, 8, 10],
                                            "xgb__reg_alpha": [0.1, 0.25, 0.5, 1, 5, 10],
                                            "xgb__reg_lambda": [0.1, 0.25, 0.5, 1, 5, 10]},
                                scoring="balanced_accuracy", # scoring can be "accuracy", "roc_auc", "f1"
                                return_train_score=True, # or else you'll never see it!
                                cv=KFold(n_splits=5, shuffle=True, random_state=0),
                                verbose = 3) # to get a consistent set of settings)

grid_search_xgb3.fit(cdata5, pheno["label"])
grid_search_xgb3.best_params_


Fitting 5 folds for each of 180 candidates, totalling 900 fits
[CV 1/5] END xgb__max_depth=2, xgb__reg_alpha=0.1, xgb__reg_lambda=0.1;, score=(train=0.983, test=0.546) total time=   0.3s
[CV 2/5] END xgb__max_depth=2, xgb__reg_alpha=0.1, xgb__reg_lambda=0.1;, score=(train=0.977, test=0.683) total time=   0.3s
[CV 3/5] END xgb__max_depth=2, xgb__reg_alpha=0.1, xgb__reg_lambda=0.1;, score=(train=0.973, test=0.609) total time=   0.3s
[CV 4/5] END xgb__max_depth=2, xgb__reg_alpha=0.1, xgb__reg_lambda=0.1;, score=(train=0.992, test=0.736) total time=   0.3s
[CV 5/5] END xgb__max_depth=2, xgb__reg_alpha=0.1, xgb__reg_lambda=0.1;, score=(train=0.980, test=0.682) total time=   0.3s
[CV 1/5] END xgb__max_depth=2, xgb__reg_alpha=0.1, xgb__reg_lambda=0.25;, score=(train=0.987, test=0.546) total time=   0.3s
[CV 2/5] END xgb__max_depth=2, xgb__reg_alpha=0.1, xgb__reg_lambda=0.25;, score=(train=0.977, test=0.670) total time=   0.3s
[CV 3/5] END xgb__max_depth=2, xgb__reg_alpha=0.1, xgb__reg_lambda=

[CV 2/5] END xgb__max_depth=2, xgb__reg_alpha=0.5, xgb__reg_lambda=0.25;, score=(train=0.973, test=0.688) total time=   0.3s
[CV 3/5] END xgb__max_depth=2, xgb__reg_alpha=0.5, xgb__reg_lambda=0.25;, score=(train=0.969, test=0.649) total time=   0.3s
[CV 4/5] END xgb__max_depth=2, xgb__reg_alpha=0.5, xgb__reg_lambda=0.25;, score=(train=0.992, test=0.784) total time=   0.3s
[CV 5/5] END xgb__max_depth=2, xgb__reg_alpha=0.5, xgb__reg_lambda=0.25;, score=(train=0.972, test=0.690) total time=   0.3s
[CV 1/5] END xgb__max_depth=2, xgb__reg_alpha=0.5, xgb__reg_lambda=0.5;, score=(train=0.970, test=0.538) total time=   0.3s
[CV 2/5] END xgb__max_depth=2, xgb__reg_alpha=0.5, xgb__reg_lambda=0.5;, score=(train=0.968, test=0.677) total time=   0.4s
[CV 3/5] END xgb__max_depth=2, xgb__reg_alpha=0.5, xgb__reg_lambda=0.5;, score=(train=0.965, test=0.629) total time=   0.3s
[CV 4/5] END xgb__max_depth=2, xgb__reg_alpha=0.5, xgb__reg_lambda=0.5;, score=(train=0.985, test=0.725) total time=   0.3s
[CV 

[CV 5/5] END xgb__max_depth=2, xgb__reg_alpha=5, xgb__reg_lambda=0.5;, score=(train=0.924, test=0.709) total time=   0.3s
[CV 1/5] END xgb__max_depth=2, xgb__reg_alpha=5, xgb__reg_lambda=1;, score=(train=0.903, test=0.529) total time=   0.3s
[CV 2/5] END xgb__max_depth=2, xgb__reg_alpha=5, xgb__reg_lambda=1;, score=(train=0.869, test=0.646) total time=   0.3s
[CV 3/5] END xgb__max_depth=2, xgb__reg_alpha=5, xgb__reg_lambda=1;, score=(train=0.885, test=0.629) total time=   0.3s
[CV 4/5] END xgb__max_depth=2, xgb__reg_alpha=5, xgb__reg_lambda=1;, score=(train=0.904, test=0.677) total time=   0.3s
[CV 5/5] END xgb__max_depth=2, xgb__reg_alpha=5, xgb__reg_lambda=1;, score=(train=0.918, test=0.689) total time=   0.3s
[CV 1/5] END xgb__max_depth=2, xgb__reg_alpha=5, xgb__reg_lambda=5;, score=(train=0.890, test=0.529) total time=   0.3s
[CV 2/5] END xgb__max_depth=2, xgb__reg_alpha=5, xgb__reg_lambda=5;, score=(train=0.842, test=0.667) total time=   0.3s
[CV 3/5] END xgb__max_depth=2, xgb__re

[CV 2/5] END xgb__max_depth=4, xgb__reg_alpha=0.1, xgb__reg_lambda=5;, score=(train=1.000, test=0.685) total time=   0.4s
[CV 3/5] END xgb__max_depth=4, xgb__reg_alpha=0.1, xgb__reg_lambda=5;, score=(train=1.000, test=0.652) total time=   0.5s
[CV 4/5] END xgb__max_depth=4, xgb__reg_alpha=0.1, xgb__reg_lambda=5;, score=(train=1.000, test=0.748) total time=   0.5s
[CV 5/5] END xgb__max_depth=4, xgb__reg_alpha=0.1, xgb__reg_lambda=5;, score=(train=1.000, test=0.729) total time=   0.5s
[CV 1/5] END xgb__max_depth=4, xgb__reg_alpha=0.1, xgb__reg_lambda=10;, score=(train=1.000, test=0.522) total time=   0.5s
[CV 2/5] END xgb__max_depth=4, xgb__reg_alpha=0.1, xgb__reg_lambda=10;, score=(train=1.000, test=0.667) total time=   0.5s
[CV 3/5] END xgb__max_depth=4, xgb__reg_alpha=0.1, xgb__reg_lambda=10;, score=(train=1.000, test=0.637) total time=   0.4s
[CV 4/5] END xgb__max_depth=4, xgb__reg_alpha=0.1, xgb__reg_lambda=10;, score=(train=1.000, test=0.695) total time=   0.5s
[CV 5/5] END xgb__ma

[CV 4/5] END xgb__max_depth=4, xgb__reg_alpha=0.5, xgb__reg_lambda=10;, score=(train=1.000, test=0.689) total time=   0.5s
[CV 5/5] END xgb__max_depth=4, xgb__reg_alpha=0.5, xgb__reg_lambda=10;, score=(train=1.000, test=0.675) total time=   0.5s
[CV 1/5] END xgb__max_depth=4, xgb__reg_alpha=1, xgb__reg_lambda=0.1;, score=(train=1.000, test=0.529) total time=   0.5s
[CV 2/5] END xgb__max_depth=4, xgb__reg_alpha=1, xgb__reg_lambda=0.1;, score=(train=1.000, test=0.677) total time=   0.5s
[CV 3/5] END xgb__max_depth=4, xgb__reg_alpha=1, xgb__reg_lambda=0.1;, score=(train=1.000, test=0.665) total time=   0.5s
[CV 4/5] END xgb__max_depth=4, xgb__reg_alpha=1, xgb__reg_lambda=0.1;, score=(train=1.000, test=0.754) total time=   0.5s
[CV 5/5] END xgb__max_depth=4, xgb__reg_alpha=1, xgb__reg_lambda=0.1;, score=(train=1.000, test=0.684) total time=   0.5s
[CV 1/5] END xgb__max_depth=4, xgb__reg_alpha=1, xgb__reg_lambda=0.25;, score=(train=1.000, test=0.538) total time=   0.5s
[CV 2/5] END xgb__max

...

[CV 4/5] END xgb__max_depth=8, xgb__reg_alpha=5, xgb__reg_lambda=0.5;, score=(train=0.996, test=0.695) total time=   0.5s
[CV 5/5] END xgb__max_depth=8, xgb__reg_alpha=5, xgb__reg_lambda=0.5;, score=(train=1.000, test=0.669) total time=   0.7s
[CV 1/5] END xgb__max_depth=8, xgb__reg_alpha=5, xgb__reg_lambda=1;, score=(train=0.996, test=0.529) total time=   0.7s
[CV 2/5] END xgb__max_depth=8, xgb__reg_alpha=5, xgb__reg_lambda=1;, score=(train=0.995, test=0.646) total time=   0.6s
[CV 3/5] END xgb__max_depth=8, xgb__reg_alpha=5, xgb__reg_lambda=1;, score=(train=0.987, test=0.603) total time=   0.5s
[CV 4/5] END xgb__max_depth=8, xgb__reg_alpha=5, xgb__reg_lambda=1;, score=(train=1.000, test=0.730) total time=   0.5s
[CV 5/5] END xgb__max_depth=8, xgb__reg_alpha=5, xgb__reg_lambda=1;, score=(train=1.000, test=0.656) total time=   0.7s
[CV 1/5] END xgb__max_depth=8, xgb__reg_alpha=5, xgb__reg_lambda=5;, score=(train=0.987, test=0.544) total time=   0.7s
[CV 2/5] END xgb__max_depth=8, xgb__

[CV 1/5] END xgb__max_depth=10, xgb__reg_alpha=0.1, xgb__reg_lambda=5;, score=(train=1.000, test=0.520) total time=   0.8s
[CV 2/5] END xgb__max_depth=10, xgb__reg_alpha=0.1, xgb__reg_lambda=5;, score=(train=1.000, test=0.703) total time=   0.8s
[CV 3/5] END xgb__max_depth=10, xgb__reg_alpha=0.1, xgb__reg_lambda=5;, score=(train=1.000, test=0.645) total time=   0.7s
[CV 4/5] END xgb__max_depth=10, xgb__reg_alpha=0.1, xgb__reg_lambda=5;, score=(train=1.000, test=0.724) total time=   0.7s
[CV 5/5] END xgb__max_depth=10, xgb__reg_alpha=0.1, xgb__reg_lambda=5;, score=(train=1.000, test=0.669) total time=   0.7s
[CV 1/5] END xgb__max_depth=10, xgb__reg_alpha=0.1, xgb__reg_lambda=10;, score=(train=1.000, test=0.529) total time=   0.8s
[CV 2/5] END xgb__max_depth=10, xgb__reg_alpha=0.1, xgb__reg_lambda=10;, score=(train=1.000, test=0.685) total time=   0.8s
[CV 3/5] END xgb__max_depth=10, xgb__reg_alpha=0.1, xgb__reg_lambda=10;, score=(train=1.000, test=0.658) total time=   0.8s
[CV 4/5] END 

[CV 2/5] END xgb__max_depth=10, xgb__reg_alpha=0.5, xgb__reg_lambda=10;, score=(train=1.000, test=0.672) total time=   0.7s
[CV 3/5] END xgb__max_depth=10, xgb__reg_alpha=0.5, xgb__reg_lambda=10;, score=(train=1.000, test=0.665) total time=   0.8s
[CV 4/5] END xgb__max_depth=10, xgb__reg_alpha=0.5, xgb__reg_lambda=10;, score=(train=1.000, test=0.695) total time=   0.9s
[CV 5/5] END xgb__max_depth=10, xgb__reg_alpha=0.5, xgb__reg_lambda=10;, score=(train=1.000, test=0.669) total time=   0.9s
[CV 1/5] END xgb__max_depth=10, xgb__reg_alpha=1, xgb__reg_lambda=0.1;, score=(train=1.000, test=0.500) total time=   0.7s
[CV 2/5] END xgb__max_depth=10, xgb__reg_alpha=1, xgb__reg_lambda=0.1;, score=(train=1.000, test=0.665) total time=   0.6s
[CV 3/5] END xgb__max_depth=10, xgb__reg_alpha=1, xgb__reg_lambda=0.1;, score=(train=1.000, test=0.665) total time=   0.6s
[CV 4/5] END xgb__max_depth=10, xgb__reg_alpha=1, xgb__reg_lambda=0.1;, score=(train=1.000, test=0.696) total time=   0.6s
[CV 5/5] END

[CV 4/5] END xgb__max_depth=10, xgb__reg_alpha=10, xgb__reg_lambda=0.1;, score=(train=0.966, test=0.695) total time=   0.4s
[CV 5/5] END xgb__max_depth=10, xgb__reg_alpha=10, xgb__reg_lambda=0.1;, score=(train=0.972, test=0.661) total time=   0.5s
[CV 1/5] END xgb__max_depth=10, xgb__reg_alpha=10, xgb__reg_lambda=0.25;, score=(train=0.970, test=0.571) total time=   0.4s
[CV 2/5] END xgb__max_depth=10, xgb__reg_alpha=10, xgb__reg_lambda=0.25;, score=(train=0.923, test=0.654) total time=   0.5s
[CV 3/5] END xgb__max_depth=10, xgb__reg_alpha=10, xgb__reg_lambda=0.25;, score=(train=0.938, test=0.597) total time=   0.4s
[CV 4/5] END xgb__max_depth=10, xgb__reg_alpha=10, xgb__reg_lambda=0.25;, score=(train=0.966, test=0.630) total time=   0.4s
[CV 5/5] END xgb__max_depth=10, xgb__reg_alpha=10, xgb__reg_lambda=0.25;, score=(train=0.972, test=0.643) total time=   0.6s
[CV 1/5] END xgb__max_depth=10, xgb__reg_alpha=10, xgb__reg_lambda=0.5;, score=(train=0.970, test=0.565) total time=   0.5s
[CV

{'xgb__max_depth': 4, 'xgb__reg_alpha': 0.1, 'xgb__reg_lambda': 0.25}

### 5-fold CV (run 5)

In [14]:
xgb_scores3 = cross_validate(xgb_pipeline3, cdata5, pheno["label"], cv=5, scoring=["accuracy",
                                                                                 "roc_auc",
                                                                                 "f1",
                                                                                 "balanced_accuracy"])
xgb_scores3

{'fit_time': array([0.60089207, 0.42127395, 0.45918298, 0.43068504, 0.41497588]),
 'score_time': array([0.02077818, 0.01892519, 0.01919532, 0.01856112, 0.01842523]),
 'test_accuracy': array([0.77358491, 0.82075472, 0.81132075, 0.78095238, 0.74285714]),
 'test_roc_auc': array([0.775     , 0.79868421, 0.83859649, 0.78622222, 0.75022222]),
 'test_f1': array([0.85542169, 0.88484848, 0.87804878, 0.85714286, 0.83435583]),
 'test_balanced_accuracy': array([0.6504386 , 0.71359649, 0.70701754, 0.67666667, 0.62      ])}

In [15]:
print("ACC")
print("mean =", np.mean(xgb_scores3["test_accuracy"]))
print("std =", np.std(xgb_scores3["test_accuracy"]), "\n")
print("AUC")
print("mean =", np.mean(xgb_scores3["test_roc_auc"]))
print("std =", np.std(xgb_scores3["test_roc_auc"]), "\n")
print("F1")
print("mean =", np.mean(xgb_scores3["test_f1"]))
print("std =", np.std(xgb_scores3["test_f1"]), "\n")
print("BAS")
print("mean =", np.mean(xgb_scores3["test_balanced_accuracy"]))
print("std =", np.std(xgb_scores3["test_balanced_accuracy"]), "\n")

ACC
mean = 0.7858939802336029
std = 0.027892271040137178 

AUC
mean = 0.7897450292397662
std = 0.02918305203908233 

F1
mean = 0.8619635274893988
std = 0.017948126582689027 

BAS
mean = 0.6735438596491228
std = 0.035029275888645704 



# 6. Neural Network (AKA Multilayer Perceptron)

In [5]:
std_scaler = StandardScaler()
mm_scaler = MinMaxScaler()
# Numpy array to Pandas DF: https://datatofish.com/numpy-array-to-pandas-dataframe/
# Get column names: https://www.geeksforgeeks.org/how-to-get-column-names-in-pandas-dataframe/
# Create a list via list comprehension: https://realpython.com/list-comprehension-python/

In [6]:
# https://machinelearningmastery.com/standardscaler-and-minmaxscaler-transforms-in-python/

# transform data
# Genotype data was Standard Scaled, then no PCA
mlp_geno = std_scaler.fit_transform(geno)
# type(geno) # pandas.core.frame.DataFrame
# type(svm_geno) # numpy.ndarray
# svm_geno.shape
mlp_geno_df = pd.DataFrame(mlp_geno, columns = list(geno.columns))
mlp_geno_df

,chr1:5284719:T:C,chr1:5286414:T:G,chr1:29385283:T:C,chr1:29390419:G:A,chr1:29390546:C:T,chr1:30051076:G:A,chr1:43183837:A:G,chr1:49120707:A:G,chr1:49127355:G:C,chr1:49199968:G:A,...,chrX:114429288:T:C,chrX:114438661:T:G,chrX:114441059:A:G,chrX:114444757:C:T,chrX:114450759:T:C,chrX:114453552:A:G,chrX:114457175:G:A,chrX:114463117:A:G,chrX:114468124:C:T,chrX:114470502:G:A
0,-1.245361,-1.254817,-0.133036,-0.081309,-0.108304,-0.094088,0.082749,1.295324,1.295175,1.219800,...,-1.591289,-1.657622,-1.657982,-1.657618,-1.657561,-1.657586,-1.657462,-1.656306,-1.656939,-1.661499
1,-1.245361,-1.254817,1.217941,1.219677,1.240570,1.285410,-1.277131,-0.048080,-0.048117,-0.105407,...,0.868029,0.840351,0.840516,0.832378,0.832339,0.832615,0.832642,0.835743,0.835740,0.837455
2,-1.245361,-1.254817,-0.078356,-0.081309,-0.043505,-0.094088,-1.277131,1.299355,1.299204,1.219800,...,-1.591289,-1.657622,-1.657982,-1.657618,-1.657561,-1.657586,-1.657462,-1.656306,-1.656939,-1.661499
3,1.479705,1.475851,-1.410661,-1.382296,-1.364608,-0.094088,1.442629,-1.386111,-1.386035,-1.430614,...,0.880387,0.852904,0.853071,0.852459,0.852419,0.852698,0.852724,0.855840,0.855843,0.857608
4,0.117854,0.110517,-0.109030,-0.081309,-0.073921,-0.089950,0.084109,-0.089726,-0.089759,-0.105407,...,-1.591289,-1.657622,-1.657982,-1.657618,-1.657561,-1.657586,-1.657462,-1.656306,-1.656939,-1.661499
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
523,-1.245361,-1.254817,-1.410661,-1.382296,-1.364608,-0.094088,-1.274411,-0.060171,-0.060206,-0.105407,...,-0.355451,-0.402359,-0.402456,-0.402580,-0.402571,-0.402444,-0.402369,-0.400233,-0.400548,-0.401946
524,0.115127,0.110517,-0.062353,-0.077406,-0.042182,-0.094088,-1.271691,-1.386111,-1.386035,-1.430614,...,0.880387,0.852904,0.853071,0.852459,0.852419,0.852698,0.852724,0.855840,0.855843,0.857608
525,1.481068,1.475851,1.256616,1.219677,1.280243,-0.094088,0.082749,-0.042707,-0.042744,-0.105407,...,-1.591289,-1.657622,-1.657982,-1.657618,-1.657561,-1.657586,-1.657462,-1.656306,-1.656939,-1.661499
526,0.117854,0.110517,-0.077023,-0.081309,-0.042182,-0.094088,-1.277131,-0.061514,-0.062893,-0.105407,...,0.880387,0.852904,0.853071,0.852459,0.852419,0.852698,0.852724,0.855840,0.855843,0.857608


In [7]:
# https://machinelearningmastery.com/standardscaler-and-minmaxscaler-transforms-in-python/

# transform data
# CCI data was MinMax Scaled, then no PCA
mlp_cci = mm_scaler.fit_transform(cci)
# type(geno) # pandas.core.frame.DataFrame
# type(svm_geno) # numpy.ndarray
# svm_geno.shape
mlp_cci_df = pd.DataFrame(mlp_cci, columns = list(cci.columns))
mlp_cci_df

,Astro_to_Astro,Astro_to_EN_L2_3_IT,Astro_to_EN_L3_5_IT_1,Astro_to_EN_L3_5_IT_2,Astro_to_EN_L3_5_IT_3,Astro_to_EN_L5_6_NP,Astro_to_EN_L6B,Astro_to_EN_L6_CT,Astro_to_EN_L6_IT_1,Astro_to_EN_L6_IT_2,...,PVM_to_EN_NF,VLMC_to_EN_NF,EN_L3_5_IT_1_to_EN_NF,EN_L5_ET_to_EN_NF,EN_L6B_to_EN_NF,EN_NF_to_EN_L3_5_IT_1,EN_NF_to_EN_L5_ET,EN_NF_to_EN_L6B,EN_NF_to_SMC,SMC_to_EN_NF
0,0.442536,0.353477,0.481003,0.400985,0.450695,0.434030,0.370786,0.413909,0.448826,0.555685,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.328878,0.274532,0.344674,0.304059,0.302334,0.243900,0.251157,0.290790,0.246103,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.546797,0.283868,0.000000,0.000000,0.409058,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.428541,0.365799,0.384439,0.400338,0.406836,0.000000,0.276461,0.346104,0.000000,0.265997,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.492905,0.592974,0.000000,0.556944,0.705731,0.000000,0.000000,0.479295,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
523,0.349413,0.358951,0.000000,0.343262,0.373900,0.000000,0.000000,0.000000,0.000000,0.358171,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
524,0.535685,0.500754,0.000000,0.445818,0.564546,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
525,0.363129,0.341103,0.357748,0.329670,0.385470,0.342754,0.385469,0.310340,0.360632,0.342676,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
526,0.359605,0.404317,0.427809,0.431414,0.488376,0.394541,0.400131,0.345804,0.403136,0.421706,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Put the combined data together (run 6)

In [8]:
cdata6 = pd.concat([mlp_geno_df, mlp_cci_df], axis=1)
seed = 0
X_train, X_test, y_train, y_test = train_test_split(cdata6, pheno, test_size=0.20, random_state=seed)
cdata6.shape

(528, 7098)

In [80]:
mlp_pipeline3 = Pipeline(steps = [("mlp", MLPClassifier(max_iter=1000,
                                                       alpha=0.5,
                                                       hidden_layer_sizes=(400,)))])

In [78]:
grid_search_mlp3 = GridSearchCV(estimator=mlp_pipeline3,
                                param_grid={"mlp__hidden_layer_sizes": [(100,), (200,), (400,), (800,)],
                                            "mlp__alpha": [0.01, 0.05, 0.1, 0.25, 0.5, 1, 5, 10, 20, 50, 100]},
                                scoring="balanced_accuracy", # scoring can be "accuracy", "roc_auc", "f1"
                                return_train_score=True, # or else you'll never see it!
                                cv=KFold(n_splits=5, shuffle=True, random_state=0),
                                verbose = 3) # to get a consistent set of settings)

grid_search_mlp3.fit(cdata6, pheno["label"])
grid_search_mlp3.best_params_


Fitting 5 folds for each of 44 candidates, totalling 220 fits
[CV 1/5] END mlp__alpha=0.01, mlp__hidden_layer_sizes=(100,);, score=(train=1.000, test=0.671) total time=   4.6s
[CV 2/5] END mlp__alpha=0.01, mlp__hidden_layer_sizes=(100,);, score=(train=1.000, test=0.704) total time=   4.2s
[CV 3/5] END mlp__alpha=0.01, mlp__hidden_layer_sizes=(100,);, score=(train=1.000, test=0.711) total time=   4.3s
[CV 4/5] END mlp__alpha=0.01, mlp__hidden_layer_sizes=(100,);, score=(train=1.000, test=0.686) total time=   4.5s
[CV 5/5] END mlp__alpha=0.01, mlp__hidden_layer_sizes=(100,);, score=(train=1.000, test=0.771) total time=   3.8s
[CV 1/5] END mlp__alpha=0.01, mlp__hidden_layer_sizes=(200,);, score=(train=1.000, test=0.598) total time=   6.0s
[CV 2/5] END mlp__alpha=0.01, mlp__hidden_layer_sizes=(200,);, score=(train=1.000, test=0.735) total time=   5.8s
[CV 3/5] END mlp__alpha=0.01, mlp__hidden_layer_sizes=(200,);, score=(train=1.000, test=0.691) total time=  11.9s
[CV 4/5] END mlp__alpha=0.

[CV 3/5] END mlp__alpha=0.25, mlp__hidden_layer_sizes=(400,);, score=(train=1.000, test=0.672) total time=  45.9s
[CV 4/5] END mlp__alpha=0.25, mlp__hidden_layer_sizes=(400,);, score=(train=0.984, test=0.609) total time=  32.1s
[CV 5/5] END mlp__alpha=0.25, mlp__hidden_layer_sizes=(400,);, score=(train=0.962, test=0.732) total time=  40.5s
[CV 1/5] END mlp__alpha=0.25, mlp__hidden_layer_sizes=(800,);, score=(train=0.846, test=0.625) total time= 1.0min
[CV 2/5] END mlp__alpha=0.25, mlp__hidden_layer_sizes=(800,);, score=(train=1.000, test=0.714) total time=  36.8s
[CV 3/5] END mlp__alpha=0.25, mlp__hidden_layer_sizes=(800,);, score=(train=1.000, test=0.684) total time= 1.0min
[CV 4/5] END mlp__alpha=0.25, mlp__hidden_layer_sizes=(800,);, score=(train=1.000, test=0.645) total time= 1.1min
[CV 5/5] END mlp__alpha=0.25, mlp__hidden_layer_sizes=(800,);, score=(train=0.983, test=0.725) total time=  50.8s
[CV 1/5] END mlp__alpha=0.5, mlp__hidden_layer_sizes=(100,);, score=(train=1.000, test=0

[CV 2/5] END mlp__alpha=10, mlp__hidden_layer_sizes=(200,);, score=(train=0.992, test=0.697) total time=   9.8s
[CV 3/5] END mlp__alpha=10, mlp__hidden_layer_sizes=(200,);, score=(train=0.960, test=0.695) total time=   7.9s
[CV 4/5] END mlp__alpha=10, mlp__hidden_layer_sizes=(200,);, score=(train=0.936, test=0.596) total time=   4.7s
[CV 5/5] END mlp__alpha=10, mlp__hidden_layer_sizes=(200,);, score=(train=0.990, test=0.705) total time=   5.8s
[CV 1/5] END mlp__alpha=10, mlp__hidden_layer_sizes=(400,);, score=(train=0.985, test=0.703) total time=  10.9s
[CV 2/5] END mlp__alpha=10, mlp__hidden_layer_sizes=(400,);, score=(train=0.953, test=0.755) total time=  10.2s
[CV 3/5] END mlp__alpha=10, mlp__hidden_layer_sizes=(400,);, score=(train=0.960, test=0.591) total time=  19.8s
[CV 4/5] END mlp__alpha=10, mlp__hidden_layer_sizes=(400,);, score=(train=0.974, test=0.679) total time=   9.0s
[CV 5/5] END mlp__alpha=10, mlp__hidden_layer_sizes=(400,);, score=(train=0.961, test=0.732) total time=

[CV 5/5] END mlp__alpha=100, mlp__hidden_layer_sizes=(800,);, score=(train=0.581, test=0.507) total time=  22.9s


{'mlp__alpha': 0.5, 'mlp__hidden_layer_sizes': (400,)}

### 5-fold CV (run 6)

In [83]:
mlp_scores3 = cross_validate(mlp_pipeline3, cdata6, pheno["label"], cv=5, scoring=["accuracy",
                                                                                 "roc_auc",
                                                                                 "f1",
                                                                                 "balanced_accuracy"])
mlp_scores3

{'fit_time': array([25.4642899 , 30.22904086, 29.92298222, 32.20307016, 39.5938642 ]),
 'score_time': array([0.12431502, 0.09215784, 0.06412077, 0.07675695, 0.09881091]),
 'test_accuracy': array([0.69811321, 0.73584906, 0.73584906, 0.78095238, 0.76190476]),
 'test_roc_auc': array([0.69649123, 0.72017544, 0.85      , 0.83288889, 0.77022222]),
 'test_f1': array([0.79220779, 0.82278481, 0.79411765, 0.85350318, 0.83443709]),
 'test_balanced_accuracy': array([0.61798246, 0.64429825, 0.75526316, 0.69666667, 0.70333333])}

In [84]:
print("ACC")
print("mean =", np.mean(mlp_scores3["test_accuracy"]))
print("std =", np.std(mlp_scores3["test_accuracy"]), "\n")
print("AUC")
print("mean =", np.mean(mlp_scores3["test_roc_auc"]))
print("std =", np.std(mlp_scores3["test_roc_auc"]), "\n")
print("F1")
print("mean =", np.mean(mlp_scores3["test_f1"]))
print("std =", np.std(mlp_scores3["test_f1"]), "\n")
print("BAS")
print("mean =", np.mean(mlp_scores3["test_balanced_accuracy"]))
print("std =", np.std(mlp_scores3["test_balanced_accuracy"]), "\n")

ACC
mean = 0.7425336927223719
std = 0.027977784579412527 

AUC
mean = 0.7739555555555555
std = 0.06027081565619068 

F1
mean = 0.8194101040398577
std = 0.023576306452648944 

BAS
mean = 0.6835087719298245
std = 0.048054133576315695 

